### Basic neural activity analysis with single camera tracking
#### analyze the firing rate PC1,2,3
#### making the demo videos
#### analyze the spike triggered pull and gaze ditribution
#### the following detailed analysis focused on pull related behavioral events

In [ ]:
import pandas as pd
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import seaborn
import scipy
import scipy.stats as st
import scipy.io
from sklearn.neighbors import KernelDensity
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from dPCA import dPCA
import string
import warnings
import pickle
import json

from scipy.ndimage import gaussian_filter1d

import os
import glob
import random
from time import time

from pgmpy.models import BayesianModel
from pgmpy.models import DynamicBayesianNetwork as DBN
from pgmpy.estimators import BayesianEstimator
from pgmpy.estimators import HillClimbSearch,BicScore
from pgmpy.base import DAG
import networkx as nx

from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multitest import multipletests

from scipy.ndimage import label


### function - get body part location for each pair of cameras

In [ ]:
from ana_functions.body_part_locs_eachpair import body_part_locs_eachpair
from ana_functions.body_part_locs_singlecam import body_part_locs_singlecam

### function - align the two cameras

In [ ]:
from ana_functions.camera_align import camera_align       

### function - merge the two pairs of cameras

In [ ]:
from ana_functions.camera_merge import camera_merge

### function - find social gaze time point

In [ ]:
from ana_functions.find_socialgaze_timepoint import find_socialgaze_timepoint
from ana_functions.find_socialgaze_timepoint_singlecam import find_socialgaze_timepoint_singlecam
from ana_functions.find_socialgaze_timepoint_singlecam_wholebody import find_socialgaze_timepoint_singlecam_wholebody
from ana_functions.find_socialgaze_timepoint_singlecam_wholebody_2 import find_socialgaze_timepoint_singlecam_wholebody_2


### function - define time point of behavioral events

In [ ]:
from ana_functions.bhv_events_timepoint import bhv_events_timepoint
from ana_functions.bhv_events_timepoint_singlecam import bhv_events_timepoint_singlecam

### function - plot behavioral events

In [ ]:
from ana_functions.plot_bhv_events import plot_bhv_events
from ana_functions.plot_bhv_events_levertube import plot_bhv_events_levertube
from ana_functions.plot_continuous_bhv_var_singlecam import plot_continuous_bhv_var_singlecam
from ana_functions.draw_self_loop import draw_self_loop
import matplotlib.patches as mpatches 
from matplotlib.collections import PatchCollection

### function - plot inter-pull interval

In [ ]:
from ana_functions.plot_interpull_interval import plot_interpull_interval

### function - make demo videos with skeleton and inportant vectors

In [ ]:
from ana_functions.tracking_video_singlecam_demo import tracking_video_singlecam_demo
from ana_functions.tracking_video_singlecam_wholebody_demo import tracking_video_singlecam_wholebody_demo
from ana_functions.tracking_video_singlecam_wholebody_withNeuron_demo import tracking_video_singlecam_wholebody_withNeuron_demo
from ana_functions.tracking_video_singlecam_wholebody_withNeuron_sepbhv_demo import tracking_video_singlecam_wholebody_withNeuron_sepbhv_demo
from ana_functions.tracking_frame_singlecam_wholebody_withNeuron_sepbhv_demo import tracking_frame_singlecam_wholebody_withNeuron_sepbhv_demo

### function - interval between all behavioral events

In [ ]:
from ana_functions.bhv_events_interval import bhv_events_interval

### function - spike analysis

In [ ]:
from ana_functions.spike_analysis_FR_calculation import spike_analysis_FR_calculation
from ana_functions.plot_spike_triggered_singlecam_bhvevent import plot_spike_triggered_singlecam_bhvevent
from ana_functions.plot_bhv_events_aligned_FR import plot_bhv_events_aligned_FR
from ana_functions.plot_strategy_aligned_FR import plot_strategy_aligned_FR

### function - PCA projection

In [ ]:
from ana_functions.PCA_around_bhv_events import PCA_around_bhv_events
from ana_functions.PCA_around_bhv_events_video import PCA_around_bhv_events_video
from ana_functions.confidence_ellipse import confidence_ellipse

### function - train the dynamic bayesian network - multi time lag (3 lags)

In [ ]:
from ana_functions.train_DBN_multiLag_withNeuron import train_DBN_multiLag
from ana_functions.train_DBN_multiLag_withNeuron import train_DBN_multiLag_create_df_only
from ana_functions.train_DBN_multiLag_withNeuron import train_DBN_multiLag_training_only
from ana_functions.train_DBN_multiLag_withNeuron import graph_to_matrix
from ana_functions.train_DBN_multiLag_withNeuron import get_weighted_dags
from ana_functions.train_DBN_multiLag_withNeuron import get_significant_edges
from ana_functions.train_DBN_multiLag_withNeuron import threshold_edges
from ana_functions.train_DBN_multiLag_withNeuron import Modulation_Index
from ana_functions.EfficientTimeShuffling import EfficientShuffle
from ana_functions.AicScore import AicScore

### function - other useful functions

In [ ]:
# for defining the meaningful social gaze (the continuous gaze distribution that is closest to the pull) 
from ana_functions.keep_closest_cluster_single_trial import keep_closest_cluster_single_trial

In [ ]:
# get useful information about pulls
from ana_functions.get_pull_infos import get_pull_infos

In [ ]:
# use the gaze vector speed and face mass speed to find the pull action start time within IPI
from ana_functions.find_sharp_increases_withinIPI import find_sharp_increases_withinIPI
from ana_functions.find_sharp_increases_withinIPI import find_sharp_increases_withinIPI_dual_speed

## Analyze each session

### prepare the basic behavioral data (especially the time stamps for each bhv events)

In [ ]:
# instead of using gaze angle threshold, use the target rectagon to deside gaze info
# ...need to update
sqr_thres_tubelever = 75 # draw the square around tube and lever
sqr_thres_face = 1.15 # a ratio for defining face boundary
sqr_thres_body = 4 # how many times to enlongate the face box boundry to the body


# get the fps of the analyzed video
fps = 30

# get the fs for neural recording
fs_spikes = 20000
fs_lfp = 1000

# frame number of the demo video
nframes = 0.5*30 # second*30fps
# nframes = 45*30 # second*30fps

# re-analyze the video or not
reanalyze_video = 0
redo_anystep = 0

# if use onset of the first increase after min
doOnsetAfterMin = 0
if not doOnsetAfterMin:
    doOnsetAfterMin_suffix = ''
elif doOnsetAfterMin:
    doOnsetAfterMin_suffix = 'PullOnsetAfterMin_'

# do OFC sessions or DLPFC sessions
do_OFC = 0
do_DLPFC  = 1
if do_OFC:
    savefile_sufix = '_OFCs'
elif do_DLPFC:
    savefile_sufix = '_DLPFCs'
else:
    savefile_sufix = ''
    
# all the videos (no misaligned ones)
# aligned with the audio
# get the session start time from "videosound_bhv_sync.py/.ipynb"
# currently the session_start_time will be manually typed in. It can be updated after a better method is used


# dodson ginger
if 1:
    if do_DLPFC:
        neural_record_conditions = [
                                    '20240531_Dodson_MC',
                                    '20240603_Dodson_MC_and_SR',
                                    '20240603_Dodson_MC_and_SR',
                                    '20240604_Dodson_MC',
                                    '20240605_Dodson_MC_and_SR',
                                    '20240605_Dodson_MC_and_SR',
                                    '20240606_Dodson_MC_and_SR',
                                    '20240606_Dodson_MC_and_SR',
                                    '20240607_Dodson_SR',
                                    '20240610_Dodson_MC',
                                    '20240611_Dodson_SR',
                                    '20240612_Dodson_MC',
                                    '20240613_Dodson_SR',
                                    '20240620_Dodson_SR',
                                    '20240719_Dodson_MC',
                                        
                                    '20250129_Dodson_MC',
                                    '20250130_Dodson_SR',
                                    '20250131_Dodson_MC',
                                
            
                                    '20250210_Dodson_SR_withKoala',
                                    '20250211_Dodson_MC_withKoala',
                                    '20250212_Dodson_SR_withKoala',
                                    '20250214_Dodson_MC_withKoala',
                                    '20250217_Dodson_SR_withKoala',
                                    '20250218_Dodson_MC_withKoala',
                                    '20250219_Dodson_SR_withKoala',
                                    '20250220_Dodson_MC_withKoala',
                                    '20250224_Dodson_KoalaAL_withKoala',
                                    '20250226_Dodson_MC_withKoala',
                                    '20250227_Dodson_KoalaAL_withKoala',
                                    '20250228_Dodson_DodsonAL_withKoala',
                                    '20250304_Dodson_DodsonAL_withKoala',
                                    '20250305_Dodson_MC_withKoala',
                                    '20250306_Dodson_KoalaAL_withKoala',
                                    '20250307_Dodson_DodsonAL_withKoala',
                                    '20250310_Dodson_MC_withKoala',
                                    '20250312_Dodson_NV_withKoala',
                                    '20250313_Dodson_NV_withKoala',
                                    '20250314_Dodson_NV_withKoala',
            
                                    '20250401_Dodson_MC_withKanga',
                                    '20250402_Dodson_MC_withKanga',
                                    '20250403_Dodson_MC_withKanga',
                                    '20250404_Dodson_SR_withKanga',
                                    '20250407_Dodson_SR_withKanga',
                                    '20250408_Dodson_SR_withKanga',
                                    '20250409_Dodson_MC_withKanga',
            
                                    '20250415_Dodson_MC_withKanga',
                                    # '20250416_Dodson_SR_withKanga', # has to remove from the later analysis, recording has problems
                                    '20250417_Dodson_MC_withKanga',
                                    '20250418_Dodson_SR_withKanga',
                                    '20250421_Dodson_SR_withKanga',
                                    '20250422_Dodson_MC_withKanga',
                                    '20250422_Dodson_SR_withKanga',
            
                                    '20250423_Dodson_MC_withKanga',
                                    '20250423_Dodson_SR_withKanga', 
                                    '20250424_Dodson_NV_withKanga',
                                    '20250424_Dodson_MC_withKanga',
                                    '20250424_Dodson_SR_withKanga',            
                                    '20250425_Dodson_NV_withKanga',
                                    '20250425_Dodson_SR_withKanga',
                                    '20250428_Dodson_NV_withKanga',
                                    '20250428_Dodson_MC_withKanga',
                                    '20250428_Dodson_SR_withKanga',  
                                    '20250429_Dodson_NV_withKanga',
                                    '20250429_Dodson_MC_withKanga',
                                    '20250429_Dodson_SR_withKanga',  
                                    '20250430_Dodson_NV_withKanga',
                                    '20250430_Dodson_MC_withKanga',
                                    '20250430_Dodson_SR_withKanga',  
            
                                   ]
        task_conditions = [
                            'MC',           
                            'MC',
                            'SR',
                            'MC',
                            'MC',
                            'SR',
                            'MC',
                            'SR',
                            'SR',
                            'MC',
                            'SR',
                            'MC',
                            'SR',
                            'SR',
                            'MC',
                            
                            'MC_withGingerNew',
                            'SR_withGingerNew',
                            'MC_withGingerNew',
            
                            'SR_withKoala',
                            'MC_withKoala',
                            'SR_withKoala',
                            'MC_withKoala',
                            'SR_withKoala',
                            'MC_withKoala',
                            'SR_withKoala',
                            'MC_withKoala',
                            'MC_KoalaAuto_withKoala',
                            'MC_withKoala',
                            'MC_KoalaAuto_withKoala',
                            'MC_DodsonAuto_withKoala',
                            'MC_DodsonAuto_withKoala',
                            'MC_withKoala',
                            'MC_KoalaAuto_withKoala',
                            'MC_DodsonAuto_withKoala',
                            'MC_withKoala',
                            'NV_withKoala',
                            'NV_withKoala',
                            'NV_withKoala',

                            'MC_withKanga',
                            'MC_withKanga',
                            'MC_withKanga',
                            'SR_withKanga',
                            'SR_withKanga',
                            'SR_withKanga',
                            'MC_withKanga',
            
                            'MC_withKanga',
                            # 'SR_withKanga',
                            'MC_withKanga',
                            'SR_withKanga',
                            'SR_withKanga',
                            'MC_withKanga',
                            'SR_withKanga',
            
                            'MC_withKanga',
                            'SR_withKanga', 
                            'NV_withKanga',
                            'MC_withKanga',
                            'SR_withKanga',            
                            'NV_withKanga',
                            'SR_withKanga',
                            'NV_withKanga',
                            'MC_withKanga',
                            'SR_withKanga',  
                            'NV_withKanga',
                            'MC_withKanga',
                            'SR_withKanga',  
                            'NV_withKanga',
                            'MC_withKanga',
                            'SR_withKanga',  
                          ]
        dates_list = [
                        '20240531',
                        '20240603_MC',
                        '20240603_SR',
                        '20240604',
                        '20240605_MC',
                        '20240605_SR',
                        '20240606_MC',
                        '20240606_SR',
                        '20240607',
                        '20240610_MC',
                        '20240611',
                        '20240612',
                        '20240613',
                        '20240620',
                        '20240719',
            
                        '20250129',
                        '20250130',
                        '20250131',
            
                        '20250210',
                        '20250211',
                        '20250212',
                        '20250214',
                        '20250217',
                        '20250218',
                        '20250219',
                        '20250220',
                        '20250224',
                        '20250226',
                        '20250227',
                        '20250228',
                        '20250304',
                        '20250305',
                        '20250306',
                        '20250307',
                        '20250310',
                        '20250312',
                        '20250313',
                        '20250314',
            
                        '20250401',
                        '20250402',
                        '20250403',
                        '20250404',
                        '20250407',
                        '20250408',
                        '20250409',
            
                        '20250415',
                        # '20250416',
                        '20250417',
                        '20250418',
                        '20250421',
                        '20250422',
                        '20250422_SR',
            
                        '20250423',
                        '20250423_SR', 
                        '20250424',
                        '20250424_MC',
                        '20250424_SR',            
                        '20250425',
                        '20250425_SR',
                        '20250428_NV',
                        '20250428_MC',
                        '20250428_SR',  
                        '20250429_NV',
                        '20250429_MC',
                        '20250429_SR',  
                        '20250430_NV',
                        '20250430_MC',
                        '20250430_SR',  
                     ]
        videodates_list = [
                            '20240531',
                            '20240603',
                            '20240603',
                            '20240604',
                            '20240605',
                            '20240605',
                            '20240606',
                            '20240606',
                            '20240607',
                            '20240610_MC',
                            '20240611',
                            '20240612',
                            '20240613',
                            '20240620',
                            '20240719',
            
                            '20250129',
                            '20250130',
                            '20250131',
                            
                            '20250210',
                            '20250211',
                            '20250212',
                            '20250214',
                            '20250217',
                            '20250218',          
                            '20250219',
                            '20250220',
                            '20250224',
                            '20250226',
                            '20250227',
                            '20250228',
                            '20250304',
                            '20250305',
                            '20250306',
                            '20250307',
                            '20250310',
                            '20250312',
                            '20250313',
                            '20250314',
            
                            '20250401',
                            '20250402',
                            '20250403',
                            '20250404',
                            '20250407',
                            '20250408',
                            '20250409',
            
                            '20250415',
                            # '20250416',
                            '20250417',
                            '20250418',
                            '20250421',
                            '20250422',
                            '20250422_SR',
            
                            '20250423',
                            '20250423_SR', 
                            '20250424',
                            '20250424_MC',
                            '20250424_SR',            
                            '20250425',
                            '20250425_SR',
                            '20250428_NV',
                            '20250428_MC',
                            '20250428_SR',  
                            '20250429_NV',
                            '20250429_MC',
                            '20250429_SR',  
                            '20250430_NV',
                            '20250430_MC',
                            '20250430_SR',  
            
                          ] # to deal with the sessions that MC and SR were in the same session
        session_start_times = [ 
                                0.00,
                                340,
                                340,
                                72.0,
                                60.1,
                                60.1,
                                82.2,
                                82.2,
                                35.8,
                                0.00,
                                29.2,
                                35.8,
                                62.5,
                                71.5,
                                54.4,
            
                                0.00,
                                0.00,
                                0.00,
            
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
            
                                0.00,
                                0.00,
                                73.5,
                                0.00,
                                76.1,
                                81.5,
                                0.00,
            
                                363,
                                # 0.00,
                                79.0,
                                162.6,
                                231.9,
                                109,
                                0.00,
            
                                0.00,
                                0.00, 
                                0.00,
                                0.00,
                                0.00,          
                                0.00,
                                93.0,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00, 
                                0.00,
                                274.4,
                                0.00,
            
                              ] # in second
        
        kilosortvers = list((np.ones(np.shape(dates_list))*4).astype(int))
        
        trig_channelnames = [ 'Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0',
                              'Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0',
                              'Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0',
                              'Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0',
                              'Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0',
                              'Dev1/ai0',# 'Dev1/ai0',
                              'Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai9','Dev1/ai9','Dev1/ai9','Dev1/ai9',
                              'Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0',
                              'Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0',
                             
                              ]
        animal1_fixedorders = ['dodson','dodson','dodson','dodson','dodson','dodson','dodson','dodson','dodson',
                               'dodson','dodson','dodson','dodson','dodson','dodson','dodson','dodson','dodson',
                               'dodson','dodson','dodson','dodson','dodson','dodson','dodson','dodson','dodson',
                               'dodson','dodson','dodson','dodson','dodson','dodson','dodson','dodson','dodson',
                               'dodson','dodson','dodson','dodson','dodson','dodson','dodson','dodson','dodson',
                               'dodson',# 'dodson',
                               'dodson','dodson','dodson','dodson','dodson','dodson','dodson',
                               'dodson','dodson','dodson','dodson','dodson','dodson','dodson','dodson','dodson',
                               'dodson','dodson','dodson','dodson','dodson',
                              ]
        recordedanimals = animal1_fixedorders 
        animal2_fixedorders = ['ginger','ginger','ginger','ginger','ginger','ginger','ginger','ginger','ginger',
                               'ginger','ginger','ginger','ginger','ginger','ginger','gingerNew','gingerNew','gingerNew',
                               'koala', 'koala', 'koala', 'koala', 'koala', 'koala', 'koala', 'koala', 'koala',
                               'koala', 'koala', 'koala', 'koala', 'koala', 'koala', 'koala', 'koala', 'koala',
                               'koala', 'koala', 'kanga', 'kanga', 'kanga', 'kanga', 'kanga', 'kanga', 'kanga',
                               'kanga', # 'kanga', 
                               'kanga', 'kanga', 'kanga', 'kanga', 'kanga', 'kanga', 'kanga',
                               'kanga', 'kanga', 'kanga', 'kanga', 'kanga', 'kanga', 'kanga', 'kanga', 'kanga',
                               'kanga', 'kanga', 'kanga', 'kanga', 'kanga',
                              ]

        animal1_filenames = ["Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson",
                             "Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson",
                             "Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson",
                             "Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson",
                             "Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson",
                             'Dodson',# 'Dodson',
                             'Dodson','Dodson','Dodson','Dodson','Dodson','Dodson','Dodson',
                             'Dodson','Dodson','Dodson','Dodson','Dodson','Dodson','Dodson','Dodson','Dodson',
                             'Dodson','Dodson','Dodson','Dodson','Dodson',
                            ]
        animal2_filenames = ["Ginger","Ginger","Ginger","Ginger","Ginger","Ginger","Ginger","Ginger","Ginger",
                             "Ginger","Ginger","Ginger","Ginger","Ginger","Ginger","Ginger","Ginger","Ginger",
                             "Koala", "Koala", "Koala", "Koala", "Koala", "Koala", "Koala", "Koala", "Koala",
                             "Koala", "Koala", "Koala", "Koala", "Koala", "Koala", "Koala", "Koala", "Koala",
                             "Koala", "Koala", "Kanga", "Kanga", "Kanga", "Kanga", "Kanga", "Kanga", "Kanga",
                             'Kanga', # 'Kanga', 
                             'Kanga', 'Kanga', 'Kanga', 'Kanga', 'Kanga', 'Kanga', 'Kanga',
                             'Kanga', 'Kanga', 'Kanga', 'Kanga', 'Kanga', 'Kanga', 'Kanga', 'Kanga', 'Kanga',
                             'Kanga', 'Kanga', 'Kanga', 'Kanga', 'Kanga',
                            ]
        
    elif do_OFC:
        # pick only five sessions for each conditions
        neural_record_conditions = [
                                     '20231101_Dodson_withGinger_MC',
                                     '20231107_Dodson_withGinger_MC',
                                     '20231122_Dodson_withGinger_MC',
                                     '20231129_Dodson_withGinger_MC',
                                     # '20231101_Dodson_withGinger_SR', # ginger didnot pull or only pulled once
                                     # '20231107_Dodson_withGinger_SR', # ginger didnot pull or only pulled once
                                     # '20231122_Dodson_withGinger_SR', # ginger didnot pull or only pulled once
                                     # '20231129_Dodson_withGinger_SR', # ginger didnot pull or only pulled once
                                   ]
        task_conditions = [
                            'MC',
                            'MC',
                            'MC',
                            'MC',
                            # 'SR',
                            # 'SR',
                            # 'SR',
                            # 'SR',
                          ]
        dates_list = [
                      "20231101_MC",
                      "20231107_MC",
                      "20231122_MC",
                      "20231129_MC",
                      # "20231101_SR",
                      # "20231107_SR",
                      # "20231122_SR",
                      # "20231129_SR",      
                     ]
        videodates_list = dates_list
        session_start_times = [ 
                                 0.00,   
                                 0.00,  
                                 0.00,  
                                 0.00, 
                                 # 0.00,   
                                 # 0.00,  
                                 # 0.00,  
                                 # 0.00, 
                              ] # in second
        kilosortvers = [ 
                         2, 
                         2, 
                         4, 
                         4,
                         # 2, 
                         # 2, 
                         # 4, 
                         # 4,
                       ]
    
        trig_channelnames = ['Dev1/ai0']*np.shape(dates_list)[0]
        animal1_fixedorders = ['dodson']*np.shape(dates_list)[0]
        recordedanimals = animal1_fixedorders
        animal2_fixedorders = ['ginger']*np.shape(dates_list)[0]

        animal1_filenames = ["Dodson"]*np.shape(dates_list)[0]
        animal2_filenames = ["Ginger"]*np.shape(dates_list)[0]

    
# dannon kanga
if 0:
    if do_DLPFC:
        neural_record_conditions = [
                                     '20240508_Kanga_SR',
                                     '20240509_Kanga_MC',
                                     '20240513_Kanga_MC',
                                     '20240514_Kanga_SR',
                                     '20240523_Kanga_MC',
                                     '20240524_Kanga_SR',
                                     '20240606_Kanga_MC',
                                     '20240613_Kanga_MC_DannonAuto',
                                     '20240614_Kanga_MC_DannonAuto',
                                     '20240617_Kanga_MC_DannonAuto',
                                     '20240618_Kanga_MC_KangaAuto',
                                     '20240619_Kanga_MC_KangaAuto',
                                     '20240620_Kanga_MC_KangaAuto',
                                     '20240621_1_Kanga_NoVis',
                                     '20240624_Kanga_NoVis',
                                     '20240626_Kanga_NoVis',
            
                                     '20240808_Kanga_MC_withGinger',
                                     '20240809_Kanga_MC_withGinger',
                                     '20240812_Kanga_MC_withGinger',
                                     '20240813_Kanga_MC_withKoala',
                                     '20240814_Kanga_MC_withKoala',
                                     '20240815_Kanga_MC_withKoala',
                                     '20240819_Kanga_MC_withVermelho',
                                     '20240821_Kanga_MC_withVermelho',
                                     '20240822_Kanga_MC_withVermelho',
            
                                     '20250415_Kanga_MC_withDodson',
                                     '20250416_Kanga_SR_withDodson',
                                     '20250417_Kanga_MC_withDodson',
                                     '20250418_Kanga_SR_withDodson',
                                     '20250421_Kanga_SR_withDodson',
                                     '20250422_Kanga_MC_withDodson',
                                     '20250422_Kanga_SR_withDodson',
            
                                    '20250423_Kanga_MC_withDodson',
                                    '20250423_Kanga_SR_withDodson', 
                                    '20250424_Kanga_NV_withDodson',
                                    '20250424_Kanga_MC_withDodson',
                                    '20250424_Kanga_SR_withDodson',            
                                    '20250425_Kanga_NV_withDodson',
                                    '20250425_Kanga_SR_withDodson',
                                    '20250428_Kanga_NV_withDodson',
                                    '20250428_Kanga_MC_withDodson',
                                    '20250428_Kanga_SR_withDodson',  
                                    '20250429_Kanga_NV_withDodson',
                                    '20250429_Kanga_MC_withDodson',
                                    '20250429_Kanga_SR_withDodson',  
                                    '20250430_Kanga_NV_withDodson',
                                    '20250430_Kanga_MC_withDodson',
                                    '20250430_Kanga_SR_withDodson',  
                                   ]
        dates_list = [
                      "20240508",
                      "20240509",
                      "20240513",
                      "20240514",
                      "20240523",
                      "20240524",
                      "20240606",
                      "20240613",
                      "20240614",
                      "20240617",
                      "20240618",
                      "20240619",
                      "20240620",
                      "20240621_1",
                      "20240624",
                      "20240626",
            
                      "20240808",
                      "20240809",
                      "20240812",
                      "20240813",
                      "20240814",
                      "20240815",
                      "20240819",
                      "20240821",
                      "20240822",
            
                      "20250415",
                      "20250416",
                      "20250417",
                      "20250418",
                      "20250421",
                      "20250422",
                      "20250422_SR",
            
                        '20250423',
                        '20250423_SR', 
                        '20250424',
                        '20250424_MC',
                        '20250424_SR',            
                        '20250425',
                        '20250425_SR',
                        '20250428_NV',
                        '20250428_MC',
                        '20250428_SR',  
                        '20250429_NV',
                        '20250429_MC',
                        '20250429_SR',  
                        '20250430_NV',
                        '20250430_MC',
                        '20250430_SR',  
                     ]
        videodates_list = dates_list
        task_conditions = [
                             'SR',
                             'MC',
                             'MC',
                             'SR',
                             'MC',
                             'SR',
                             'MC',
                             'MC_DannonAuto',
                             'MC_DannonAuto',
                             'MC_DannonAuto',
                             'MC_KangaAuto',
                             'MC_KangaAuto',
                             'MC_KangaAuto',
                             'NV',
                             'NV',
                             'NV',   
                            
                             'MC_withGinger',
                             'MC_withGinger',
                             'MC_withGinger',
                             'MC_withKoala',
                             'MC_withKoala',
                             'MC_withKoala',
                             'MC_withVermelho',
                             'MC_withVermelho',
                             'MC_withVermelho',
            
                             'MC_withDodson',
                             'SR_withDodson',
                             'MC_withDodson',
                             'SR_withDodson',
                             'SR_withDodson',
                             'MC_withDodson',
                             'SR_withDodson',
            
                             'MC_withDodson',
                            'SR_withDodson', 
                            'NV_withDodson',
                            'MC_withDodson',
                            'SR_withDodson',            
                            'NV_withDodson',
                            'SR_withDodson',
                            'NV_withDodson',
                            'MC_withDodson',
                            'SR_withDodson',  
                            'NV_withDodson',
                            'MC_withDodson',
                            'SR_withDodson',  
                            'NV_withDodson',
                            'MC_withDodson',
                            'SR_withDodson',  
                          ]
        session_start_times = [ 
                                 0.00,
                                 36.0,
                                 69.5,
                                 0.00,
                                 62.0,
                                 0.00,
                                 89.0,
                                 0.00,
                                 0.00,
                                 0.00,
                                 165.8,
                                 96.0, 
                                 0.00,
                                 0.00,
                                 0.00,
                                 48.0,
                                
                                 59.2,
                                 49.5,
                                 40.0,
                                 50.0,
                                 0.00,
                                 69.8,
                                 85.0,
                                 212.9,
                                 68.5,
            
                                 363,
                                 0.00,
                                 79.0,
                                 162.6,
                                 231.9,
                                 109,
                                 0.00,
            
                                0.00,
                                0.00, 
                                0.00,
                                0.00,
                                0.00,          
                                0.00,
                                93.0,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00,
                                0.00, 
                                0.00,
                                274.4,
                                0.00,
                              ] # in second
        kilosortvers = list((np.ones(np.shape(dates_list))*4).astype(int))
        
        trig_channelnames = ['Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0',
                             'Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0',
                             'Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0','Dev1/ai0',
                             'Dev1/ai0','Dev1/ai9','Dev1/ai9','Dev1/ai9','Dev1/ai9','Dev1/ai9','Dev1/ai0','Dev1/ai0',
                             'Dev1/ai0','Dev1/ai0','Dev1/ai9','Dev1/ai9','Dev1/ai9','Dev1/ai9','Dev1/ai9','Dev1/ai9',
                             'Dev1/ai9','Dev1/ai9','Dev1/ai9','Dev1/ai9','Dev1/ai9','Dev1/ai9','Dev1/ai9','Dev1/ai9',
                              ]
        
        animal1_fixedorders = ['dannon','dannon','dannon','dannon','dannon','dannon','dannon','dannon',
                               'dannon','dannon','dannon','dannon','dannon','dannon','dannon','dannon',
                               'ginger','ginger','ginger','koala','koala','koala','vermelho','vermelho',
                               'vermelho','dodson','dodson','dodson','dodson','dodson','dodson','dodson',
                               'dodson','dodson','dodson','dodson','dodson','dodson','dodson','dodson',
                               'dodson','dodson','dodson','dodson','dodson','dodson','dodson','dodson',
                              ]
        animal2_fixedorders = ['kanga','kanga','kanga','kanga','kanga','kanga','kanga','kanga',
                               'kanga','kanga','kanga','kanga','kanga','kanga','kanga','kanga',
                               'kanga','kanga','kanga','kanga','kanga','kanga','kanga','kanga',
                               'kanga','kanga','kanga','kanga','kanga','kanga','kanga','kanga',
                               'kanga','kanga','kanga','kanga','kanga','kanga','kanga','kanga',
                               'kanga','kanga','kanga','kanga','kanga','kanga','kanga','kanga',
                              ]
        recordedanimals = animal2_fixedorders

        animal1_filenames = ["Dannon","Dannon","Dannon","Dannon","Dannon","Dannon","Dannon","Dannon",
                             "Dannon","Dannon","Dannon","Dannon","Dannon","Dannon","Dannon","Dannon",
                             "Ginger","Ginger","Ginger", "Kanga", "Kanga", "Kanga", "Kanga", "Kanga",
                              "Kanga","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson",
                             "Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson",
                             "Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson","Dodson",
                             
                            ]
        animal2_filenames = ["Kanga","Kanga","Kanga","Kanga","Kanga","Kanga","Kanga","Kanga",
                             "Kanga","Kanga","Kanga","Kanga","Kanga","Kanga","Kanga","Kanga",
                             "Kanga","Kanga","Kanga","Koala","Koala","Koala","Vermelho","Vermelho",
                             "Vermelho","Kanga","Kanga","Kanga","Kanga","Kanga","Kanga","Kanga",
                             "Kanga","Kanga","Kanga","Kanga","Kanga","Kanga","Kanga","Kanga",
                             "Kanga","Kanga","Kanga","Kanga","Kanga","Kanga","Kanga","Kanga",
                            ]
        
    elif do_OFC:
        # pick only five sessions for each conditions
        neural_record_conditions = [
                                     
                                   ]
        dates_list = [
                      
                     ]
        videodates_list = dates_list
        task_conditions = [
                           
                          ]
        session_start_times = [ 
                                
                              ] # in second
        kilosortvers = [ 

                       ]
    
        animal1_fixedorders = ['dannon']*np.shape(dates_list)[0]
        animal2_fixedorders = ['kanga']*np.shape(dates_list)[0]
        recordedanimals = animal2_fixedorders
        
        animal1_filenames = ["Dannon"]*np.shape(dates_list)[0]
        animal2_filenames = ["Kanga"]*np.shape(dates_list)[0]
    

    
# a test case
if 0: # kanga example
    neural_record_conditions = ['20250415_Kanga_MC_withDodson']
    dates_list = ["20250415"]
    videodates_list = dates_list
    task_conditions = ['MC_withDodson']
    session_start_times = [363] # in second
    kilosortvers = [4]
    trig_channelnames = ['Dev1/ai9']
    animal1_fixedorders = ['dodson']
    animal2_fixedorders = ['kanga']
    recordedanimals = animal2_fixedorders
    animal1_filenames = ["Dodson"]
    animal2_filenames = ["Kanga"]
if 0: # dodson example 
    neural_record_conditions = ['20250415_Dodson_MC_withKanga']
    dates_list = ["20250415"]
    videodates_list = dates_list
    task_conditions = ['MC_withKanga']
    session_start_times = [363] # in second
    kilosortvers = [4]
    trig_channelnames = ['Dev1/ai0']
    animal1_fixedorders = ['dodson']
    recordedanimals = animal1_fixedorders
    animal2_fixedorders = ['kanga']
    animal1_filenames = ["Dodson"]
    animal2_filenames = ["Kanga"]
    
ndates = np.shape(dates_list)[0]

session_start_frames = session_start_times * fps # fps is 30Hz

totalsess_time = 600

# video tracking results info
animalnames_videotrack = ['dodson','scorch'] # does not really mean dodson and scorch, instead, indicate animal1 and animal2
bodypartnames_videotrack = ['rightTuft','whiteBlaze','leftTuft','rightEye','leftEye','mouth']


# which camera to analyzed
cameraID = 'camera-2'
cameraID_short = 'cam2'

considerlevertube = 1
considertubeonly = 0

# location of levers and tubes for camera 2
# # camera 1
# lever_locs_camI = {'dodson':np.array([645,600]),'scorch':np.array([425,435])}
# tube_locs_camI  = {'dodson':np.array([1350,630]),'scorch':np.array([555,345])}
# # camera 2
# # location of the estimiated middle of the box
lever_locs_camI = {'dodson':np.array([1325,615]),'scorch':np.array([560,615])}
# # location of the estimated lever
# lever_locs_camI = {'dodson':np.array([1335,715]),'scorch':np.array([550,715])}
tube_locs_camI  = {'dodson':np.array([1550,515]),'scorch':np.array([350,515])}
# # old
# # lever_locs_camI = {'dodson':np.array([1335,715]),'scorch':np.array([550,715])}
# # tube_locs_camI  = {'dodson':np.array([1650,490]),'scorch':np.array([250,490])}
# # camera 3
# lever_locs_camI = {'dodson':np.array([1580,440]),'scorch':np.array([1296,540])}
# tube_locs_camI  = {'dodson':np.array([1470,375]),'scorch':np.array([805,475])}


if np.shape(session_start_times)[0] != np.shape(dates_list)[0]:
    exit()

    
# define bhv events summarizing variables     
tasktypes_all_dates = np.zeros((ndates,1))
coopthres_all_dates = np.zeros((ndates,1))

succ_rate_all_dates = np.zeros((ndates,1))
interpullintv_all_dates = np.zeros((ndates,1))
trialnum_all_dates = np.zeros((ndates,1))

owgaze1_num_all_dates = np.zeros((ndates,1))
owgaze2_num_all_dates = np.zeros((ndates,1))
mtgaze1_num_all_dates = np.zeros((ndates,1))
mtgaze2_num_all_dates = np.zeros((ndates,1))
pull1_num_all_dates = np.zeros((ndates,1))
pull2_num_all_dates = np.zeros((ndates,1))

bhv_intv_all_dates = dict.fromkeys(dates_list, [])

pull_trig_events_all_dates = dict.fromkeys(dates_list, [])
succpull_trig_events_all_dates = dict.fromkeys(dates_list, [])
failpull_trig_events_all_dates = dict.fromkeys(dates_list, [])

pull_infos_all_dates = dict.fromkeys(dates_list, []) # keep some useful information about pulls - time from last reward, number of preceding failed pull etc

spike_trig_events_all_dates = dict.fromkeys(dates_list, [])

bhvevents_aligned_FR_all_dates = dict.fromkeys(dates_list, [])
bhvevents_aligned_FR_allevents_all_dates = dict.fromkeys(dates_list, [])

strategy_aligned_FR_all_dates = dict.fromkeys(dates_list, [])
strategy_aligned_FR_allevents_all_dates = dict.fromkeys(dates_list, [])

# where to save the summarizing data
data_saved_folder = '/gpfs/radev/pi/nandy/jadi_gibbs_data/VideoTracker_SocialInter/3d_recontruction_analysis_self_and_coop_task_data_saved/'

# neural data folder
neural_data_folder = '/gpfs/radev/pi/nandy/jadi_gibbs_data/Marmoset_neural_recording/'

    

In [ ]:
print(np.shape(neural_record_conditions))
print(np.shape(task_conditions))
print(np.shape(dates_list))
print(np.shape(videodates_list)) 
print(np.shape(session_start_times))

print(np.shape(kilosortvers))

print(np.shape(trig_channelnames))
print(np.shape(animal1_fixedorders)) 
print(np.shape(recordedanimals))
print(np.shape(animal2_fixedorders))

print(np.shape(animal1_filenames))
print(np.shape(animal2_filenames))  

In [ ]:
# basic behavior analysis (define time stamps for each bhv events, etc)

try:
    if redo_anystep:
        dummy
    
    #
    print('loading all data')
    
    # load saved data
    data_saved_subfolder = data_saved_folder+'data_saved_singlecam_wholebody'+savefile_sufix+'/'+cameraID+'/'+animal1_fixedorders[0]+animal2_fixedorders[0]+'/'
    
    with open(data_saved_subfolder+'/owgaze1_num_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        owgaze1_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/owgaze2_num_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        owgaze2_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/mtgaze1_num_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        mtgaze1_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/mtgaze2_num_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        mtgaze2_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/pull1_num_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        pull1_num_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/pull2_num_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        pull2_num_all_dates = pickle.load(f)

    with open(data_saved_subfolder+'/tasktypes_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        tasktypes_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/coopthres_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        coopthres_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/succ_rate_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        succ_rate_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/interpullintv_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        interpullintv_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/trialnum_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        trialnum_all_dates = pickle.load(f)
    with open(data_saved_subfolder+'/bhv_intv_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        bhv_intv_all_dates = pickle.load(f)

    with open(data_saved_subfolder+'/pull_trig_events_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        pull_trig_events_all_dates = pickle.load(f)    
    with open(data_saved_subfolder+'/succpull_trig_events_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        succpull_trig_events_all_dates = pickle.load(f)    
    with open(data_saved_subfolder+'/failpull_trig_events_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        failpull_trig_events_all_dates = pickle.load(f)    
    
    with open(data_saved_subfolder+'/pull_infos_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        pull_infos_all_dates  = pickle.load(f)      
    
    # this one should be calculated in the XX_PullStartfocused_xx code
    with open(data_saved_subfolder+'/pull_rts_all_dates_'+doOnsetAfterMin_suffix+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        pull_rts_all_dates  = pickle.load(f)
    
    with open(data_saved_subfolder+'/spike_trig_events_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        spike_trig_events_all_dates = pickle.load(f) 
        
    with open(data_saved_subfolder+'/bhvevents_aligned_FR_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        bhvevents_aligned_FR_all_dates = pickle.load(f) 
    with open(data_saved_subfolder+'/bhvevents_aligned_FR_allevents_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        bhvevents_aligned_FR_allevents_all_dates = pickle.load(f) 
        
    with open(data_saved_subfolder+'/strategy_aligned_FR_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        strategy_aligned_FR_all_dates = pickle.load(f) 
    with open(data_saved_subfolder+'/strategy_aligned_FR_allevents_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'rb') as f:
        strategy_aligned_FR_allevents_all_dates = pickle.load(f) 
        
    print('all data from all dates are loaded')

except:

    print('analyze all dates')

    for idate in np.arange(0,ndates,1):
    
        date_tgt = dates_list[idate]
        videodate_tgt = videodates_list[idate]
        
        neural_record_condition = neural_record_conditions[idate]
        
        session_start_time = session_start_times[idate]
        
        kilosortver = kilosortvers[idate]

        trig_channelname = trig_channelnames[idate]
        
        animal1_filename = animal1_filenames[idate]
        animal2_filename = animal2_filenames[idate]
        
        animal1_fixedorder = [animal1_fixedorders[idate]]
        animal2_fixedorder = [animal2_fixedorders[idate]]
        
        recordedanimal = recordedanimals[idate]

        # folder and file path
        camera12_analyzed_path = "/gpfs/radev/pi/nandy/jadi_gibbs_data/VideoTracker_SocialInter/test_video_cooperative_task_3d/"+videodate_tgt+"_"+animal1_filename+"_"+animal2_filename+"_camera12/"
        camera23_analyzed_path = "/gpfs/radev/pi/nandy/jadi_gibbs_data/VideoTracker_SocialInter/test_video_cooperative_task_3d/"+videodate_tgt+"_"+animal1_filename+"_"+animal2_filename+"_camera23/"
        
        # 
        try: 
            singlecam_ana_type = "DLC_dlcrnetms5_marmoset_tracking_with_middle_camera_withHeadchamberFeb28shuffle1_167500"
            bodyparts_camI_camIJ = camera12_analyzed_path+videodate_tgt+"_"+animal1_filename+"_"+animal2_filename+"_"+cameraID+singlecam_ana_type+"_el_filtered.h5"
            if not os.path.exists(bodyparts_camI_camIJ):
                singlecam_ana_type = "DLC_dlcrnetms5_marmoset_tracking_with_middle_camera_withHeadchamberFeb28shuffle1_80000"
                bodyparts_camI_camIJ = camera12_analyzed_path+videodate_tgt+"_"+animal1_filename+"_"+animal2_filename+"_"+cameraID+singlecam_ana_type+"_el_filtered.h5"
            if not os.path.exists(bodyparts_camI_camIJ):
                singlecam_ana_type = "DLC_dlcrnetms5_marmoset_tracking_with_middle_cameraSep1shuffle1_150000"
                bodyparts_camI_camIJ = camera12_analyzed_path+videodate_tgt+"_"+animal1_filename+"_"+animal2_filename+"_"+cameraID+singlecam_ana_type+"_el_filtered.h5"                
            # get the bodypart data from files
            bodyparts_locs_camI = body_part_locs_singlecam(bodyparts_camI_camIJ,singlecam_ana_type,animalnames_videotrack,bodypartnames_videotrack,videodate_tgt)
            video_file_original = camera12_analyzed_path+videodate_tgt+"_"+animal1_filename+"_"+animal2_filename+"_"+cameraID+".mp4"
        except:
            singlecam_ana_type = "DLC_dlcrnetms5_marmoset_tracking_with_middle_camera_withHeadchamberFeb28shuffle1_167500"
            bodyparts_camI_camIJ = camera23_analyzed_path+videodate_tgt+"_"+animal1_filename+"_"+animal2_filename+"_"+cameraID+singlecam_ana_type+"_el_filtered.h5"
            if not os.path.exists(bodyparts_camI_camIJ):
                singlecam_ana_type = "DLC_dlcrnetms5_marmoset_tracking_with_middle_camera_withHeadchamberFeb28shuffle1_80000"
                bodyparts_camI_camIJ = camera23_analyzed_path+videodate_tgt+"_"+animal1_filename+"_"+animal2_filename+"_"+cameraID+singlecam_ana_type+"_el_filtered.h5"
            if not os.path.exists(bodyparts_camI_camIJ):
                singlecam_ana_type = "DLC_dlcrnetms5_marmoset_tracking_with_middle_cameraSep1shuffle1_150000"
                bodyparts_camI_camIJ = camera23_analyzed_path+videodate_tgt+"_"+animal1_filename+"_"+animal2_filename+"_"+cameraID+singlecam_ana_type+"_el_filtered.h5"
            
            # get the bodypart data from files
            bodyparts_locs_camI = body_part_locs_singlecam(bodyparts_camI_camIJ,singlecam_ana_type,animalnames_videotrack,bodypartnames_videotrack,videodate_tgt)
            video_file_original = camera23_analyzed_path+videodate_tgt+"_"+animal1_filename+"_"+animal2_filename+"_"+cameraID+".mp4"        
        
        
        # load behavioral results
        try:
            bhv_data_path = "/gpfs/radev/pi/nandy/jadi_gibbs_data/VideoTracker_SocialInter/marmoset_tracking_bhv_data_from_task_code/"+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"/"
            trial_record_json = glob.glob(bhv_data_path +date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_TrialRecord_" + "*.json")
            bhv_data_json = glob.glob(bhv_data_path + date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_bhv_data_" + "*.json")
            session_info_json = glob.glob(bhv_data_path + date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_session_info_" + "*.json")
            ni_data_json = glob.glob(bhv_data_path + date_tgt+"_"+animal2_filename+"_"+animal1_filename+"_ni_data_" + "*.json")
            #
            trial_record = pd.read_json(trial_record_json[0])
            bhv_data = pd.read_json(bhv_data_json[0])
            session_info = pd.read_json(session_info_json[0])
            # 
            with open(ni_data_json[0]) as f:
                for line in f:
                    ni_data=json.loads(line)   
        except:
            bhv_data_path = "/gpfs/radev/pi/nandy/jadi_gibbs_data/VideoTracker_SocialInter/marmoset_tracking_bhv_data_from_task_code/"+date_tgt+"_"+animal1_filename+"_"+animal2_filename+"/"
            trial_record_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_TrialRecord_" + "*.json")
            bhv_data_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_bhv_data_" + "*.json")
            session_info_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_session_info_" + "*.json")
            ni_data_json = glob.glob(bhv_data_path + date_tgt+"_"+animal1_filename+"_"+animal2_filename+"_ni_data_" + "*.json")
            #
            trial_record = pd.read_json(trial_record_json[0])
            bhv_data = pd.read_json(bhv_data_json[0])
            session_info = pd.read_json(session_info_json[0])
            #
            with open(ni_data_json[0]) as f:
                for line in f:
                    ni_data=json.loads(line)

        # get animal info from the session information
        animal1 = session_info['lever1_animal'][0].lower()
        animal2 = session_info['lever2_animal'][0].lower()

        
        # get task type and cooperation threshold
        try:
            coop_thres = session_info["pulltime_thres"][0]
            tasktype = session_info["task_type"][0]
        except:
            coop_thres = 0
            tasktype = 1
        tasktypes_all_dates[idate] = tasktype
        coopthres_all_dates[idate] = coop_thres   

        # successful trial or not
        succtrial_ornot = np.array((trial_record['rewarded']>0).astype(int))
        succpull1_ornot = np.array((np.isin(bhv_data[bhv_data['behavior_events']==1]['trial_number'],trial_record[trial_record['rewarded']>0]['trial_number'])).astype(int))
        succpull2_ornot = np.array((np.isin(bhv_data[bhv_data['behavior_events']==2]['trial_number'],trial_record[trial_record['rewarded']>0]['trial_number'])).astype(int))
        succpulls_ornot = [succpull1_ornot,succpull2_ornot]
            
        # clean up the trial_record
        warnings.filterwarnings('ignore')
        trial_record_clean = pd.DataFrame(columns=trial_record.columns)
        # for itrial in np.arange(0,np.max(trial_record['trial_number']),1):
        for itrial in trial_record['trial_number']:
            # trial_record_clean.loc[itrial] = trial_record[trial_record['trial_number']==itrial+1].iloc[[0]]
            trial_record_clean = trial_record_clean.append(trial_record[trial_record['trial_number']==itrial].iloc[[0]])
        trial_record_clean = trial_record_clean.reset_index(drop = True)

        # change bhv_data time to the absolute time
        time_points_new = pd.DataFrame(np.zeros(np.shape(bhv_data)[0]),columns=["time_points_new"])
        # for itrial in np.arange(0,np.max(trial_record_clean['trial_number']),1):
        for itrial in np.arange(0,np.shape(trial_record_clean)[0],1):
            # ind = bhv_data["trial_number"]==itrial+1
            ind = bhv_data["trial_number"]==trial_record_clean['trial_number'][itrial]
            new_time_itrial = bhv_data[ind]["time_points"] + trial_record_clean["trial_starttime"].iloc[itrial]
            time_points_new["time_points_new"][ind] = new_time_itrial
        bhv_data["time_points"] = time_points_new["time_points_new"]
        bhv_data = bhv_data[bhv_data["time_points"] != 0]


        # analyze behavior results
        # succ_rate_all_dates[idate] = np.sum(trial_record_clean["rewarded"]>0)/np.shape(trial_record_clean)[0]
        succ_rate_all_dates[idate] = np.sum((bhv_data['behavior_events']==3)|(bhv_data['behavior_events']==4))/np.sum((bhv_data['behavior_events']==1)|(bhv_data['behavior_events']==2))
        trialnum_all_dates[idate] = np.shape(trial_record_clean)[0]
        #
        pullid = np.array(bhv_data[(bhv_data['behavior_events']==1) | (bhv_data['behavior_events']==2)]["behavior_events"])
        pulltime = np.array(bhv_data[(bhv_data['behavior_events']==1) | (bhv_data['behavior_events']==2)]["time_points"])
        pullid_diff = np.abs(pullid[1:] - pullid[0:-1])
        pulltime_diff = pulltime[1:] - pulltime[0:-1]
        interpull_intv = pulltime_diff[pullid_diff==1]
        interpull_intv = interpull_intv[interpull_intv<10]
        mean_interpull_intv = np.nanmean(interpull_intv)
        std_interpull_intv = np.nanstd(interpull_intv)
        #
        interpullintv_all_dates[idate] = mean_interpull_intv
        # 
        if np.isin(animal1,animal1_fixedorder):
            pull1_num_all_dates[idate] = np.sum(bhv_data['behavior_events']==1) 
            pull2_num_all_dates[idate] = np.sum(bhv_data['behavior_events']==2)
        else:
            pull1_num_all_dates[idate] = np.sum(bhv_data['behavior_events']==2) 
            pull2_num_all_dates[idate] = np.sum(bhv_data['behavior_events']==1)

        
        # load behavioral event results
        try:
            # dummy
            print('load social gaze with '+cameraID+' only of '+date_tgt)
            with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_look_ornot.pkl', 'rb') as f:
                output_look_ornot = pickle.load(f)
            with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_allvectors.pkl', 'rb') as f:
                output_allvectors = pickle.load(f)
            with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_allangles.pkl', 'rb') as f:
                output_allangles = pickle.load(f)  
            with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_key_locations.pkl', 'rb') as f:
                output_key_locations = pickle.load(f)
        except:   
            print('analyze social gaze with '+cameraID+' only of '+date_tgt)
            # get social gaze information 
            output_look_ornot, output_allvectors, output_allangles = find_socialgaze_timepoint_singlecam_wholebody(bodyparts_locs_camI,lever_locs_camI,tube_locs_camI,
                                                                                                                   considerlevertube,considertubeonly,sqr_thres_tubelever,
                                                                                                                   sqr_thres_face,sqr_thres_body)
            output_key_locations = find_socialgaze_timepoint_singlecam_wholebody_2(bodyparts_locs_camI,lever_locs_camI,tube_locs_camI,considerlevertube)
            
            # save data
            current_dir = data_saved_folder+'/bhv_events_singlecam_wholebody/'+animal1_fixedorder[0]+animal2_fixedorder[0]
            add_date_dir = os.path.join(current_dir,cameraID+'/'+date_tgt)
            if not os.path.exists(add_date_dir):
                os.makedirs(add_date_dir)
            #
            with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_look_ornot.pkl', 'wb') as f:
                pickle.dump(output_look_ornot, f)
            with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_allvectors.pkl', 'wb') as f:
                pickle.dump(output_allvectors, f)
            with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_allangles.pkl', 'wb') as f:
                pickle.dump(output_allangles, f)
            with open(data_saved_folder+"bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/output_key_locations.pkl', 'wb') as f:
                pickle.dump(output_key_locations, f)
                

        look_at_other_or_not_merge = output_look_ornot['look_at_other_or_not_merge']
        look_at_tube_or_not_merge = output_look_ornot['look_at_tube_or_not_merge']
        look_at_lever_or_not_merge = output_look_ornot['look_at_lever_or_not_merge']
        look_at_otherlever_or_not_merge = output_look_ornot['look_at_otherlever_or_not_merge']
        look_at_otherface_or_not_merge = output_look_ornot['look_at_otherface_or_not_merge']
        
        # change the unit to second and align to the start of the session
        session_start_time = session_start_times[idate]
        look_at_other_or_not_merge['time_in_second'] = np.arange(0,np.shape(look_at_other_or_not_merge['dodson'])[0],1)/fps - session_start_time
        look_at_lever_or_not_merge['time_in_second'] = np.arange(0,np.shape(look_at_lever_or_not_merge['dodson'])[0],1)/fps - session_start_time
        look_at_tube_or_not_merge['time_in_second'] = np.arange(0,np.shape(look_at_tube_or_not_merge['dodson'])[0],1)/fps - session_start_time 
        look_at_otherlever_or_not_merge['time_in_second'] = np.arange(0,np.shape(look_at_otherlever_or_not_merge['dodson'])[0],1)/fps - session_start_time
        look_at_otherface_or_not_merge['time_in_second'] = np.arange(0,np.shape(look_at_otherface_or_not_merge['dodson'])[0],1)/fps - session_start_time

        
        # find time point of behavioral events
        output_time_points_socialgaze ,output_time_points_levertube = bhv_events_timepoint_singlecam(bhv_data,look_at_other_or_not_merge,look_at_lever_or_not_merge,look_at_tube_or_not_merge)
        time_point_pull1 = output_time_points_socialgaze['time_point_pull1']
        time_point_pull2 = output_time_points_socialgaze['time_point_pull2']
        oneway_gaze1 = output_time_points_socialgaze['oneway_gaze1']
        oneway_gaze2 = output_time_points_socialgaze['oneway_gaze2']
        mutual_gaze1 = output_time_points_socialgaze['mutual_gaze1']
        mutual_gaze2 = output_time_points_socialgaze['mutual_gaze2']
        # 
        # mostly just for the sessions in which MC and SR are in the same session 
        firstpulltime = np.nanmin([np.nanmin(time_point_pull1),np.nanmin(time_point_pull2)])
        oneway_gaze1 = oneway_gaze1[oneway_gaze1>(firstpulltime-15)] # 15s before the first pull (animal1 or 2) count as the active period
        oneway_gaze2 = oneway_gaze2[oneway_gaze2>(firstpulltime-15)]
        mutual_gaze1 = mutual_gaze1[mutual_gaze1>(firstpulltime-15)]
        mutual_gaze2 = mutual_gaze2[mutual_gaze2>(firstpulltime-15)]  
        #    
        # newly added condition: only consider gaze during the active pulling time (15s after the last pull)    
        lastpulltime = np.nanmax([np.nanmax(time_point_pull1),np.nanmax(time_point_pull2)])
        oneway_gaze1 = oneway_gaze1[oneway_gaze1<(lastpulltime+15)]    
        oneway_gaze2 = oneway_gaze2[oneway_gaze2<(lastpulltime+15)]
        mutual_gaze1 = mutual_gaze1[mutual_gaze1<(lastpulltime+15)]
        mutual_gaze2 = mutual_gaze2[mutual_gaze2<(lastpulltime+15)] 
            
        # define successful pulls and failed pulls
        if 0: # old definition; not in use
            trialnum_succ = np.array(trial_record_clean['trial_number'][trial_record_clean['rewarded']>0])
            bhv_data_succ = bhv_data[np.isin(bhv_data['trial_number'],trialnum_succ)]
            #
            time_point_pull1_succ = bhv_data_succ["time_points"][bhv_data_succ["behavior_events"]==1]
            time_point_pull2_succ = bhv_data_succ["time_points"][bhv_data_succ["behavior_events"]==2]
            time_point_pull1_succ = np.round(time_point_pull1_succ,1)
            time_point_pull2_succ = np.round(time_point_pull2_succ,1)
            #
            trialnum_fail = np.array(trial_record_clean['trial_number'][trial_record_clean['rewarded']==0])
            bhv_data_fail = bhv_data[np.isin(bhv_data['trial_number'],trialnum_fail)]
            #
            time_point_pull1_fail = bhv_data_fail["time_points"][bhv_data_fail["behavior_events"]==1]
            time_point_pull2_fail = bhv_data_fail["time_points"][bhv_data_fail["behavior_events"]==2]
            time_point_pull1_fail = np.round(time_point_pull1_fail,1)
            time_point_pull2_fail = np.round(time_point_pull2_fail,1)
        else:
            # a new definition of successful and failed pulls
            # separate successful and failed pulls
            # step 1 all pull and juice
            time_point_pull1 = bhv_data["time_points"][bhv_data["behavior_events"]==1]
            time_point_pull2 = bhv_data["time_points"][bhv_data["behavior_events"]==2]
            time_point_juice1 = bhv_data["time_points"][bhv_data["behavior_events"]==3]
            time_point_juice2 = bhv_data["time_points"][bhv_data["behavior_events"]==4]
            # step 2:
            # pull 1
            # Find the last pull before each juice
            successful_pull1 = [time_point_pull1[time_point_pull1 < juice].max() for juice in time_point_juice1]
            # Convert to Pandas Series
            successful_pull1 = pd.Series(successful_pull1, index=time_point_juice1.index)
            # Find failed pulls (pulls that are not successful)
            failed_pull1 = time_point_pull1[~time_point_pull1.isin(successful_pull1)]
            # pull 2
            # Find the last pull before each juice
            successful_pull2 = [time_point_pull2[time_point_pull2 < juice].max() for juice in time_point_juice2]
            # Convert to Pandas Series
            successful_pull2 = pd.Series(successful_pull2, index=time_point_juice2.index)
            # Find failed pulls (pulls that are not successful)
            failed_pull2 = time_point_pull2[~time_point_pull2.isin(successful_pull2)]
            #
            # step 3:
            time_point_pull1_succ = np.round(successful_pull1,1)
            time_point_pull2_succ = np.round(successful_pull2,1)
            time_point_pull1_fail = np.round(failed_pull1,1)
            time_point_pull2_fail = np.round(failed_pull2,1)
        # 
        time_point_pulls_succfail = { "pull1_succ":time_point_pull1_succ,
                                      "pull2_succ":time_point_pull2_succ,
                                      "pull1_fail":time_point_pull1_fail,
                                      "pull2_fail":time_point_pull2_fail,
                                    }
        
        # 
        # based on time point pull and juice, define some features for each pull action
        pull_infos_all_dates = get_pull_infos(animal1, animal2, time_point_pull1, time_point_pull2, 
                                              time_point_juice1, time_point_juice2)

    
            
        # new total session time (instead of 600s) - total time of the video recording
        totalsess_time = np.floor(np.shape(output_look_ornot['look_at_lever_or_not_merge']['dodson'])[0]/30) 
                
        # # plot behavioral events
        if np.isin(animal1,animal1_fixedorder):
                plot_bhv_events(date_tgt,animal1, animal2, session_start_time, totalsess_time, time_point_pull1, time_point_pull2, oneway_gaze1, oneway_gaze2, mutual_gaze1, mutual_gaze2)
        else:
                plot_bhv_events(date_tgt,animal2, animal1, session_start_time, totalsess_time, time_point_pull2, time_point_pull1, oneway_gaze2, oneway_gaze1, mutual_gaze2, mutual_gaze1)
        #
        # save behavioral events plot
        if 0:
            current_dir = data_saved_folder+'/bhv_events_singlecam_wholebody/'+animal1_fixedorder[0]+animal2_fixedorder[0]
            add_date_dir = os.path.join(current_dir,cameraID+'/'+date_tgt)
            if not os.path.exists(add_date_dir):
                os.makedirs(add_date_dir)
            plt.savefig(data_saved_folder+"/bhv_events_singlecam_wholebody/"+animal1_fixedorder[0]+animal2_fixedorder[0]+"/"+cameraID+'/'+date_tgt+'/'+date_tgt+"_"+cameraID_short+".pdf")

        #
        if np.isin(animal1,animal1_fixedorder):
            owgaze1_num_all_dates[idate] = np.shape(oneway_gaze1)[0]
            owgaze2_num_all_dates[idate] = np.shape(oneway_gaze2)[0]
            mtgaze1_num_all_dates[idate] = np.shape(mutual_gaze1)[0]
            mtgaze2_num_all_dates[idate] = np.shape(mutual_gaze2)[0]
        else:            
            owgaze1_num_all_dates[idate] = np.shape(oneway_gaze2)[0]
            owgaze2_num_all_dates[idate] = np.shape(oneway_gaze1)[0]
            mtgaze1_num_all_dates[idate] = np.shape(mutual_gaze2)[0]
            mtgaze2_num_all_dates[idate] = np.shape(mutual_gaze1)[0]
            

     
        
        # analyze the events interval, especially for the pull to other and other to pull interval
        # could be used for define time bin for DBN
        if 0:
            _,_,_,pullTOother_itv, otherTOpull_itv = bhv_events_interval(totalsess_time, session_start_time, time_point_pull1, time_point_pull2, 
                                                                         oneway_gaze1, oneway_gaze2, mutual_gaze1, mutual_gaze2)
            #
            pull_other_pool_itv = np.concatenate((pullTOother_itv,otherTOpull_itv))
            bhv_intv_all_dates[date_tgt] = {'pull_to_other':pullTOother_itv,'other_to_pull':otherTOpull_itv,
                            'pull_other_pooled': pull_other_pool_itv}
        
        
        # plot key continuous behavioral variables
        if 1:
            filepath_cont_var = data_saved_folder+'bhv_events_continuous_variables_singlecam_wholebody/'+animal1_fixedorder[0]+animal2_fixedorder[0]+'/'+cameraID+'/'+date_tgt+'/'
            if not os.path.exists(filepath_cont_var):
                os.makedirs(filepath_cont_var)

            savefig = 1
            
            aligntwins = 4 # 5 second
            
            min_length = np.shape(look_at_other_or_not_merge['dodson'])[0] # frame numbers of the video recording

            # NOTE! This one used the wrong and old version of separating successful and failed 
            pull_trig_events_summary, _, _ = plot_continuous_bhv_var_singlecam(filepath_cont_var+date_tgt+cameraID,
                                    aligntwins, savefig, animal1, animal2, 
                                    session_start_time, min_length, succpulls_ornot, time_point_pull1, time_point_pull2, 
                                    oneway_gaze1, oneway_gaze2, mutual_gaze1, mutual_gaze2, animalnames_videotrack,
                                    output_look_ornot, output_allvectors, output_allangles,output_key_locations)
            pull_trig_events_all_dates[date_tgt] = pull_trig_events_summary
            
            # successful pull
            try:
                pull_trig_events_summary, _, _ = plot_continuous_bhv_var_singlecam(filepath_cont_var+date_tgt+cameraID,
                                        aligntwins, savefig, animal1, animal2, 
                                        session_start_time, min_length, succpulls_ornot, 
                                        time_point_pull1_succ, time_point_pull2_succ, 
                                        oneway_gaze1, oneway_gaze2, mutual_gaze1, mutual_gaze2, animalnames_videotrack,
                                        output_look_ornot, output_allvectors, output_allangles,output_key_locations)
                succpull_trig_events_all_dates[date_tgt] = pull_trig_events_summary
            except:
                succpull_trig_events_all_dates[date_tgt] = np.nan
            
            # failed pull
            try:
                pull_trig_events_summary, _, _ = plot_continuous_bhv_var_singlecam(filepath_cont_var+date_tgt+cameraID,
                                        aligntwins, savefig, animal1, animal2, 
                                        session_start_time, min_length, succpulls_ornot, 
                                        time_point_pull1_fail, time_point_pull2_fail, 
                                        oneway_gaze1, oneway_gaze2, mutual_gaze1, mutual_gaze2, animalnames_videotrack,
                                        output_look_ornot, output_allvectors, output_allangles,output_key_locations)
                failpull_trig_events_all_dates[date_tgt] = pull_trig_events_summary
            except:
                failpull_trig_events_all_dates[date_tgt] = np.nan
                
        
        # session starting time compared with the neural recording
        session_start_time_niboard_offset = ni_data['session_t0_offset'] # in the unit of second
        try:
            neural_start_time_niboard_offset = ni_data['trigger_ts'][0]['elapsed_time'] # in the unit of second
        except: # for the multi-animal recording setup
            neural_start_time_niboard_offset = next(
                entry['timepoints'][0]['elapsed_time']
                for entry in ni_data['trigger_ts']
                if entry['channel_name'] == f"{trig_channelname}")
        neural_start_time_session_start_offset = neural_start_time_niboard_offset-session_start_time_niboard_offset
    
    
        # load channel maps
        channel_map_file = '/home/ws523/kilisort_spikesorting/Channel-Maps/Neuronexus_whitematter_2x32.mat'
        # channel_map_file = '/home/ws523/kilisort_spikesorting/Channel-Maps/Neuronexus_whitematter_2x32_kilosort4_new.mat'
        channel_map_data = scipy.io.loadmat(channel_map_file)
            
        # # load spike sorting results
        if 1:
            print('load spike data for '+neural_record_condition)
            if kilosortver == 2:
                spike_time_file = neural_data_folder+neural_record_condition+'/Kilosort/spike_times.npy'
                spike_time_data = np.load(spike_time_file)
            elif kilosortver == 4:
                spike_time_file = neural_data_folder+neural_record_condition+'/kilosort4_6500HzNotch/spike_times.npy'
                spike_time_data = np.load(spike_time_file)
            # 
            # align the FR recording time stamps
            spike_time_data = spike_time_data + fs_spikes*neural_start_time_session_start_offset
            # down-sample the spike recording resolution to 30Hz
            spike_time_data = spike_time_data/fs_spikes*fps
            spike_time_data = np.round(spike_time_data)
            #
            if kilosortver == 2:
                spike_clusters_file = neural_data_folder+neural_record_condition+'/Kilosort/spike_clusters.npy'
                spike_clusters_data = np.load(spike_clusters_file)
                spike_channels_data = np.copy(spike_clusters_data)
            elif kilosortver == 4:
                spike_clusters_file = neural_data_folder+neural_record_condition+'/kilosort4_6500HzNotch/spike_clusters.npy'
                spike_clusters_data = np.load(spike_clusters_file)
                spike_channels_data = np.copy(spike_clusters_data)
            #
            if kilosortver == 2:
                channel_maps_file = neural_data_folder+neural_record_condition+'/Kilosort/channel_map.npy'
                channel_maps_data = np.load(channel_maps_file)
            elif kilosortver == 4:
                channel_maps_file = neural_data_folder+neural_record_condition+'/kilosort4_6500HzNotch/channel_map.npy'
                channel_maps_data = np.load(channel_maps_file)
            #
            if kilosortver == 2:
                channel_pos_file = neural_data_folder+neural_record_condition+'/Kilosort/channel_positions.npy'
                channel_pos_data = np.load(channel_pos_file)
            elif kilosortver == 4:
                channel_pos_file = neural_data_folder+neural_record_condition+'/kilosort4_6500HzNotch/channel_positions.npy'
                channel_pos_data = np.load(channel_pos_file)
            #
            if kilosortver == 2:
                clusters_info_file = neural_data_folder+neural_record_condition+'/Kilosort/cluster_info.tsv'
                clusters_info_data = pd.read_csv(clusters_info_file,sep="\t")
            elif kilosortver == 4:
                clusters_info_file = neural_data_folder+neural_record_condition+'/kilosort4_6500HzNotch/cluster_info.tsv'
                clusters_info_data = pd.read_csv(clusters_info_file,sep="\t")
            #
            # only get the spikes that are manually checked
            try:
                good_clusters = clusters_info_data[(clusters_info_data.group=='good')|(clusters_info_data.group=='mua')]['cluster_id'].values
            except:
                good_clusters = clusters_info_data[(clusters_info_data.group=='good')|(clusters_info_data.group=='mua')]['id'].values
            #
            clusters_info_data = clusters_info_data[~pd.isnull(clusters_info_data.group)]
            #
            spike_time_data = spike_time_data[np.isin(spike_clusters_data,good_clusters)]
            spike_channels_data = spike_channels_data[np.isin(spike_clusters_data,good_clusters)]
            spike_clusters_data = spike_clusters_data[np.isin(spike_clusters_data,good_clusters)]
            
            #
            nclusters = np.shape(clusters_info_data)[0]
            #
            for icluster in np.arange(0,nclusters,1):
                try:
                    cluster_id = clusters_info_data['id'].iloc[icluster]
                except:
                    cluster_id = clusters_info_data['cluster_id'].iloc[icluster]
                spike_channels_data[np.isin(spike_clusters_data,cluster_id)] = clusters_info_data['ch'].iloc[icluster]   
            # 
            # get the channel to depth information, change 2 shanks to 1 shank 
            try:
                channel_depth=np.hstack([channel_pos_data[channel_pos_data[:,0]==0,1]*2,channel_pos_data[channel_pos_data[:,0]==1,1]*2+1])
                # channel_depth=np.hstack([channel_pos_data[channel_pos_data[:,0]==0,1],channel_pos_data[channel_pos_data[:,0]==31.2,1]])            
                # channel_to_depth = np.vstack([channel_maps_data.T[0],channel_depth])
                channel_to_depth = np.vstack([channel_maps_data.T,channel_depth])
            except:
                channel_depth=np.hstack([channel_pos_data[channel_pos_data[:,0]==0,1],channel_pos_data[channel_pos_data[:,0]==31.2,1]])            
                # channel_to_depth = np.vstack([channel_maps_data.T[0],channel_depth])
                channel_to_depth = np.vstack([channel_maps_data.T,channel_depth])
                channel_to_depth[1] = channel_to_depth[1]/30-64 # make the y axis consistent
            #
           
            
            # calculate the firing rate
            # FR_kernel = 0.20 # in the unit of second
            FR_kernel = 1/30 # in the unit of second # 1/30 same resolution as the video recording
            # FR_kernel is sent to to be this if want to explore it's relationship with continuous trackng data
            
            totalsess_time_forFR = np.floor(np.shape(output_look_ornot['look_at_lever_or_not_merge']['dodson'])[0]/30)  # to match the total time of the video recording
            _,FR_timepoint_allch,FR_allch,FR_zscore_allch = spike_analysis_FR_calculation(fps, FR_kernel, totalsess_time_forFR,
                                                                                          spike_clusters_data, spike_time_data)
            # _,FR_timepoint_allch,FR_allch,FR_zscore_allch = spike_analysis_FR_calculation(fps,FR_kernel,totalsess_time_forFR,
            #                                                                              spike_channels_data, spike_time_data)
            # behavioral events aligned firing rate for each unit
            if 1: 
                print('plot event aligned firing rate')
                #
                savefig = 1
                save_path = data_saved_folder+"fig_for_basic_neural_analysis_allsessions_basicEvents/"+cameraID+"/"+\
                            animal1_filename+"_"+animal2_filename+'_'+recordedanimal+'Recorded'+"/"+date_tgt
                if not os.path.exists(save_path):
                    os.makedirs(save_path)
                #
                aligntwins = 4 # 5 second
                gaze_thresold = 0.2 # min length threshold to define if a gaze is real gaze or noise, in the unit of second 
                #
                bhvevents_aligned_FR_average_all,bhvevents_aligned_FR_allevents_all = plot_bhv_events_aligned_FR(date_tgt,savefig,save_path, animal1, animal2,time_point_pull1,time_point_pull2,time_point_pulls_succfail,
                                           oneway_gaze1,oneway_gaze2,mutual_gaze1,mutual_gaze2,gaze_thresold,totalsess_time_forFR,
                                           aligntwins,fps,FR_timepoint_allch,FR_zscore_allch,clusters_info_data)
                
                bhvevents_aligned_FR_all_dates[date_tgt] = bhvevents_aligned_FR_average_all
                bhvevents_aligned_FR_allevents_all_dates[date_tgt] = bhvevents_aligned_FR_allevents_all
                
            
            # the three strategy aligned firing rate for each unit
            if 1: 
                print('plot strategy aligned firing rate')
                #
                savefig = 1
                save_path = data_saved_folder+"fig_for_basic_neural_analysis_allsessions_basicEvents/"+cameraID+"/"+\
                            animal1_filename+"_"+animal2_filename+'_'+recordedanimal+'Recorded'+"/"+date_tgt
                if not os.path.exists(save_path):
                    os.makedirs(save_path)
                #
                stg_twins = 1.5 # 3s, the behavioral event interval used to define strategy, consistent with DBN 3s time lags
                aligntwins = 4 # 5 second
                gaze_thresold = 0.2 # min length threshold to define if a gaze is real gaze or noise, in the unit of second 
                #
                strategy_aligned_FR_average_all,strategy_aligned_FR_allevents_all = plot_strategy_aligned_FR(date_tgt,savefig,save_path, animal1, animal2,time_point_pull1,time_point_pull2,time_point_pulls_succfail,
                                           oneway_gaze1,oneway_gaze2,mutual_gaze1,mutual_gaze2,gaze_thresold,totalsess_time_forFR,
                                           aligntwins,stg_twins,fps,FR_timepoint_allch,FR_zscore_allch,clusters_info_data)
                
                strategy_aligned_FR_all_dates[date_tgt] = strategy_aligned_FR_average_all
                strategy_aligned_FR_allevents_all_dates[date_tgt] = strategy_aligned_FR_allevents_all
                
            
            #
            # Run PCA analysis
            FR_zscore_allch_np_merged = np.array(pd.DataFrame(FR_zscore_allch).T)
            FR_zscore_allch_np_merged = FR_zscore_allch_np_merged[~np.isnan(np.sum(FR_zscore_allch_np_merged,axis=1)),:]
            # # run PCA on the entire session
            pca = PCA(n_components=3)
            FR_zscore_allch_PCs = pca.fit_transform(FR_zscore_allch_np_merged.T)
            #
            # # run PCA around the -PCAtwins to PCAtwins for each behavioral events
            PCAtwins = 4 # 5 second
            gaze_thresold = 0.5 # min length threshold to define if a gaze is real gaze or noise, in the unit of second 
            savefigs = 0 
            if 0:
                PCA_around_bhv_events(FR_timepoint_allch,FR_zscore_allch_np_merged,time_point_pull1,time_point_pull2,time_point_pulls_succfail, 
                              oneway_gaze1,oneway_gaze2,mutual_gaze1,mutual_gaze2,gaze_thresold,totalsess_time_forFR,PCAtwins,fps,
                              savefigs,data_saved_folder,cameraID,animal1_filename,animal2_filename,date_tgt)
            if 0:
                if (np.isin(animal1, ['dodson'])) | (np.isin(animal2, ['kanga'])):
                    PCA_around_bhv_events_video(FR_timepoint_allch,FR_zscore_allch_np_merged,time_point_pull1,time_point_pull2,time_point_pulls_succfail, 
                                      oneway_gaze1,oneway_gaze2,mutual_gaze1,mutual_gaze2,gaze_thresold,totalsess_time_forFR,PCAtwins,fps,
                                      data_saved_folder,cameraID,animal1_filename,animal2_filename,date_tgt)
                elif (np.isin(animal2, ['dodson'])) | (np.isin(animal1, ['kanga'])):
                    time_point_pulls_succfail_rev = time_point_pulls_succfail.copy()
                    time_point_pulls_succfail_rev['pull1_succ'] = time_point_pulls_succfail['pull2_succ']
                    time_point_pulls_succfail_rev['pull1_fail'] = time_point_pulls_succfail['pull2_fail']
                    time_point_pulls_succfail_rev['pull2_succ'] = time_point_pulls_succfail['pull1_succ']
                    time_point_pulls_succfail_rev['pull2_fail'] = time_point_pulls_succfail['pull1_fail']
                    PCA_around_bhv_events_video(FR_timepoint_allch,FR_zscore_allch_np_merged,time_point_pull2,time_point_pull1,time_point_pulls_succfail_rev, 
                                      oneway_gaze2,oneway_gaze1,mutual_gaze2,mutual_gaze1,gaze_thresold,totalsess_time_forFR,PCAtwins,fps,
                                      data_saved_folder,cameraID,animal1_filename,animal2_filename,date_tgt)
            
            
            
            # do the spike triggered average of different bhv variables, for the single camera tracking, look at the pulling and social gaze actions
            # the goal is to get a sense for glm
            if 1: 
                print('plot spike triggered bhv variables')

                savefig = 1
                save_path = data_saved_folder+"fig_for_basic_neural_analysis_allsessions_basicEvents/"+cameraID+"/"+\
                            animal1_filename+"_"+animal2_filename+'_'+recordedanimal+'Recorded'+"/"+date_tgt
                if not os.path.exists(save_path):
                    os.makedirs(save_path)
                #
                do_shuffle = 0
                #
                min_length = np.shape(look_at_other_or_not_merge['dodson'])[0] # frame numbers of the video recording
                #
                trig_twins = [-4,4] # the time window to examine the spike triggered average, in the unit of s
                
                gaze_thresold = 0.2
                
                stg_twins = 3 # 3s, the behavioral event interval used to define strategy, consistent with DBN 3s time lags
                #
                spike_trig_average_all =  plot_spike_triggered_singlecam_bhvevent(date_tgt,savefig,save_path, animal1, animal2, session_start_time,min_length, trig_twins,
                                                                              stg_twins, time_point_pull1, time_point_pull2, time_point_pulls_succfail,
                                                                              oneway_gaze1,oneway_gaze2,mutual_gaze1,mutual_gaze2,gaze_thresold,animalnames_videotrack,
                                                                              spike_clusters_data, spike_time_data,spike_channels_data,do_shuffle)

                spike_trig_events_all_dates[date_tgt] = spike_trig_average_all

            
        # load filtered lfp
        if 0:
            print('load LFP data for '+neural_record_condition)
            lfp_filt_filename = neural_data_folder+neural_record_condition+'/lfp_filt_subsample.txt' # already downsample to 30Hz
            lfp_filt_data_df = genfromtxt(lfp_filt_filename, delimiter=',')
            # aligned to the session start
            lfp_filt_sess_aligned=lfp_filt_data_df[:,int(-neural_start_time_session_start_offset*30):]
            # normalize the activity to 0 - 1
            lfp_filt_sess_aligned = (lfp_filt_sess_aligned-np.min(lfp_filt_sess_aligned))/(np.max(lfp_filt_sess_aligned)-np.min(lfp_filt_sess_aligned))

        
        # plot the tracking demo video
        if 0: 
            print('make the demo videos')
            if 0:
                # all the bhv traces in the same panel
                tracking_video_singlecam_wholebody_withNeuron_demo(bodyparts_locs_camI,output_look_ornot,output_allvectors,output_allangles,
                                                  lever_locs_camI,tube_locs_camI,time_point_pull1,time_point_pull2,
                                                  animalnames_videotrack,bodypartnames_videotrack,date_tgt,
                                                  animal1_filename,animal2_filename,session_start_time,fps,nframes,cameraID,
                                                  video_file_original,sqr_thres_tubelever,sqr_thres_face,sqr_thres_body,
                                                  spike_time_data,lfp_filt_sess_aligned,spike_channels_data,channel_to_depth)
            if 1:
                # all the bhv traces are in separate panels
                tracking_video_singlecam_wholebody_withNeuron_sepbhv_demo(bodyparts_locs_camI,output_look_ornot,output_allvectors,output_allangles,
                                                 lever_locs_camI,tube_locs_camI,time_point_pull1,time_point_pull2,
                                                 animalnames_videotrack,bodypartnames_videotrack,date_tgt,
                                                 animal1_filename,animal2_filename,session_start_time,fps,nframes,cameraID,
                                                 video_file_original,sqr_thres_tubelever,sqr_thres_face,sqr_thres_body,
                                                 spike_time_data,lfp_filt_sess_aligned,spike_channels_data,channel_to_depth)
        
        # plot the example frame from the tracking demo video
        if 0: 
            print('print the example frame from the demo videos')
            if 1:
                example_frame = 60*30+1
                start_frame = 55*30
                # all the bhv traces are in separate panels
                tracking_frame_singlecam_wholebody_withNeuron_sepbhv_demo(bodyparts_locs_camI,output_look_ornot,output_allvectors,output_allangles,
                                                 lever_locs_camI,tube_locs_camI,time_point_pull1,time_point_pull2,
                                                 animalnames_videotrack,bodypartnames_videotrack,date_tgt,
                                                 animal1_filename,animal2_filename,session_start_time,fps,start_frame,example_frame,cameraID,
                                                 video_file_original,sqr_thres_tubelever,sqr_thres_face,sqr_thres_body,
                                                 spike_time_data,lfp_filt_sess_aligned,spike_channels_data,channel_to_depth)
                savefig = 1
                save_path = data_saved_folder+"fig_for_basic_neural_analysis_allsessions_basicEvents/"+cameraID+"/"+\
                            animal1_filename+"_"+animal2_filename+'_'+recordedanimal+'Recorded'+"/"+date_tgt+"/"
                if not os.path.exists(save_path):
                    os.makedirs(save_path)
                if savefig:
                    plt.savefig(save_path+'singlecam_wholebody_tracking_withNeuron_sepbhv_demo_oneframe.pdf')
        

    # save data
    if 0:
        
        data_saved_subfolder = data_saved_folder+'data_saved_singlecam_wholebody'+savefile_sufix+'/'+cameraID+'/'+animal1_fixedorders[0]+animal2_fixedorders[0]+'/'
        if not os.path.exists(data_saved_subfolder):
            os.makedirs(data_saved_subfolder)
                
        # with open(data_saved_subfolder+'/DBN_input_data_alltypes_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
        #     pickle.dump(DBN_input_data_alltypes, f)

        with open(data_saved_subfolder+'/owgaze1_num_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(owgaze1_num_all_dates, f)
        with open(data_saved_subfolder+'/owgaze2_num_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(owgaze2_num_all_dates, f)
        with open(data_saved_subfolder+'/mtgaze1_num_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(mtgaze1_num_all_dates, f)
        with open(data_saved_subfolder+'/mtgaze2_num_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(mtgaze2_num_all_dates, f)
        with open(data_saved_subfolder+'/pull1_num_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(pull1_num_all_dates, f)
        with open(data_saved_subfolder+'/pull2_num_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(pull2_num_all_dates, f)

        with open(data_saved_subfolder+'/tasktypes_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(tasktypes_all_dates, f)
        with open(data_saved_subfolder+'/coopthres_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(coopthres_all_dates, f)
        with open(data_saved_subfolder+'/succ_rate_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(succ_rate_all_dates, f)
        with open(data_saved_subfolder+'/interpullintv_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(interpullintv_all_dates, f)
        with open(data_saved_subfolder+'/trialnum_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(trialnum_all_dates, f)
        with open(data_saved_subfolder+'/bhv_intv_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(bhv_intv_all_dates, f)
            
        with open(data_saved_subfolder+'/pull_trig_events_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(pull_trig_events_all_dates, f) 
        with open(data_saved_subfolder+'/succpull_trig_events_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(succpull_trig_events_all_dates, f) 
        with open(data_saved_subfolder+'/failpull_trig_events_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(failpull_trig_events_all_dates, f) 
            
        with open(data_saved_subfolder+'/pull_infos_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(pull_infos_all_dates, f)             
            
        with open(data_saved_subfolder+'/spike_trig_events_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(spike_trig_events_all_dates, f)  
    
        with open(data_saved_subfolder+'/bhvevents_aligned_FR_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(bhvevents_aligned_FR_all_dates, f) 
        with open(data_saved_subfolder+'/bhvevents_aligned_FR_allevents_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(bhvevents_aligned_FR_allevents_all_dates, f) 
            
        with open(data_saved_subfolder+'/strategy_aligned_FR_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(strategy_aligned_FR_all_dates, f) 
        with open(data_saved_subfolder+'/strategy_aligned_FR_allevents_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(strategy_aligned_FR_allevents_all_dates, f) 
    
    
    # only save a subset 
    if 0:
        data_saved_subfolder = data_saved_folder+'data_saved_singlecam_wholebody'+savefile_sufix+'/'+cameraID+'/'+animal1_fixedorders[0]+animal2_fixedorders[0]+'/'
        if not os.path.exists(data_saved_subfolder):
            os.makedirs(data_saved_subfolder)
    
        with open(data_saved_subfolder+'/pull_infos_all_dates_'+animal1_fixedorders[0]+animal2_fixedorders[0]+'.pkl', 'wb') as f:
            pickle.dump(pull_infos_all_dates, f)   

   
    

In [ ]:
# pull_trig_events_all_dates['20240508'].keys()

In [ ]:
# pull_infos_all_dates['20240508'].keys()

In [ ]:
# pull_rts_all_dates['20240531']

#### re-organized the data
#### for the activity aligned at the different single behavioral events, mostly focus on self pull
#### the ultamate goal is to analyze the difference in single trial and if gaze related variables related to any of them
#### make some prilimary plot for sanity check

In [ ]:

# choose one pull_trig_events type to work with
# options: ['gaze_other_angle','gaze_tube_angle','gaze_lever_angle','animal_animal_dist',
#           'animal_tube_dist','animal_lever_dist','othergaze_self_angle',
#           'mass_move_speed','gaze_angle_speed','otherani_otherlever_dist',
#           'socialgaze_prob','othergaze_prob']
#
# pull_trig_events_tgtname = 'otherani_otherlever_dist' 
pull_trig_events_tgtname = 'socialgaze_prob' # for testing if individual trial different was from gaze start time
# pull_trig_events_tgtname = 'othergaze_prob' # if to test things aligned to partner's pull (in that case, the subject's gaze becomes othergaze) 


# Keep these as additional controls
pull_trig_otherpull_name = 'otherpull_prob'
pull_trig_selfpull_name = 'selfpull_prob'
pull_time_pre_reward_name = 'time_from_last_reward'
pull_num_pre_failpull_name = 'num_preceding_failpull'
pull_rt_name = 'pull_rt'


bhvevents_aligned_FR_allevents_all_dates_df = pd.DataFrame(columns=['dates','condition','act_animal','bhv_name',
                                                                    'succrate','clusterID',
                                                                    'channelID','FR_allevents'])
bhvevents_aligned_FR_all_dates_df = pd.DataFrame(columns=['dates','condition','act_animal','bhv_name',
                                                          'succrate','clusterID',
                                                          'channelID','FR_average'])

for idate in np.arange(0,ndates,1):
    date_tgt = dates_list[idate]
    task_condition = task_conditions[idate]

    succrate = succ_rate_all_dates[idate]
    
    bhv_types = list(bhvevents_aligned_FR_allevents_all_dates[date_tgt].keys())

    for ibhv_type in bhv_types:

        clusterIDs = list(bhvevents_aligned_FR_allevents_all_dates[date_tgt][ibhv_type].keys())

        ibhv_type_split = ibhv_type.split()
        if np.shape(ibhv_type_split)[0]==3:
            ibhv_type_split[1] = ibhv_type_split[1]+'_'+ibhv_type_split[2]

        # load the pull_trig_continuous_events
        if ibhv_type_split[1] == 'pull':
            try:
                pull_trig_events_tgt = pull_trig_events_all_dates[date_tgt][(ibhv_type_split[0],pull_trig_events_tgtname)]
                pull_trig_otherpull = pull_trig_events_all_dates[date_tgt][(ibhv_type_split[0],pull_trig_otherpull_name)]
                pull_trig_selfpull = pull_trig_events_all_dates[date_tgt][(ibhv_type_split[0],pull_trig_selfpull_name)]
                pull_trig_prerewardtime = np.array(list(pull_infos_all_dates[date_tgt][(ibhv_type_split[0],pull_time_pre_reward_name)]))
                pull_num_pre_failpull = np.array(list(pull_infos_all_dates[date_tgt][(ibhv_type_split[0],pull_num_pre_failpull_name)]))
                pull_rt = np.array(list(pull_rts_all_dates[date_tgt][ibhv_type_split[0]]))
                succornot_pull = (np.hstack([pull_num_pre_failpull[1:],0])==0)*1
            except:
                pull_trig_events_tgt = np.nan
                pull_trig_otherpull = np.nan
                pull_trig_selfpull = np.nan
                pull_trig_prerewardtime = np.nan
                pull_num_pre_failpull = np.nan
                pull_rt = np.nan
                succornot_pull = np.nan
        #
        elif ibhv_type_split[1] == 'succpull':
            try:
                pull_trig_events_tgt = succpull_trig_events_all_dates[date_tgt][(ibhv_type_split[0],pull_trig_events_tgtname)]
                pull_trig_otherpull = succpull_trig_events_all_dates[date_tgt][(ibhv_type_split[0],pull_trig_otherpull_name)]
                pull_trig_selfpull = succpull_trig_events_all_dates[date_tgt][(ibhv_type_split[0],pull_trig_selfpull_name)]
                pull_trig_prerewardtime = np.nan
                pull_num_pre_failpull = np.nan
                pull_rt = np.nan
                succornot_pull = np.nan
            except:
                pull_trig_events_tgt = np.nan
                pull_trig_otherpull = np.nan
                pull_trig_selfpull = np.nan
                pull_trig_prerewardtime = np.nan
                pull_num_pre_failpull = np.nan
                pull_rt = np.nan
                succornot_pull = np.nan
        #
        elif ibhv_type_split[1] == 'failpull':
            try:
                pull_trig_events_tgt = failpull_trig_events_all_dates[date_tgt][(ibhv_type_split[0],pull_trig_events_tgtname)]
                pull_trig_otherpull = failpull_trig_events_all_dates[date_tgt][(ibhv_type_split[0],pull_trig_otherpull_name)]
                pull_trig_selfpull = failpull_trig_events_all_dates[date_tgt][(ibhv_type_split[0],pull_trig_selfpull_name)]
                pull_trig_prerewardtime = np.nan
                pull_num_pre_failpull = np.nan
                pull_rt = np.nan
                succornot_pull = np.nan
            except:
                pull_trig_events_tgt = np.nan
                pull_trig_otherpull = np.nan
                pull_trig_selfpull = np.nan
                pull_trig_prerewardtime = np.nan
                pull_num_pre_failpull = np.nan
                pull_rt = np.nan
                succornot_pull = np.nan
        #
        else:
            pull_trig_events_tgt = np.nan
            
        for iclusterID in clusterIDs:   

            ichannelID = bhvevents_aligned_FR_allevents_all_dates[date_tgt][ibhv_type][iclusterID]['ch']
            iFR_average = bhvevents_aligned_FR_allevents_all_dates[date_tgt][ibhv_type][iclusterID]['FR_allevents']

            bhvevents_aligned_FR_allevents_all_dates_df = bhvevents_aligned_FR_allevents_all_dates_df.append({'dates': date_tgt, 
                                                                                    'condition':task_condition,
                                                                                    'act_animal':ibhv_type_split[0],
                                                                                    'bhv_name': ibhv_type_split[1],
                                                                                    'succrate':succrate,
                                                                                    'clusterID':iclusterID,
                                                                                    'channelID':ichannelID,
                                                                                    'FR_allevents':iFR_average,
                                                                                     pull_trig_events_tgtname:pull_trig_events_tgt,                          
                                                                                     pull_trig_otherpull_name:pull_trig_otherpull,                          
                                                                                     pull_trig_selfpull_name:pull_trig_selfpull,
                                                                                     pull_time_pre_reward_name:pull_trig_prerewardtime,
                                                                                     pull_num_pre_failpull_name:pull_num_pre_failpull,
                                                                                     'succornot_pull':succornot_pull,
                                                                                     pull_rt_name:pull_rt,
                                                                                    }, ignore_index=True)

            #
            ichannelID = bhvevents_aligned_FR_all_dates[date_tgt][ibhv_type][iclusterID]['ch']
            iFR_average = bhvevents_aligned_FR_all_dates[date_tgt][ibhv_type][iclusterID]['FR_average']

            bhvevents_aligned_FR_all_dates_df = bhvevents_aligned_FR_all_dates_df.append({'dates': date_tgt, 
                                                                                    'condition':task_condition,
                                                                                    'act_animal':ibhv_type_split[0],
                                                                                    'bhv_name': ibhv_type_split[1],
                                                                                    'succrate':succrate,
                                                                                    'clusterID':iclusterID,
                                                                                    'channelID':ichannelID,
                                                                                    'FR_average':iFR_average,
                                                                                   }, ignore_index=True)
                

In [ ]:
bhvevents_aligned_FR_allevents_all_dates_df

In [ ]:
bhvevents_aligned_FR_allevents_all_dates_df.columns

In [ ]:
np.unique(bhvevents_aligned_FR_allevents_all_dates_df['condition'])

In [ ]:
# act_animals_to_ana = np.unique(bhvevents_aligned_FR_allevents_all_dates_df['act_animal'])
# act_animals_to_ana = ['kanga']
act_animals_to_ana = ['dodson']
# act_animals_to_ana = ['kanga_partner'] # align thing to partner, remember to pair this with pull_trig_events_tgtname = 'othergaze_prob'
# act_animals_to_ana = ['kanga','kanga_partner']
# act_animals_to_ana = ['dodson','dodson_partner']

nanimal_to_ana = np.shape(act_animals_to_ana)[0]
#
# bhv_names_to_ana = np.unique(bhvevents_aligned_FR_allevents_all_dates_df['bhv_name'])
# bhv_names_to_ana = ['succpull','failpull']
# bhv_names_to_ana = ['succpull']
# bhv_names_to_ana = ['failpull']
bhv_names_to_ana = ['pull']
# bhv_names_to_ana = ['gazestart']
nbhvnames_to_ana = np.shape(bhv_names_to_ana)[0]
bhvname_clrs = ['r','y','g','b','c','m','#458B74','#FFC710','#FF1493','#A9A9A9','#8B4513']

#
conditions_to_ana = np.unique(bhvevents_aligned_FR_allevents_all_dates_df['condition'])
# conditions_to_ana = ['MC',]
# conditions_to_ana = ['SR']
# conditions_to_ana = ['MC_DannonAuto']
#
# for Kanga only
# conditions_to_ana = ['MC', 'MC_DannonAuto', 'MC_KangaAuto', 'MC_withDodson','MC_withGinger', 
#                      'MC_withKoala', 'MC_withVermelho', 'NV', ]
# conditions_to_ana = ['MC', 'MC_withDodson','MC_withGinger', 
#                      'MC_withKoala', 'MC_withVermelho', 'SR', 'SR_withDodson' ]
# 
# for dodson only
# conditions_to_ana = ['MC', 'MC_DodsonAuto_withKoala', 'MC_KoalaAuto_withKoala',
#                       'MC_withGingerNew', 'MC_withKanga', 'MC_withKoala',  ]
#
# conditions_to_ana = ['MC', 
#               'MC_withGingerNew', 'MC_withKanga', 'MC_withKoala', 'SR', 'SR_withGingerNew', 'SR_withKanga',
#              'SR_withKoala',  ]



nconds_to_ana = np.shape(conditions_to_ana)[0]

### basic basic sanity check plot, bhv event aligned mean FR 

In [ ]:
if 1:   
        
    # prepare the data
    mean_FR_df = pd.DataFrame(columns=['dates','condition','act_animal','bhv_name',
                                       'clusterID','meanFR'])
    
    # load the data 
    for icond_ana in np.arange(0,nconds_to_ana,1):
        cond_ana = conditions_to_ana[icond_ana]
        ind_cond = bhvevents_aligned_FR_allevents_all_dates_df['condition']==cond_ana

        for ianimal_ana in np.arange(0,nanimal_to_ana,1):
            act_animal_ana = act_animals_to_ana[ianimal_ana]
            
            if not 'partner' in act_animal_ana:
                ind_animal = bhvevents_aligned_FR_allevents_all_dates_df['act_animal']==act_animal_ana
            elif 'partner' in act_animal_ana:
                animal_dontwant = act_animal_ana.split('_')[0]
                ind_animal = bhvevents_aligned_FR_allevents_all_dates_df['act_animal']!=animal_dontwant
                
            # get the dates
            dates_ana = np.unique(bhvevents_aligned_FR_allevents_all_dates_df[ind_animal & ind_cond]['dates'])
            ndates_ana = np.shape(dates_ana)[0]

            for idate_ana in np.arange(0,ndates_ana,1):
                date_ana = dates_ana[idate_ana]
                ind_date = bhvevents_aligned_FR_allevents_all_dates_df['dates']==date_ana

                # get the neurons 
                neurons_ana = np.unique(bhvevents_aligned_FR_allevents_all_dates_df[ind_animal & ind_cond & ind_date]['clusterID'])
                nneurons = np.shape(neurons_ana)[0]

                for ineuron in np.arange(0,nneurons,1):
                    clusterID_ineuron = neurons_ana[ineuron]
                    ind_neuron = bhvevents_aligned_FR_allevents_all_dates_df['clusterID']==clusterID_ineuron


                    for ibhvname_ana in np.arange(0,nbhvnames_to_ana,1):
                        bhvname_ana = bhv_names_to_ana[ibhvname_ana]
                        ind_bhv = bhvevents_aligned_FR_allevents_all_dates_df['bhv_name']==bhvname_ana

                        ind_ana = ind_animal & ind_bhv & ind_cond & ind_neuron & ind_date 

                        bhvevents_aligned_FR_allevents_tgt = bhvevents_aligned_FR_allevents_all_dates_df[ind_ana]

                        #
                        # load and plot bhv event ('pull') aligned FR
                        FRs_allevents_ineuron = np.array(bhvevents_aligned_FR_allevents_tgt['FR_allevents'])[0]

                        nevents = np.shape(FRs_allevents_ineuron)[1]
                        
                        if nevents > 0:
                            FRsmoothed_allevents_ineuron = gaussian_filter1d(FRs_allevents_ineuron, sigma=6, axis=0)
                            
                            # Compute mean and SEM while ignoring NaNs
                            mean_trace = np.nanmean(FRsmoothed_allevents_ineuron, axis=1)
                            time_trace = np.arange(-4,4,1/fps)  # Assuming time is just indices
                        #
                        else:
                            FRsmoothed_allevents_ineuron = np.nan
                            time_trace = np.arange(-4,4,1/fps)  # Assuming time is just indices
                            mean_trace = np.ones(np.shape(time_trace))*np.nan
                            
                        #
                        mean_FR_df = mean_FR_df.append({'dates': date_ana, 
                                                        'condition':cond_ana,
                                                        'act_animal':act_animal_ana,
                                                        'bhv_name': bhvname_ana,
                                                        'clusterID':clusterID_ineuron,
                                                        'meanFR':mean_trace,     
                                                       }, ignore_index=True)
                            
    
    # do the plotting
    
    #####
    # to make the condition more general
    # Define the function for generalizing condition
    def generalize_condition(cond):
        if cond == "MC" or cond.startswith("MC_with"):
            return "MC"
        elif cond == "SR" or cond.startswith("SR_with"):
            return "SR"
        elif cond == "NV" or cond.startswith("NV_with"):
            return "NV"
        else:
            return cond  # default to original condition if no match

    # Apply the function to create the new column
    mean_FR_df["condition_general"] = mean_FR_df["condition"].apply(generalize_condition)
    #####
    
    # Get unique animals and generalized conditions
    animals = mean_FR_df['act_animal'].unique()
    conditions_general_toplot = np.unique(mean_FR_df['condition_general'])

    # Count subplots needed
    ncols = len(conditions_general_toplot)
    nrows = len(animals)

    fig = plt.figure(figsize=(6 * ncols, 4 * nrows))
    gs = gridspec.GridSpec(nrows, ncols + 1, width_ratios=[1] * ncols + [0.05], wspace=0.4, hspace=0.6)

    # Determine global color limits across all data
    all_vmax, all_vmin = [], []

    for animal in animals:
        for cond in conditions_general_toplot:
            df_sub = mean_FR_df[(mean_FR_df['act_animal'] == animal) &
                                (mean_FR_df['condition_general'] == cond)]
            if not df_sub.empty:
                meanFR_matrix = np.vstack(df_sub['meanFR'].values)
                all_vmax.append(np.max(meanFR_matrix))
                all_vmin.append(np.min(meanFR_matrix))

    global_vmax = max(all_vmax)
    global_vmin = min(all_vmin)
    center_val = 0 if global_vmin < 0 else np.mean([global_vmin, global_vmax])
    vmax = global_vmax
    vmin = 2 * center_val - vmax

    # Start plotting
    for i_animal, animal in enumerate(animals):
        for i_cond, cond in enumerate(conditions_general_toplot):
            ax = fig.add_subplot(gs[i_animal, i_cond])
            df_sub = mean_FR_df[(mean_FR_df['act_animal'] == animal) &
                                (mean_FR_df['condition_general'] == cond)]

            if df_sub.empty:
                ax.axis('off')
                continue

            meanFR_matrix = np.vstack(df_sub['meanFR'].values)
            peak_times = np.argmax(meanFR_matrix, axis=1)
            sort_idx = np.argsort(peak_times)
            meanFR_sorted = meanFR_matrix[sort_idx, :]

            im = ax.imshow(meanFR_sorted,
                           aspect='auto',
                           cmap='seismic',
                           extent=[time_trace[0], time_trace[-1], 0, meanFR_sorted.shape[0]],
                           interpolation='nearest',
                           vmin=vmin,
                           vmax=vmax)

            ax.axvline(0, color='k', linestyle='--', linewidth=1)
            ax.set_title(f'{animal} - {cond}')
            ax.set_xlabel('Time (s)')
            if i_cond == 0:
                ax.set_ylabel('Neuron index')

    # Add colorbar in the last column
    cax = fig.add_subplot(gs[:, -1])
    cbar = fig.colorbar(im, cax=cax)
    cbar.set_label('Mean Firing Rate (Hz)')

    plt.tight_layout()
    plt.show()
    
    #                        
    savefig = 0
    if savefig:
        figsavefolder = data_saved_folder+"fig_for_basic_neural_analysis_allsessions_basicEvents_PCA_makeBhvNeuronVideos_Pullfocused_continuousBhv/"+\
                        cameraID+"/"+animal1_filenames[0]+"_"+animal2_filenames[0]+"/FR_and_gaze_quantile_fig/"    

        if not os.path.exists(figsavefolder):
            os.makedirs(figsavefolder)

        fig.savefig(figsavefolder+'bhvevents_aligned_averaged_FR_across_allevents_'+bhvname_ana+
                     savefile_sufix+'.pdf')

### sanity check plot; mean self pull aligned gaze

In [ ]:
if 0:   
    
    sanitycheck_bhv_toplot_name = 'socialgaze_prob'
    
    # prepare the data
    sanitycheck_bhv_df = pd.DataFrame(columns=['dates','condition','act_animal','bhv_name', 'bhvID', ])
        
    # load the data 
    for icond_ana in np.arange(0,nconds_to_ana,1):
        cond_ana = conditions_to_ana[icond_ana]
        ind_cond = bhvevents_aligned_FR_allevents_all_dates_df['condition']==cond_ana

        for ianimal_ana in np.arange(0,nanimal_to_ana,1):
            act_animal_ana = act_animals_to_ana[ianimal_ana]
            
            if not 'partner' in act_animal_ana:
                ind_animal = bhvevents_aligned_FR_allevents_all_dates_df['act_animal']==act_animal_ana
            elif 'partner' in act_animal_ana:
                animal_dontwant = act_animal_ana.split('_')[0]
                ind_animal = bhvevents_aligned_FR_allevents_all_dates_df['act_animal']!=animal_dontwant
                
            # get the dates
            dates_ana = np.unique(bhvevents_aligned_FR_allevents_all_dates_df[ind_animal & ind_cond]['dates'])
            ndates_ana = np.shape(dates_ana)[0]

            for idate_ana in np.arange(0,ndates_ana,1):
                date_ana = dates_ana[idate_ana]
                ind_date = bhvevents_aligned_FR_allevents_all_dates_df['dates']==date_ana

                # get the neurons 
                neurons_ana = np.unique(bhvevents_aligned_FR_allevents_all_dates_df[ind_animal & ind_cond & ind_date]['clusterID'])
                nneurons = np.shape(neurons_ana)[0]

                # 
                # only one neuron since looking at bhv only
                # for ineuron in np.arange(0,nneurons,1):
                for ineuron in np.arange(0,1,1):
                    clusterID_ineuron = neurons_ana[ineuron]
                    ind_neuron = bhvevents_aligned_FR_allevents_all_dates_df['clusterID']==clusterID_ineuron


                    for ibhvname_ana in np.arange(0,nbhvnames_to_ana,1):
                        bhvname_ana = bhv_names_to_ana[ibhvname_ana]
                        ind_bhv = bhvevents_aligned_FR_allevents_all_dates_df['bhv_name']==bhvname_ana

                        ind_ana = ind_animal & ind_bhv & ind_cond & ind_neuron & ind_date 

                        bhvevents_allevents_tgt = bhvevents_aligned_FR_allevents_all_dates_df[ind_ana]
                        
                        sanitycheck_bhv = np.array(bhvevents_allevents_tgt[sanitycheck_bhv_toplot_name])[0]
                        
                        #
                        nevents = np.shape(sanitycheck_bhv)[0]
                        
                        
                        for ievent in np.arange(0,nevents,1):
                            
                            # calculate the social gaze accumulation
                            if sanitycheck_bhv_toplot_name == 'socialgaze_prob':
                                
                                gaze_prob = sanitycheck_bhv[ievent]
                                
                                try:
                                    timewins = np.arange(-4,4,1/fps)
                                    dt = 1/fps
                                    gaze_accum = np.trapz(gaze_prob[timewins<0], dx=dt)
                                except:
                                    gaze_accum = np.nan
                            else:
                                gaze_accum = np.nan
                                
                            # remove trials that has no gaze_accum
                            if gaze_accum == 0:
                                gaze_accum = np.nan
                            
                            
                            sanitycheck_bhv_df = sanitycheck_bhv_df.append({'dates': date_ana, 
                                                                            'condition':cond_ana,
                                                                            'act_animal':act_animal_ana,
                                                                            'bhv_name': bhvname_ana,
                                                                            'bhvID':ievent,
                                                                            sanitycheck_bhv_toplot_name:sanitycheck_bhv[ievent],     
                                                                            'gaze_accum':gaze_accum,
                                                                           }, ignore_index=True)
            
    # do the plotting
    
    #####
    # to make the condition more general
    # Define the function for generalizing condition
    def generalize_condition(cond):
        if cond == "MC" or cond.startswith("MC_with"):
            return "MC"
        elif cond == "SR" or cond.startswith("SR_with"):
            return "SR"
        elif cond == "NV" or cond.startswith("NV_with"):
            return "NV"
        else:
            return cond  # default to original condition if no match

    # Apply the function to create the new column
    sanitycheck_bhv_df["condition_general"] = sanitycheck_bhv_df["condition"].apply(generalize_condition)
    #####
    
    # 
    onlyMCSR = 1
    if onlyMCSR:
        ind_MCSR = np.isin(sanitycheck_bhv_df['condition_general'],['MC','SR'])
        sanitycheck_bhv_df = sanitycheck_bhv_df[ind_MCSR]
                        
    # Group by condition_general and calculate mean ± sem (standard error of mean)
    grouped = sanitycheck_bhv_df.groupby('condition_general')[sanitycheck_bhv_toplot_name].apply(list)

    # Prepare plot
    figg = plt.figure(figsize=(6, 6))

    for cond, traces in grouped.items():
        # Convert list of lists to a 2D numpy array: shape = (num_trials, trace_length)
        traces_array = np.array(traces)
        mean_trace = np.nanmean(traces_array,axis=0)
        sem_trace = np.nanstd(traces_array,axis=0) / np.sqrt(traces_array.shape[0])

        x = np.arange(-4,4,1/fps)
        plt.plot(x, mean_trace, label=cond)
        plt.fill_between(x, mean_trace - sem_trace, mean_trace + sem_trace, alpha=0.3)
        
    plt.plot([0,0],[-0.01,0.11],'--')
    plt.xlim([-4.2,4.2])
    # plt.ylim([-0.01,0.11])
        
    plt.xlabel("time (s)")
    plt.ylabel(sanitycheck_bhv_toplot_name)
    plt.title("Mean Social Gaze Trace by Condition")
    plt.legend()
    plt.tight_layout()

    
    #
    ind_MC = np.isin(sanitycheck_bhv_df['condition_general'],['MC'])
    sanitycheck_bhv_MC_df = sanitycheck_bhv_df[ind_MC].copy()
    
    
    
    
    
    #                        
    savefig = 0
    if savefig:
        figsavefolder = data_saved_folder+"fig_for_basic_neural_analysis_allsessions_basicEvents_PCA_makeBhvNeuronVideos_Pullfocused_continuousBhv/"+\
                        cameraID+"/"+animal1_filenames[0]+"_"+animal2_filenames[0]+"/"    

        if not os.path.exists(figsavefolder):
            os.makedirs(figsavefolder)

        figg.savefig(figsavefolder+bhvname_ana+'_aligned_'+sanitycheck_bhv_toplot_name+
                     savefile_sufix+'.pdf')

### sanity check plot; mean pull aligned firing rate and pull aligned gaze events (3 gaussian kernel smoothed)
#### add the option to look at gaze accumulation over time
#### also use this code to defined significant neurons - label neurons that significantly encode gaze accumulation before pull, this is for later analysis

In [ ]:
if 1:   
    # gaze_duration_type = 'before_pull'  # 'around_pull', 'before_pull', 'after_pull'
    gaze_duration_type = 'around_pull'  # 'around_pull', 'before_pull', 'after_pull'
    
    # calculate the gaze accumulation if the condition allows (calculate the auc)
    doGazeAccum = 0
        
    significant_neurons_data_df = pd.DataFrame(columns=['dates','condition','act_animal','bhv_name',
                                                        'clusterID','significance_or_not',
                                                        'gaze_duration_type','gaze_variable_name'])
    
    # load the data 
    for icond_ana in np.arange(0,nconds_to_ana,1):
        cond_ana = conditions_to_ana[icond_ana]
        ind_cond = bhvevents_aligned_FR_allevents_all_dates_df['condition']==cond_ana

        for ianimal_ana in np.arange(0,nanimal_to_ana,1):
            act_animal_ana = act_animals_to_ana[ianimal_ana]
            
            if not 'partner' in act_animal_ana:
                ind_animal = bhvevents_aligned_FR_allevents_all_dates_df['act_animal']==act_animal_ana
            elif 'partner' in act_animal_ana:
                animal_dontwant = act_animal_ana.split('_')[0]
                ind_animal = bhvevents_aligned_FR_allevents_all_dates_df['act_animal']!=animal_dontwant

            
            # get the dates
            dates_ana = np.unique(bhvevents_aligned_FR_allevents_all_dates_df[ind_animal & ind_cond]['dates'])
            ndates_ana = np.shape(dates_ana)[0]

            for idate_ana in np.arange(0,ndates_ana,1):
                date_ana = dates_ana[idate_ana]
                ind_date = bhvevents_aligned_FR_allevents_all_dates_df['dates']==date_ana

                # get the neurons 
                neurons_ana = np.unique(bhvevents_aligned_FR_allevents_all_dates_df[ind_animal & ind_cond & ind_date]['clusterID'])
                nneurons = np.shape(neurons_ana)[0]

                # Determine subplot grid (5 columns, dynamic rows)
                ncols = 5
                nrows = int(np.ceil(nneurons / ncols))

                fig, axes = plt.subplots(nrows, ncols, figsize=(ncols * 6, nrows * 6), constrained_layout=True)
                axes = np.ravel(axes)  # Flatten for easy indexing
                
                # === New heatmap plot per date for neuron correlation over time ===
                fig_corr, ax_corr = plt.subplots(figsize=(10, max(6, 0.3 * nneurons)))

                # Store r_trace and p_trace for each neuron
                r_traces_all_neurons = []
                p_traces_all_neurons = []

                for ineuron in np.arange(0,nneurons,1):
                    clusterID_ineuron = neurons_ana[ineuron]
                    ind_neuron = bhvevents_aligned_FR_allevents_all_dates_df['clusterID']==clusterID_ineuron

                    ax = axes[ineuron]  # Get the subplot for this neuron

                    for ibhvname_ana in np.arange(0,nbhvnames_to_ana,1):
                        bhvname_ana = bhv_names_to_ana[ibhvname_ana]
                        ind_bhv = bhvevents_aligned_FR_allevents_all_dates_df['bhv_name']==bhvname_ana

                        ind_ana = ind_animal & ind_bhv & ind_cond & ind_neuron & ind_date 

                        bhvevents_aligned_FR_allevents_tgt = bhvevents_aligned_FR_allevents_all_dates_df[ind_ana]

                        #
                        # load and plot bhv event ('pull') aligned FR
                        FRs_allevents_ineuron = np.array(bhvevents_aligned_FR_allevents_tgt['FR_allevents'])[0]

                        nevents = np.shape(FRs_allevents_ineuron)[1]
                        
                        if nevents > 0:
                            FRsmoothed_allevents_ineuron = gaussian_filter1d(FRs_allevents_ineuron, sigma=6, axis=0)

                            # Compute mean and SEM while ignoring NaNs
                            mean_trace = np.nanmean(FRsmoothed_allevents_ineuron, axis=1)
                            std_trace = np.nanstd(FRsmoothed_allevents_ineuron, axis=1)
                            sem_trace = std_trace / np.sqrt(nevents)  # Standard error of the mean

                            # Plot the results
                            time_trace = np.arange(-4,4,1/fps)  # Assuming time is just indices

                            # Plot each behavior as a separate trace
                            ax.plot(time_trace, mean_trace, label=bhvname_ana+'(n='+str(nevents)+')', 
                                    color=bhvname_clrs[ibhvname_ana])
                            ax.fill_between(time_trace, mean_trace - sem_trace, mean_trace + sem_trace, 
                                            color=bhvname_clrs[ibhvname_ana], alpha=0.3)
                        #
                        else:
                            FRsmoothed_allevents_ineuron = np.nan
                            
                        #
                        # load and plot the pull aligned continuous bhv variables
                        conBhv_allevents_ineuron = np.array(bhvevents_aligned_FR_allevents_tgt[pull_trig_events_tgtname])[0]
                        conBhv_allevents_ineuron = np.array(conBhv_allevents_ineuron)
                        conBhv_allevents_ineuron = conBhv_allevents_ineuron.transpose()
                        
                        #
                        # calculate the gaze accumulation if the condition allows (calculate the auc)
                        if doGazeAccum:
                            from sklearn.metrics import auc
                            if (pull_trig_events_tgtname == 'socialgaze_prob') |\
                               (pull_trig_events_tgtname == 'othergaze_prob') :
                                num_time_points = conBhv_allevents_ineuron.shape[0]
                                num_conbhv = conBhv_allevents_ineuron.shape[1]
                                accumulated_auc = np.zeros((num_time_points, num_conbhv))
                                # Create a time axis (assuming equal spacing)
                                time_ind = np.arange(num_time_points)
                                #
                                for i in range(num_conbhv):
                                    for t in range(1, num_time_points):
                                        # Calculate AUC up to the current time point for the i-th conbhv
                                        y = conBhv_allevents_ineuron[:t+1, i]
                                        accumulated_auc[t, i] = auc(time_ind[:t+1], y)
                            #
                            conBhv_allevents_ineuron = accumulated_auc
                            
                        # zscored the behavioral events
                        # Flatten the data
                        flattened = conBhv_allevents_ineuron.flatten()
                        # Z-score the entire dataset as a single distribution
                        flattened_z = np.full_like(flattened, np.nan)
                        valid_mask = ~np.isnan(flattened)
                        flattened_z[valid_mask] = st.zscore(flattened[valid_mask])
                        # Reshape back to original shape
                        conBhv_allevents_ineuron_z = flattened_z.reshape(conBhv_allevents_ineuron.shape)
                        # 
                        conBhv_allevents_ineuron = conBhv_allevents_ineuron_z
    
                        try:
                            nevents = np.shape(conBhv_allevents_ineuron)[1]
                        except:
                            nevents = 0
                        
                        try:
                            FRconBhv_allevents_ineuron = gaussian_filter1d(conBhv_allevents_ineuron, sigma=6, axis=0)
                        except:
                            FRconBhv_allevents_ineuron = np.nan
                            
                        # if the pull aligned FR and bhv have different number
                        try:
                            nevents_fr = np.shape(FRs_allevents_ineuron)[1]
                        except:
                            nevents_fr = 0
                            
                        if not  nevents_fr == nevents: 
                            print(date_ana+' mismatched number')
                            if nevents_fr < nevents:
                                FRconBhv_allevents_ineuron = FRconBhv_allevents_ineuron[:,0:nevents_fr]
                            else:
                                FRs_allevents_ineuron = FRs_allevents_ineuron[:,0:nevents]
                            
                        
                        # Compute correlation coefficient between FR and behavior at each time point
                        try:
                            corrs = np.full(FRsmoothed_allevents_ineuron.shape[0], np.nan)
                            pvals = np.full(FRsmoothed_allevents_ineuron.shape[0], np.nan)

                            for t in range(FRsmoothed_allevents_ineuron.shape[0]):
                                fr_t = FRsmoothed_allevents_ineuron[t, :]
                                bhv_t = FRconBhv_allevents_ineuron[t, :]

                                valid_mask = ~np.isnan(fr_t) & ~np.isnan(bhv_t)
                                if np.sum(valid_mask) > 5:  # Only compute if enough data points
                                    r, p = st.pearsonr(fr_t[valid_mask], bhv_t[valid_mask])
                                    corrs[t] = r
                                    pvals[t] = p    
                        except:
                            time_trace = np.arange(-4,4,1/fps)
                            corrs = np.full(time_trace.shape[0], np.nan)
                            pvals = np.full(time_trace.shape[0], np.nan)


                        r_traces_all_neurons.append(corrs)
                        p_traces_all_neurons.append(pvals)

                        # decide if this neuron is significant or not
                        if gaze_duration_type == 'around_pull':
                            significant_neuron = np.sum(pvals<0.01)>0
                        elif gaze_duration_type == 'before_pull':
                            significant_neuron = np.sum(pvals[time_trace<0]<0.01)>0
                        elif gaze_duration_type == 'after_pull':
                            significant_neuron = np.sum(pvals[time_trace>0]<0.01)>0
                                                
                        #
                        # put information about the significance 
                        if doGazeAccum:
                            significant_neurons_data_df = significant_neurons_data_df.append({'dates': date_ana, 
                                                                                    'condition':cond_ana,
                                                                                    'act_animal':act_animal_ana,
                                                                                    'bhv_name': bhvname_ana,
                                                                                    'clusterID':clusterID_ineuron,
                                                                                    'significance_or_not':significant_neuron,
                                                                                    'gaze_duration_type':gaze_duration_type,
                                                                                    'gaze_variable_name':'gaze_accum',     
                                                                                   }, ignore_index=True)
                        else:
                            significant_neurons_data_df = significant_neurons_data_df.append({'dates': date_ana, 
                                                                                    'condition':cond_ana,
                                                                                    'act_animal':act_animal_ana,
                                                                                    'bhv_name': bhvname_ana,
                                                                                    'clusterID':clusterID_ineuron,
                                                                                    'significance_or_not':significant_neuron,
                                                                                    'gaze_duration_type':gaze_duration_type,
                                                                                    'gaze_variable_name':'socialgaze_prob',     
                                                                                   }, ignore_index=True)
                        
                        
                        if nevents > 0:
                            # Compute mean and SEM while ignoring NaNs
                            mean_trace = np.nanmean(FRconBhv_allevents_ineuron, axis=1)
                            std_trace = np.nanstd(FRconBhv_allevents_ineuron, axis=1)
                            sem_trace = std_trace / np.sqrt(nevents)  # Standard error of the mean

                            # Plot each behavior as a separate trace
                            if doGazeAccum:
                                ax.plot(time_trace, mean_trace, label='pull_trig_'+pull_trig_events_tgtname+'_AUC(n='+str(nevents)+')', 
                                        color='#808080')
                                ax.fill_between(time_trace, mean_trace - sem_trace, mean_trace + sem_trace, 
                                                color='#808080', alpha=0.3)
                            else:
                                ax.plot(time_trace, mean_trace, label='pull_trig_'+pull_trig_events_tgtname+'(n='+str(nevents)+')', 
                                        color='#808080')
                                ax.fill_between(time_trace, mean_trace - sem_trace, mean_trace + sem_trace, 
                                                color='#808080', alpha=0.3)


                            # Create a twin axis for the correlation plot
                            ax2 = ax.twinx()                   

                            # Plot correlation coefficient trace on the right y-axis
                            ax2.plot(time_trace, corrs, color='black', linestyle='--', label='FR–Bhv r')
                            ax2.set_ylabel("Correlation (r)", color='black')

                            # Highlight significant timepoints (p < 0.01) with red dots on the right y-axis
                            significant_mask = (pvals < 0.01) & ~np.isnan(pvals)
                            ax2.plot(time_trace[significant_mask], corrs[significant_mask], 'ro', label='p < 0.01')

                            # Set the label for the right axis
                            ax2.set_ylabel("Correlation (r)", color='black')
                            ax2.tick_params(axis='y', labelcolor='black')

                            # Optionally adjust limits or formatting if necessary
                            ax2.set_ylim(-1, 1)  # Adjust this as necessary for your data range


                    ax.set_title(f"Neuron {clusterID_ineuron}")
                    ax.set_xlabel("Time (s)")
                    ax.set_ylabel("Firing Rate (a.u.)")
                    # ax.set_title(act_animal_ana+' '+cond_ana+' '+date_ana+' cell#'+clusterID_ineuron)
                    ax.legend()

                # Hide empty subplots if nneurons < total grid size
                for i in range(nneurons, len(axes)):
                    fig.delaxes(axes[i])

                # Figure title
                fig.suptitle(f"{act_animal_ana} {cond_ana} {date_ana}", fontsize=14)

                
                # Convert to numpy array for heatmap
                r_traces_all_neurons = np.array(r_traces_all_neurons)

                # === Sort r_traces by the time of their first peak ===
                peak_times = []
                for trace in r_traces_all_neurons:
                    if np.all(np.isnan(trace)):
                        peak_times.append(np.inf)
                    else:
                        peak_idx = np.nanargmax(trace)
                        peak_times.append(time_trace[peak_idx])

                # Get sorting indices based on peak times
                sorted_indices = np.argsort(peak_times)
                r_traces_sorted = r_traces_all_neurons[sorted_indices, :]

                # Plot heatmap of r values
                im = ax_corr.imshow(r_traces_sorted, aspect='auto', cmap='gray_r', interpolation='none',
                                    extent=[time_trace[0], time_trace[-1], 0, nneurons],
                                    vmin=-0.7, vmax=0.7)

                # Overlay significance as red dots
                for i, idx in enumerate(sorted_indices):
                    sig_times = np.where(p_traces_all_neurons[idx] < 0.01)[0]
                    for t in sig_times:
                        ax_corr.plot(time_trace[t], i + 0.5, 'r.', markersize=3)  # i+0.5 to center in the row

                # Add vertical dashed line at time = 0
                ax_corr.axvline(x=0, color='black', linestyle='--', linewidth=1)

                # Add vertical dashed line at time zero
                ax_corr.axvline(x=0, linestyle='--', color='gray', linewidth=1)

                ax_corr.set_xlabel("Time (s)")
                ax_corr.set_ylabel("Neuron (sorted by peak time)")
                ax_corr.set_title(f"Neuron-wise Corr(Gaze, FR) Heatmap (Sorted): {act_animal_ana} {cond_ana} {date_ana}")
                cbar = fig_corr.colorbar(im, ax=ax_corr)
                cbar.set_label('Pearson r')
                
                # plt.show()
                
                savefig = 0
                if savefig:
                    figsavefolder = data_saved_folder+"fig_for_basic_neural_analysis_allsessions_basicEvents_PCA_makeBhvNeuronVideos_Pullfocused_continuousBhv/"+cameraID+"/"+animal1_filenames[0]+"_"+animal2_filenames[0]+'/'+date_ana+'/'

                    if not os.path.exists(figsavefolder):
                        os.makedirs(figsavefolder)

                    if doGazeAccum:
                        fig.savefig(figsavefolder+'individualneurons_meanFR_and_mean_'+bhvname_ana+'_'+
                                     pull_trig_events_tgtname+'_auc'+savefile_sufix+'.pdf')

                        fig_corr.savefig(figsavefolder + 'neuron_corr_heatmap_FR_and_' + bhvname_ana+'_'+
                                         pull_trig_events_tgtname+'_auc'+savefile_sufix+'.pdf')
                    else:
                        fig.savefig(figsavefolder+'individualneurons_meanFR_and_mean_'+bhvname_ana+'_'+
                                     pull_trig_events_tgtname+savefile_sufix+'.pdf')

                        fig_corr.savefig(figsavefolder + 'neuron_corr_heatmap_FR_and_' + bhvname_ana+'_'+
                                         pull_trig_events_tgtname+savefile_sufix+'.pdf')

                # Close the figures to avoid memory issues
                plt.close(fig)
                plt.close(fig_corr)



In [ ]:
significant_neurons_data_df

In [ ]:
if 0:
    from scipy.integrate import cumtrapz  # Add this import
    from matplotlib.patches import Patch
    
    ind1 = bhvevents_aligned_FR_allevents_all_dates_df['condition']=='MC_withGinger'
    ind2 = bhvevents_aligned_FR_allevents_all_dates_df['dates']=='20240808'
    ind3 = bhvevents_aligned_FR_allevents_all_dates_df['bhv_name'] == 'pull'
    ind4 = bhvevents_aligned_FR_allevents_all_dates_df['act_animal']=='kanga'
    conBhv_allevents_ineuron = np.array(bhvevents_aligned_FR_allevents_all_dates_df[ind1&ind2&ind3&ind4][pull_trig_events_tgtname])[0]
    
    np.shape(conBhv_allevents_ineuron)
    
    # Setup
    trace = conBhv_allevents_ineuron[6]
    time_trace = np.arange(-4, 4, 1/30)
    
    # try to only use the meaningful gazes
    # filtered_trace = keep_closest_cluster_single_trial(trace, time_trace)
    # trace = filtered_trace

    # Compute accumulated AUC using trapezoidal integration
    accum_auc = cumtrapz(trace, time_trace, initial=0)

    # Create plot with dual y-axis
    fig, ax1 = plt.subplots(figsize=(8, 4))

    # Plot the gaze distribution
    color1 = 'tab:blue'
    line1, = ax1.plot(time_trace, trace, color=color1, label='Gaze Distribution')
    fill = ax1.fill_between(time_trace, trace, alpha=0.3, color=color1)
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Gaze Distribution', color=color1)
    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.axvline(0, color='k', linestyle='--', linewidth=1)
    ax1.set_title('Social gaze aligned pull')

    # Create a second y-axis for accumulated AUC
    ax2 = ax1.twinx()
    color2 = 'tab:red'
    line2, = ax2.plot(time_trace, accum_auc, color=color2, label='Accumulated AUC')
    ax2.set_ylabel('Accumulated AUC', color=color2)
    ax2.tick_params(axis='y', labelcolor=color2)

    # Legend: lines and manual patch for shaded area
    legend_elements = [
        line1,
        Patch(facecolor=color1, alpha=0.3, label='AUC Area'),
        line2
    ]
    ax1.legend(handles=legend_elements, loc='upper right')

    fig.tight_layout()
    
    savefig = 0
    if savefig:
        figsavefolder = data_saved_folder+"fig_for_basic_neural_analysis_allsessions_basicEvents_PCA_makeBhvNeuronVideos_Pullfocused_continuousBhv/"+cameraID+"/"+animal1_filenames[0]+"_"+animal2_filenames[0]+'/'

        if not os.path.exists(figsavefolder):
            os.makedirs(figsavefolder)

        fig.savefig(figsavefolder+'example_event_pull_gaze_accum.pdf')

        

#### run PCA on the neuron space, run different days separately for each condition
#### for the activity aligned at the different bhv events
#### run PCA for all bhvevent together combined

In [ ]:
if 1:
        
    # Step 1 - run PCA separately
    # save the simple PCA data
    FRPCA_all_sessions_allevents_sum_df = pd.DataFrame(columns=['condition','session','succrate','act_animal',
                                                                'bhv_name','bhv_id','PCs',])

    for icond_ana in np.arange(0,nconds_to_ana,1):
        cond_ana = conditions_to_ana[icond_ana]
        ind_cond = bhvevents_aligned_FR_allevents_all_dates_df['condition']==cond_ana

        for ianimal_ana in np.arange(0,nanimal_to_ana,1):
            act_animal_ana = act_animals_to_ana[ianimal_ana]
               
            if not 'partner' in act_animal_ana:
                ind_animal = bhvevents_aligned_FR_allevents_all_dates_df['act_animal']==act_animal_ana
            elif 'partner' in act_animal_ana:
                animal_dontwant = act_animal_ana.split('_')[0]
                ind_animal = bhvevents_aligned_FR_allevents_all_dates_df['act_animal']!=animal_dontwant

            # get the dates
            dates_ana = np.unique(bhvevents_aligned_FR_allevents_all_dates_df[ind_animal & ind_cond]['dates'])
            ndates_ana = np.shape(dates_ana)[0]

            for idate_ana in np.arange(0,ndates_ana,1):
                date_ana = dates_ana[idate_ana]
                ind_date = bhvevents_aligned_FR_allevents_all_dates_df['dates']==date_ana         

                for ibhvname_ana in np.arange(0,nbhvnames_to_ana,1):
                    bhvname_ana = bhv_names_to_ana[ibhvname_ana]
                    ind_bhv = bhvevents_aligned_FR_allevents_all_dates_df['bhv_name']==bhvname_ana

                    ind_ana = ind_animal & ind_bhv & ind_cond & ind_date

                    bhvevents_aligned_FR_allevents_tgt = bhvevents_aligned_FR_allevents_all_dates_df[ind_ana]

                    succrate = np.array(bhvevents_aligned_FR_allevents_tgt['succrate'])[0][0]
                    
                    # to better combine different bhv events, choose the same amount
                    nbhv_topick = 50

                    # Convert list of arrays into a single NumPy array 
                    data_array = np.array(list(bhvevents_aligned_FR_allevents_tgt['FR_allevents']))  # Shape (n neuron, t time stamp, m bhv events)

                    valid_bhvs = ~np.any(np.isnan(data_array), axis=(0, 1))  # Shape (144,)
                    data_array = data_array[:, :, valid_bhvs]

                    nneurons = np.shape(data_array)[0]
                    timepointnums = np.shape(data_array)[1]
                    mbhv_total = np.shape(data_array)[2]

                    # Randomly select bhv events with replacement, once for all neurons
                    selected_bhvs = np.random.choice(mbhv_total, nbhv_topick, replace=True)
                    sampled_data = data_array[:, :, selected_bhvs]

                    # Reshape by flattening the last two dimensions
                    final_array = sampled_data.reshape(nneurons, -1)

                    PCA_dataset_ibv = final_array

                    # combine all bhv for running PCA in the same neural space
                    if ibhvname_ana == 0:
                        PCA_dataset = PCA_dataset_ibv
                    else:
                        PCA_dataset = np.hstack([PCA_dataset,PCA_dataset_ibv])

                # remove nan raw from the data set
                # ind_nan = np.isnan(np.sum(PCA_dataset,axis=0))
                # PCA_dataset = PCA_dataset_test[:,~ind_nan]
                ind_nan = np.isnan(np.sum(PCA_dataset,axis=1))
                PCA_dataset = PCA_dataset[~ind_nan,:]
                PCA_dataset = np.transpose(PCA_dataset)

                # Run PCA on this concatenated data 
                pca = PCA(n_components=3)
                pca.fit(PCA_dataset)

                totalneuronNum = np.shape(PCA_dataset)[1]

                # project on the individual events
                for ibhvname_ana in np.arange(0,nbhvnames_to_ana,1):
                    bhvname_ana = bhv_names_to_ana[ibhvname_ana]
                    ind_bhv = bhvevents_aligned_FR_allevents_all_dates_df['bhv_name']==bhvname_ana

                    ind_ana = ind_animal & ind_bhv & ind_cond & ind_date

                    bhvevents_aligned_FR_allevents_tgt = bhvevents_aligned_FR_allevents_all_dates_df[ind_ana]

                    # get the pull triggered continuous variable of target
                    data_array_conBhv = np.array(list(bhvevents_aligned_FR_allevents_tgt[pull_trig_events_tgtname]))
                    data_array_conBhv = np.nanmean(data_array_conBhv,axis=0)
                    data_array_conBhv = data_array_conBhv.transpose()
                    
                    # get the self and other pull variable
                    data_array_otherpull = np.array(list(bhvevents_aligned_FR_allevents_tgt[pull_trig_otherpull_name]))
                    data_array_otherpull = np.nanmean(data_array_otherpull,axis=0)
                    data_array_otherpull = data_array_otherpull.transpose()
                    #
                    data_array_selfpull = np.array(list(bhvevents_aligned_FR_allevents_tgt[pull_trig_selfpull_name]))
                    data_array_selfpull = np.nanmean(data_array_selfpull,axis=0)
                    data_array_selfpull = data_array_selfpull.transpose()
                    
                    # get the pull triggered last reward time
                    data_array_lastreward = np.array(list(bhvevents_aligned_FR_allevents_tgt[pull_time_pre_reward_name]))
                    data_array_lastreward = data_array_lastreward[0]
                    
                    # get the pull reaction time
                    data_array_rt = np.array(list(bhvevents_aligned_FR_allevents_tgt[pull_rt_name]))
                    data_array_rt = data_array_rt[0]
                    
                    # Convert list of arrays into a single NumPy array 
                    data_array = np.array(list(bhvevents_aligned_FR_allevents_tgt['FR_allevents']))  # Shape (n neuron, t time stamp, m bhv events)

                    mbhv_total = np.shape(data_array)[2]

                    for ibhv in np.arange(0,mbhv_total,1):

                        data_ibhv = data_array[:,:,ibhv]

                        # get the pull triggered continous variables of target for individual events
                        try:
                            data_array_conBhv_ibhv = data_array_conBhv[:,ibhv]
                        except:
                            data_array_conBhv_ibhv = np.full((timepointnums, 1), np.nan)
                        
                        # for the socialgaze_prob, only use the meaningful ones
                        if 0:
                            if pull_trig_events_tgtname == 'socialgaze_prob':
                                trace = data_array_conBhv_ibhv
                                time_trace = np.arange(-4, 4, 1/30)
                                filtered_trace = keep_closest_cluster_single_trial(trace, time_trace)
                                data_array_conBhv_ibhv = filtered_trace
                                
                        #
                        # get the pull triggered self and pther pull for individual events
                        try:
                            data_array_otherpull_ibhv = data_array_otherpull[:,ibhv]
                        except:
                            data_array_otherpull_ibhv = np.full((timepointnums, 1), np.nan)
                        #
                        try:
                            data_array_selfpull_ibhv = data_array_selfpull[:,ibhv]
                        except:
                            data_array_selfpull_ibhv = np.full((timepointnums, 1), np.nan)
                            
                        #    
                        # get the pull triggered last reward time
                        try:
                            data_array_lastreward_ibhv = data_array_lastreward[ibhv]
                        except:
                            data_array_lastreward_ibhv = np.nan
                            
                        #
                        # get the pull reaction time
                        try:
                            data_array_rt_ibhv = data_array_rt[ibhv]
                        except:
                            data_array_rt_ibhv = np.nan
              

                        # for firing rate, project on the PC space    
                        try:
                            PCA_proj_ibhv = pca.transform(np.transpose(data_ibhv))
                        except:
                            PCA_proj_ibhv = np.full((timepointnums, 3), np.nan)

                        FRPCA_all_sessions_allevents_sum_df = FRPCA_all_sessions_allevents_sum_df.append({'condition':cond_ana,
                                                                                'act_animal':act_animal_ana,
                                                                                'bhv_name': bhvname_ana,
                                                                                'session':date_ana,
                                                                                'succrate':succrate,
                                                                                'bhv_id':ibhv,
                                                                                'PCs':PCA_proj_ibhv,
                                                                                'neuronNumBeforePCA':totalneuronNum,
                                                                                pull_trig_events_tgtname:data_array_conBhv_ibhv,                         
                                                                                pull_trig_otherpull_name:data_array_otherpull_ibhv,
                                                                                pull_trig_selfpull_name:data_array_selfpull_ibhv,
                                                                                pull_time_pre_reward_name:data_array_lastreward_ibhv,
                                                                                pull_rt_name: data_array_rt_ibhv,
                                                                                 }, ignore_index=True)


In [ ]:
FRPCA_all_sessions_allevents_sum_df

In [ ]:
FRPCA_all_sessions_allevents_sum_df.columns

In [ ]:
plt.hist(FRPCA_all_sessions_allevents_sum_df['time_from_last_reward'])

#### first analysis to test hypothesis

In [ ]:
# step 2 for each PCA trace, calculate the length, curvature, and/or tortusity for comparison later
# test hypothesis: 1. for testing if individual trial different was from gaze start time/stop time/gaze duration
    
import statsmodels.formula.api as smf
import statsmodels.api as sm

FRPCAfeatures_all_sessions_allevents_sum_df = pd.DataFrame(columns=['condition','session','act_animal','succrate',
                                                                    'bhv_name','bhv_id',
                                                                    'PClength','PCcurv','PCtort','PCspeed','PCsmoothness',
                                                                    'PCspeed_trace','PCcurv_trace',
                                                                    ])
FRPCAfeatures_gazeduration_corr_all_sessions_df = pd.DataFrame(columns=['condition','session','succrate',
                                                                        'act_animal','bhv_name',])


# newly added control!!
# only look at pull aligned events that has no preceding self pull 
doSingleSelfPulls = 1

# only look at pull aligned events that do not have juice deliever effect from the previous pull
doFarLastRewards = 0

# only look at pulls that has large reaction time
doLargeRTpulls = 1
largeRTthreshold = 3 # the threshold for defining large threahold


# add three kinds of gaze duration definition (around pull, before pull, after pull)
gaze_duration_type = 'before_pull' # 'around_pull', 'before_pull', 'after_pull'

#
for ianimal_ana in np.arange(0,nanimal_to_ana,1):
    act_animal_ana = act_animals_to_ana[ianimal_ana]
    ind_animal = FRPCA_all_sessions_allevents_sum_df['act_animal']==act_animal_ana
        
    # get the dates
    dates_toplot = np.unique(FRPCA_all_sessions_allevents_sum_df[ind_animal]['session'])
    ndates_toplot = np.shape(dates_toplot)[0]
    
    # figures 
    fig1, axs1 = plt.subplots(nconds_to_ana,ndates_toplot)
    fig1.set_figheight(24*nconds_to_ana)
    fig1.set_figwidth(8*ndates_toplot)
    # Ensure axs1 is always 2D
    axs1 = np.atleast_2d(axs1)
    axs1_flat = axs1.flatten()  # Flatten for easier iteration
    # Track used axes
    used_axes = set()

    for icond_ana in np.arange(0,nconds_to_ana,1):
        cond_ana = conditions_to_ana[icond_ana]
        ind_cond = FRPCA_all_sessions_allevents_sum_df['condition']==cond_ana

        # get the dates
        dates_ana = np.unique(FRPCA_all_sessions_allevents_sum_df[ind_animal & ind_cond]['session'])
        ndates_ana = np.shape(dates_ana)[0]

        for idate_ana in np.arange(0,ndates_ana,1):
            date_ana = dates_ana[idate_ana]
            ind_date = FRPCA_all_sessions_allevents_sum_df['session']==date_ana         

            for ibhvname_ana in np.arange(0,nbhvnames_to_ana,1):
                bhvname_ana = bhv_names_to_ana[ibhvname_ana]
                ind_bhv = FRPCA_all_sessions_allevents_sum_df['bhv_name']==bhvname_ana

                ind_ana = ind_animal & ind_bhv & ind_cond & ind_date

                FRPCA_allevents_toana = FRPCA_all_sessions_allevents_sum_df[ind_ana]

                bhv_ids = np.array(FRPCA_allevents_toana['bhv_id'])
                nbhvevents = np.shape(bhv_ids)[0]

                for ibhv_id in np.arange(0,nbhvevents,1):

                    bhv_id = bhv_ids[ibhv_id]
                    ind_bhvid = FRPCA_allevents_toana['bhv_id'] == bhv_id

                    # 
                    # count self pull before self pull, the goal is to make sure the effect we will find later is not because of the preceding pulls
                    from scipy.signal import find_peaks
                    #
                    pulltrig_selfpull = np.array(FRPCA_allevents_toana[ind_bhvid][pull_trig_selfpull_name])[0]
                    #
                    x_full = np.arange(-4,4,1/fps)
                    pre_mask = x_full<-0.15
                    try:
                        # selfpull_num = np.trapz(pull_trig_selfpull_ievent[pre_mask], dx=dt)
                        data = pulltrig_selfpull[pre_mask]
                        peaks, _ = find_peaks(data)
                        selfpull_num = len(peaks)
                    except:
                        selfpull_num = np.nan
                    #
                    if selfpull_num > 0:
                        lastselfpull_time = x_full[peaks[-1]]
                    else:
                        lastselfpull_time =  np.nan

                    # 
                    # add the info about the last reward time
                    lastreward_time = -np.array(FRPCA_allevents_toana[ind_bhvid][pull_time_pre_reward_name])[0]
                    
                    # 
                    # try to define a relavant time window for each pull
                    # this is almost the reaction time
                    # last_relavant_time = np.nanmax([lastreward_time, lastselfpull_time, -4])
                    last_relavant_time = np.nanmax([lastreward_time, lastselfpull_time])
                    
                    #
                    # add information about the pull reaction time
                    pull_rt = np.array(FRPCA_allevents_toana[ind_bhvid][pull_rt_name])[0]
                    
                    # 
                    # analyze the pull triggered behavioral events
                    if (pull_trig_events_tgtname == 'socialgaze_prob') |\
                       (pull_trig_events_tgtname == 'othergaze_prob') :
                        # calculate the gaze start and gaze stop time, and finally gaze duration
                        try:
                            pulltrig_conBhv = np.array(FRPCA_allevents_toana[ind_bhvid][pull_trig_events_tgtname])[0]

                            # Find the point of first increasing and last decrease to estimate gaze start and end
                            #
                            timewins = np.arange(-4,4,1/fps) # make sure it align with the setting in the previous section
                            #
                            # if gaze_duration_type == 'before_pull':
                            #     pulltrig_conBhv[timewins>0] = 0
                            # elif gaze_duration_type == 'after_pull':
                            #     pulltrig_conBhv[timewins<0] = 0                           
                            
                            if 1:
                                first_increase_idx = np.where(np.diff(pulltrig_conBhv) > 0)[0][0] + 1
                                #
                                last_decrease_idx = np.where(np.diff(pulltrig_conBhv) < 0)[0][-1] + 1  # Find last decrease
                                #
                                gazestart_time = timewins[first_increase_idx].copy()
                                gazestop_time = timewins[last_decrease_idx].copy()
                            if 0:
                                # Find peaks
                                peaks, _ = scipy.signal.find_peaks(pulltrig_conBhv)
                                #
                                # Get first and last peak
                                first_peak = peaks[0] 
                                last_peak = peaks[-1]
                                #
                                gazestart_time = timewins[first_peak].copy()
                                gazestop_time = timewins[last_peak].copy()
                            #
                            # change the gazestart and gazestop time based on the gaze duration definition
                            if gaze_duration_type == 'around_pull':
                                gazestart_time = gazestart_time
                                gazestop_time = gazestop_time
                            if gaze_duration_type == 'before_pull':
                                if (gazestart_time > 0):
                                    gazestart_time = np.nan
                                    gazestop_time = np.nan
                                elif (gazestop_time > 0):
                                    gazestop_time = 0
                            if gaze_duration_type == 'after_pull':
                                if (gazestop_time < 0):
                                    gazestart_time = np.nan
                                    gazestop_time = np.nan
                                elif (gazestart_time < 0):
                                    gazestart_time = 0                                                
                            #
                            # if (gazestart_time == timewins[0]) | (gazestart_time == timewins[-1]):
                            #     gazestart_time = np.nan
                            # if (gazestop_time == timewins[0]) | (gazestop_time == timewins[-1]):
                            #     gazestop_time = np.nan
                            if (gazestop_time < gazestart_time):
                                gazestart_time = np.nan
                                gazestop_time = np.nan                           
                        except:
                            gazestart_time = np.nan
                            gazestop_time = np.nan
                            
                        # calculate the gaze accumulation (use auc to estimate)
                        try:
                            timewins = np.arange(-4,4,1/fps) # make sure it align with the setting in the previous section
                            dt = 1 / fps  # sampling interval in seconds
                            #
                            if gaze_duration_type == 'around_pull':
                                auc = np.trapz(pulltrig_conBhv, dx=dt)
                            if gaze_duration_type == 'before_pull':
                                auc = np.trapz(pulltrig_conBhv[timewins<0], dx=dt)
                            if gaze_duration_type == 'after_pull':
                                auc = np.trapz(pulltrig_conBhv[timewins>0], dx=dt)
                            #
                            gaze_accum = auc
                        #
                        except:
                            gaze_accum = np.nan
                            
                        # remove trial with zero gaze_accumulation
                        if gaze_accum == 0:
                            gaze_accum = np.nan
                                
                            
                    
                    # 
                    # analyze the PCs 
                    FRPCA_ievent_toana = np.array(FRPCA_allevents_toana[ind_bhvid]['PCs'])[0]

                    # smooth the pc trajectory
                    if 0:
                        FRPCA_ievent_toana = np.apply_along_axis(gaussian_filter1d, axis=0, 
                                                                 arr=FRPCA_ievent_toana, sigma=6)

                    # calculate the length, curvature and tortuosity
                    PC_traj = FRPCA_ievent_toana.copy()  # Shape (240, 3)
                    
                    #
                    if gaze_duration_type == 'before_pull':
                        ntimepoints = np.shape(PC_traj)[0]
                        PC_traj = PC_traj[0:int(ntimepoints/2),:]
                    elif gaze_duration_type == 'after_pull':
                        ntimepoints = np.shape(PC_traj)[0]
                        PC_traj = PC_traj[int(ntimepoints/2):,:]

                    # Compute differences between consecutive points
                    diffs = np.diff(PC_traj, axis=0)

                    # Compute segment lengths
                    segment_lengths = np.linalg.norm(diffs, axis=1)
                    total_length = np.sum(segment_lengths)  # Arc length of trajectory

                    # Compute curvature
                    # First derivatives
                    dX_dt = np.gradient(PC_traj[:, 0])
                    dY_dt = np.gradient(PC_traj[:, 1])
                    dZ_dt = np.gradient(PC_traj[:, 2])
                    dV = np.vstack((dX_dt, dY_dt, dZ_dt)).T

                    # Second derivatives
                    d2X_dt2 = np.gradient(dX_dt)
                    d2Y_dt2 = np.gradient(dY_dt)
                    d2Z_dt2 = np.gradient(dZ_dt)
                    d2V = np.vstack((d2X_dt2, d2Y_dt2, d2Z_dt2)).T

                    # Curvature formula: ||dV x d2V|| / ||dV||^3
                    cross_prod = np.cross(dV[:-1], d2V[:-1])  # Compute cross product
                    curvature = np.linalg.norm(cross_prod, axis=1) / (np.linalg.norm(dV[:-1], axis=1) ** 3 + 1e-10)

                    # Compute tortuosity: Total length / Euclidean distance between start and end
                    euclidean_distance = np.linalg.norm(PC_traj[-1] - PC_traj[0])
                    tortuosity = total_length / euclidean_distance if euclidean_distance > 0 else np.nan
                    
                    # Compute speed 
                    dt = 1.0 / fps  # Time between frames
                    # Velocity: first derivative of position
                    velocity = np.gradient(PC_traj, axis=0) / dt
                    # Speed: magnitude of velocity
                    speed = np.linalg.norm(velocity, axis=1)
                    
                    # Compute Smoothness - A simple way to compute trajectory smoothness is to look at the jerk 
                    # — the third derivative of position (how quickly acceleration changes), 
                    # which reflects sudden directional/velocity shifts
                    # Acceleration: second derivative
                    acceleration = np.gradient(velocity, axis=0) / dt
                    # Jerk: third derivative
                    jerk = np.gradient(acceleration, axis=0) / dt
                    # Smoothness metric: integrated squared jerk over time
                    squared_jerk = np.linalg.norm(jerk, axis=1) ** 2
                    smoothness = np.sum(squared_jerk) * dt

                    FRPCAfeatures_all_sessions_allevents_sum_df = FRPCAfeatures_all_sessions_allevents_sum_df.append({
                                                                                'condition':cond_ana,
                                                                                'act_animal':act_animal_ana,
                                                                                'bhv_name': bhvname_ana,
                                                                                'session':date_ana,
                                                                                'succrate':np.array(FRPCA_allevents_toana[ind_bhvid]['succrate'])[0],
                                                                                'bhv_id':ibhv_id,
                                                                                'PClength':total_length,
                                                                                'PCcurv':np.nanmean(curvature),
                                                                                'PCtort':tortuosity,
                                                                                'PCspeed':np.nanmean(speed),
                                                                                'PCsmoothness':smoothness,
                                                                                'PCspeed_trace':speed,
                                                                                'PCcurv_trace':curvature,
                                                                                'gazestart_time':gazestart_time,
                                                                                'gazestop_time':gazestop_time,
                                                                                'gaze_accum':gaze_accum,
                                                                                'neuronNumBeforePCA':np.array(FRPCA_allevents_toana[ind_bhvid]['neuronNumBeforePCA'])[0],
                                                                                'selfpull_num':selfpull_num,
                                                                                'lastreward_time':lastreward_time,
                                                                                'lastselfpull_time':lastselfpull_time,
                                                                                'last_relavant_time':last_relavant_time,
                                                                                'pull_rt':pull_rt,
                                                                            }, ignore_index=True)
                    
                 
                # 
                # remove events that has multiple self pulls before the aligned self pulls 
                if doSingleSelfPulls:
                    ind_good = FRPCAfeatures_all_sessions_allevents_sum_df['selfpull_num']==0
                    FRPCAfeatures_all_sessions_allevents_sum_df = FRPCAfeatures_all_sessions_allevents_sum_df[ind_good]
                
                #
                # remove events that still has the effect from previous juice
                if doFarLastRewards:
                    # reward should be beyond the -4s time window we are looking at, and also consider the reward effect last ~1s
                    ind_good = FRPCAfeatures_all_sessions_allevents_sum_df['lastreward_time']<=(-4-1)
                    FRPCAfeatures_all_sessions_allevents_sum_df = FRPCAfeatures_all_sessions_allevents_sum_df[ind_good]
                    
                #
                # remove events that still has small reaction time
                if doLargeRTpulls:
                    # reward should be beyond the -4s time window we are looking at, and also consider the reward effect last ~1s
                    ind_good = FRPCAfeatures_all_sessions_allevents_sum_df['pull_rt']>=largeRTthreshold
                    FRPCAfeatures_all_sessions_allevents_sum_df = FRPCAfeatures_all_sessions_allevents_sum_df[ind_good]
                    
                #
                # remove the zero (nan) gaze accumulation level
                ind_nan = np.isnan(FRPCAfeatures_all_sessions_allevents_sum_df['gaze_accum'])
                FRPCAfeatures_all_sessions_allevents_sum_df = FRPCAfeatures_all_sessions_allevents_sum_df[~ind_nan]
                
                
                # after pool all the events related data together do some plotting and calculate the correlation
                ind_sess_toplot = FRPCAfeatures_all_sessions_allevents_sum_df['session'] == date_ana
                ind_ani_toplot = FRPCAfeatures_all_sessions_allevents_sum_df['act_animal'] == act_animal_ana
                ind_bhv_toplot = FRPCAfeatures_all_sessions_allevents_sum_df['bhv_name'] == bhvname_ana
                ind_cond_toplot = FRPCAfeatures_all_sessions_allevents_sum_df['condition'] == cond_ana
                
                ind_toplot = ind_sess_toplot & ind_ani_toplot & ind_bhv_toplot & ind_cond_toplot
                FRPCAfeatures_toplot = FRPCAfeatures_all_sessions_allevents_sum_df[ind_toplot]
                
                yyy_types = ['PCcurv','PClength','PCsmoothness','PCspeed']
                nytypes = np.shape(yyy_types)[0]
                
                # xxx_type = 'gaze_duration'
                # xxx_type = 'gaze_accumulation'
                xxx_type = 'pull_rt'
                
                # Use gridspec to divide axs1[icond_ana, idate_ana] into nytypes rows
                gs = gridspec.GridSpecFromSubplotSpec(nytypes, 1, subplot_spec=axs1[icond_ana, idate_ana], hspace=0.3)

                for iytype in np.arange(0,nytypes,1):
                    
                    ax = fig1.add_subplot(gs[iytype])  # Create subplots within the existing grid cell
                    used_axes.add(ax)  # Mark this axis as used
                    
                    if xxx_type == 'gaze_duration':
                        xxx = FRPCAfeatures_toplot['gazestop_time'] - FRPCAfeatures_toplot['gazestart_time']
                    elif xxx_type == 'gaze_accumulation':
                        xxx = FRPCAfeatures_toplot['gaze_accum']
                    else:
                        xxx = FRPCAfeatures_toplot[xxx_type]
                    
                    yyy_type = yyy_types[iytype]
                    yyy = FRPCAfeatures_toplot[yyy_type]

                    ind_nan = np.isnan(xxx) | np.isnan(yyy)
                    xxx = xxx[~ind_nan]
                    yyy = yyy[~ind_nan]

                    # Compute correlation
                    if len(xxx) > 1:
                        r, p = st.pearsonr(xxx, yyy)
                        # Fit regression line
                        slope, intercept = np.polyfit(xxx, yyy, 1)
                        x_vals = np.array([min(xxx), max(xxx)])
                        y_vals = slope * x_vals + intercept
                        ax.plot(x_vals, y_vals, color='red', linestyle='--', label='Regression line')
                    else:
                        r, p = np.nan, np.nan

                    # Scatter plot
                    ax.plot(xxx, yyy, 'o', label='gaze around pull')

                    # Title and labels
                    ax.set_title(bhvname_ana + ' of ' + act_animal_ana + ' in ' +
                                 cond_ana + ' ' + date_ana + '\n neuron #=' +
                                 str(FRPCAfeatures_toplot['neuronNumBeforePCA'].iloc[0]), fontsize=12)
                    ax.set_ylabel(yyy_type, fontsize=12)

                    # Add correlation text
                    ax.text(0.05, 0.9, f"r = {r:.3f}\np = {p:.3f}", transform=ax.transAxes, fontsize=12,
                            verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5, edgecolor='gray'))

                    # Optional: show legend
                    ax.legend()

                    #
                    # plot the one without any gaze as a comparison
                    xxx = FRPCAfeatures_toplot['gazestop_time'] - FRPCAfeatures_toplot['gazestart_time']
                    
                    yyy_type = yyy_types[iytype]
                    yyy = FRPCAfeatures_toplot[yyy_type]
                    
                    ind_nan = (np.isnan(xxx)) & (~np.isnan(yyy))
                    yyy = yyy[ind_nan]
                    xxx = np.zeros(np.shape(yyy))
                    
                    ax.plot(xxx, yyy, 'ro',label='no gaze around pull')
                    
                    ax.set_xlabel(xxx_type+' around pull (s)',fontsize=12)  # Set xlabel only for the last subplot in the stack
                    ax.legend(loc='lower right')
            
                    # 
                    FRPCAfeatures_gazeduration_corr_all_sessions_df = FRPCAfeatures_gazeduration_corr_all_sessions_df.append({
                                                                                'condition':cond_ana,
                                                                                'act_animal':act_animal_ana,
                                                                                'bhv_name': bhvname_ana,
                                                                                'session':date_ana,
                                                                                'succrate':np.array(FRPCA_allevents_toana[ind_bhvid]['succrate'])[0],
                                                                                'corr_'+yyy_type+'_vs_'+xxx_type:r,
                                                                                'pcorr_'+yyy_type+'_vs_'+xxx_type:p,
                                                                                'gazeduration_definition':gaze_duration_type,
                                                                               }, ignore_index=True)
                                                               
                                                               
    # fig1.tight_layout()
    
    #  Hide unused subplots
    for ax in axs1_flat:
        if ax not in used_axes:  # If an axis wasn't used, hide it
            ax.set_visible(False)

    
    
    savefig = 0
    if savefig:
        figsavefolder = data_saved_folder+"fig_for_basic_neural_analysis_allsessions_basicEvents_PCA_makeBhvNeuronVideos_Pullfocused_continuousBhv/"+\
                        cameraID+"/"+animal1_filenames[0]+"_"+animal2_filenames[0]+"/FRsPCA_fig/"

        if not os.path.exists(figsavefolder):
            os.makedirs(figsavefolder)

        fig1.savefig(figsavefolder+'bhvevents_aligned_PCspace_trajectory_features_and_continuousBhv_'+bhvname_ana+'_'+
                     pull_trig_events_tgtname+'_'+xxx_type+'_'+gaze_duration_type+savefile_sufix+'.pdf')
        


In [ ]:
ind_tgt = np.isin(FRPCAfeatures_all_sessions_allevents_sum_df['condition'],['MC'])
plt.hist(FRPCAfeatures_all_sessions_allevents_sum_df[ind_tgt]['last_relavant_time'])

In [ ]:
ind_tgt = FRPCAfeatures_all_sessions_allevents_sum_df['selfpull_num']>=0
np.unique(FRPCAfeatures_all_sessions_allevents_sum_df[ind_tgt]['condition'])

In [ ]:
np.nanmean(FRPCAfeatures_all_sessions_allevents_sum_df['lastreward_time'])

In [ ]:
ind_tgt = np.isin(FRPCAfeatures_all_sessions_allevents_sum_df['condition'],['MC'])
xxx = FRPCAfeatures_all_sessions_allevents_sum_df[ind_tgt]['pull_rt']
yyy = FRPCAfeatures_all_sessions_allevents_sum_df[ind_tgt]['gaze_accum']
plt.plot(xxx,yyy,'.')

In [ ]:
np.unique(FRPCAfeatures_all_sessions_allevents_sum_df['selfpull_num'])

In [ ]:
FRPCAfeatures_gazeduration_corr_all_sessions_df.columns

In [ ]:
# show all the session-by-session correlation between PC trajectory features and gaze duration 
if 1:    
    import itertools
    
    #####
    # to make the condition more general
    # Define the function for generalizing condition
    def generalize_condition(cond):
        if cond == "MC" or cond.startswith("MC_with"):
            return "MC"
        elif cond == "SR" or cond.startswith("SR_with"):
            return "SR"
        elif cond == "NV" or cond.startswith("NV_with"):
            return "NV"
        else:
            return cond  # default to original condition if no match

    # Apply the function to create the new column
    FRPCAfeatures_gazeduration_corr_all_sessions_df["condition_general"] = \
        FRPCAfeatures_gazeduration_corr_all_sessions_df["condition"].apply(generalize_condition)
    
    #####
    
    # corr_type = 'corr_PClength_vs_gaze_duration'
    # corr_type = 'corr_PClength_vs_gaze_accumulation'
    # corr_type = 'corr_PCcurv_vs_gaze_accumulation'
    corr_type = 'corr_PClength_vs_pull_rt'


    fig2, axs2 = plt.subplots(1, 1)
    fig2.set_figheight(5)
    fig2.set_figwidth(12)

    # Define a consistent color
    box_color = 'skyblue'

    # Plot boxplot and swarmplot
    seaborn.violinplot(ax=axs2, data=FRPCAfeatures_gazeduration_corr_all_sessions_df,
                x='condition_general', y=corr_type,
                color=box_color)

    seaborn.swarmplot(ax=axs2, data=FRPCAfeatures_gazeduration_corr_all_sessions_df,
                   x='condition_general', y=corr_type,
                   color='b', size=6)

    # Rotate x-axis ticks
    axs2.set_xticklabels(axs2.get_xticklabels(), rotation=90)

    # Significance from 0 (Wilcoxon signed-rank)
    conditions = FRPCAfeatures_gazeduration_corr_all_sessions_df['condition_general'].unique()
    y_max = FRPCAfeatures_gazeduration_corr_all_sessions_df[corr_type].max()
    y_min = FRPCAfeatures_gazeduration_corr_all_sessions_df[corr_type].min()

    y_offset = (y_max - y_min) * 0.15  # vertical spacing for significance bars

    for i, cond in enumerate(conditions):
        data = FRPCAfeatures_gazeduration_corr_all_sessions_df[
            FRPCAfeatures_gazeduration_corr_all_sessions_df['condition_general'] == cond][corr_type].dropna()
        if len(data) > 0 and np.any(data != 0):  # Wilcoxon requires non-zero variation
            try:
                stat, p = st.wilcoxon(data)
                if p < 0.01:
                    axs2.text(i, data.max() + y_offset, '*', ha='center', va='bottom', fontsize=16, color='black')
            except ValueError:
                pass  # skip if data not suitable for Wilcoxon

    # Pairwise comparisons (Mann–Whitney U)
    pairs = list(itertools.combinations(range(len(conditions)), 2))
    for j, (i1, i2) in enumerate(pairs):
        cond1 = conditions[i1]
        cond2 = conditions[i2]
        data1 = FRPCAfeatures_gazeduration_corr_all_sessions_df[
            FRPCAfeatures_gazeduration_corr_all_sessions_df['condition_general'] == cond1][corr_type].dropna()
        data2 = FRPCAfeatures_gazeduration_corr_all_sessions_df[
            FRPCAfeatures_gazeduration_corr_all_sessions_df['condition_general'] == cond2][corr_type].dropna()

        if len(data1) > 0 and len(data2) > 0:
            stat, p = st.mannwhitneyu(data1, data2, alternative='two-sided')
            if p < 0.01:
                y = max(data1.max(), data2.max()) + y_offset * (j + 2)
                axs2.plot([i1, i2], [y, y], lw=1.5, c='black')
                axs2.text((i1 + i2) / 2, y + y_offset * 0.2, '*', ha='center', va='bottom', fontsize=16)
     
    # fig2.tight_layout()

    savefig = 0
    if savefig:
        figsavefolder = data_saved_folder+"fig_for_basic_neural_analysis_allsessions_basicEvents_PCA_makeBhvNeuronVideos_Pullfocused_continuousBhv/"+\
                        cameraID+"/"+animal1_filenames[0]+"_"+animal2_filenames[0]+"/FRsPCA_fig/"

        if not os.path.exists(figsavefolder):
            os.makedirs(figsavefolder)

        fig2.savefig(figsavefolder+'bhvevents_aligned_PCspace_trajectory_features_and_continuousBhv_'+bhvname_ana+'_'+
                     pull_trig_events_tgtname+'_'+gaze_duration_type+'_'+corr_type+savefile_sufix+'.pdf')
        

    # to see if there's correlation between correlation coefficient of PC features vs gaze duration, and succ rate 
    from scipy.stats import pearsonr

    # Drop rows where either column is NaN
    subset_df = FRPCAfeatures_gazeduration_corr_all_sessions_df[['condition_general', 'succrate', corr_type]].dropna()

    # Group by condition and calculate Pearson correlation and p-value
    def compute_corr_pval(group):
        if len(group) < 2:
            return pd.Series({'r': None, 'p': None})  # Not enough data to correlate
        r, p = pearsonr(group['succrate'], group[corr_type])
        return pd.Series({'r': r, 'p': p})

    grouped_corrs = subset_df.groupby('condition_general').apply(compute_corr_pval)

    # Display the result
    print(f'Correlation between {corr_type} and succrate:')
    print(grouped_corrs)
        

In [ ]:
# show the single trial trajectories of an example session - compared the nogaze_pull and gaze_pull
if 0:
    # Create a 3D figure
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')

    # animal_exmaple = 'dodson'
    # session_example = '20240603_MC'
    animal_exmaple = 'kanga'
    session_example = '202408123","20250403_SR'

    ind_example = FRPCAfeatures_all_sessions_allevents_sum_df['session'] == session_example
    FRPCAfeatures_example = FRPCAfeatures_all_sessions_allevents_sum_df[ind_example]
    #
    ind_example = FRPCA_all_sessions_allevents_sum_df['session']==session_example
    FRPCA_example = FRPCA_all_sessions_allevents_sum_df[ind_example]

    # plot the nogazearound_pull tracjectory
    ind_nogaze = (np.isnan(FRPCAfeatures_example['gazestart_time'])) & (np.isnan(FRPCAfeatures_example['gazestop_time']))
    nogaze_pullIDs = np.array(FRPCAfeatures_example[ind_nogaze]['bhv_id'])
    #
    ind_nogazepullIDs = np.isin(FRPCA_example['bhv_id'],nogaze_pullIDs)
    FRPCA_nogazepulls = np.array(FRPCA_example[ind_nogazepullIDs]['PCs'])
    n_nogazepulls = np.shape(FRPCA_nogazepulls)[0]
    #
    for ii in np.arange(0,n_nogazepulls,1):
    # for ii in np.arange(0,1,1):
        xxx = gaussian_filter1d(FRPCA_nogazepulls[ii][:,0],6)
        yyy = gaussian_filter1d(FRPCA_nogazepulls[ii][:,1],6)
        zzz = gaussian_filter1d(FRPCA_nogazepulls[ii][:,2],6)

        if ii == 0:
            ax.plot3D(xxx, yyy, zzz, color='r', linewidth=2, label='no gaze around pull FR PC trajectory')
        else:
            ax.plot3D(xxx, yyy, zzz, color='r', linewidth=2)

    # plot the gazearound pull trajectory
    FRPCA_gazepulls = np.array(FRPCA_example[~ind_nogazepullIDs]['PCs'])
    n_gazepulls = np.shape(FRPCA_gazepulls)[0]
    ind_gazepulls_toplot = np.random.choice(np.arange(0, n_gazepulls, 1), size=n_nogazepulls, replace=False)
    # 
    for ii in ind_gazepulls_toplot:
        xxx = gaussian_filter1d(FRPCA_gazepulls[ii][:,0],6)
        yyy = gaussian_filter1d(FRPCA_gazepulls[ii][:,1],6)
        zzz = gaussian_filter1d(FRPCA_gazepulls[ii][:,2],6)

        if ii == ind_gazepulls_toplot[0]:
            ax.plot3D(xxx, yyy, zzz, color='b', linewidth=2, label='gaze around pull FR PC trajectory')
        else:
            ax.plot3D(xxx, yyy, zzz, color='b', linewidth=2)

    ax.legend()

In [ ]:
# show the single trial trajectories of an example session - compared the gaze_pull separating three quantile based on the gaze duration
if 0:
    # Create a 3D figure
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')

    # animal_exmaple = 'dodson'
    # session_example = '20250409'
    # animal_exmaple = 'dodson'
    # session_example = '20250415'
    
    animal_exmaple = 'kanga'
    session_example = '20240808'
    # animal_exmaple = 'kanga'
    # session_example = '20240606'
    # animal_exmaple = 'kanga'
    # session_example = '20250415'


    ind_example = FRPCAfeatures_all_sessions_allevents_sum_df['session'] == session_example
    FRPCAfeatures_example = FRPCAfeatures_all_sessions_allevents_sum_df[ind_example]
    #
    ind_example = FRPCA_all_sessions_allevents_sum_df['session']==session_example
    FRPCA_example = FRPCA_all_sessions_allevents_sum_df[ind_example]
    
    #
    # gaze_durations = FRPCAfeatures_example['gazestop_time'] - FRPCAfeatures_example['gazestart_time']
    # alternatively
    xxx_type = 'pull_rt' # 'gaze_accum' or 'pull_rt', or etc
    gaze_durations = FRPCAfeatures_example[xxx_type]
    gaze_durations = np.array(gaze_durations)
    #
    # Compute tertile (33rd and 67th percentiles), ignoring NaNs
    q1, q2 = np.nanquantile(gaze_durations, [1/3, 2/3])
    #
    # Separate data into three groups
    ind_low = gaze_durations <= q1
    ind_mid = (gaze_durations > q1) & (gaze_durations <= q2)
    ind_high = gaze_durations > q2
    
    for ii in np.arange(0,3,1):
        if ii == 0:
            meanPCA_traj = np.nanmean(np.stack(FRPCA_example[ind_low]['PCs'], axis=0),axis=0)
            traj_clr = 'b'
            traj_lab = 'low '+xxx_type+' ' + str(np.sum(ind_low))+' trials'
        elif ii == 1:
            meanPCA_traj = np.nanmean(np.stack(FRPCA_example[ind_mid]['PCs'], axis=0),axis=0)
            traj_clr = 'r'
            traj_lab = 'mid '+xxx_type+' ' + str(np.sum(ind_mid))+' trials'
        elif ii == 2:
            meanPCA_traj = np.nanmean(np.stack(FRPCA_example[ind_high]['PCs'], axis=0),axis=0)
            traj_clr = 'y'
            traj_lab = 'high '+xxx_type+' ' + str(np.sum(ind_high))+' trials'
           
        ntimepoints = np.shape(meanPCA_traj)[0]
        
        xxx = gaussian_filter1d(meanPCA_traj[:,0],6)
        yyy = gaussian_filter1d(meanPCA_traj[:,1],6)
        zzz = gaussian_filter1d(meanPCA_traj[:,2],6)
        if gaze_duration_type == 'before_pull':
            xxx = gaussian_filter1d(meanPCA_traj[:int(ntimepoints/2),0],3)
            yyy = gaussian_filter1d(meanPCA_traj[:int(ntimepoints/2),1],3)
            zzz = gaussian_filter1d(meanPCA_traj[:int(ntimepoints/2),2],3)   
        elif gaze_duration_type == 'after_pull':
            xxx = gaussian_filter1d(meanPCA_traj[int(ntimepoints/2):,0],3)
            yyy = gaussian_filter1d(meanPCA_traj[int(ntimepoints/2):,1],3)
            zzz = gaussian_filter1d(meanPCA_traj[int(ntimepoints/2):,2],3)   
            
        ax.plot3D(xxx, yyy, zzz, color=traj_clr, linewidth=2, label=traj_lab)
        ax.plot3D(xxx[0], yyy[0], zzz[0], color=traj_clr, marker ='o',markersize = 12)
        ax.plot3D(xxx[-1], yyy[-1], zzz[-1], color=traj_clr, marker ='s',markersize = 12)

    
    ax.legend()
    
    savefig = 1
    if savefig:
        figsavefolder = data_saved_folder+"fig_for_basic_neural_analysis_allsessions_basicEvents_PCA_makeBhvNeuronVideos_Pullfocused_continuousBhv/"+cameraID+"/"+animal1_filenames[0]+"_"+animal2_filenames[0]+'/'+session_example+'/'

        if not os.path.exists(figsavefolder):
            os.makedirs(figsavefolder)

        fig.savefig(figsavefolder+'forExample_PCAtrajectory_in_'+xxx_type+'_quantiles.pdf')

    
    
    

In [ ]:
# show the single trial  an example session - compared the gaze_pull separating three quantile based on the gaze duration
# average and show the speed or curvature trace of each quantile
if 0:
    # Create a figure
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)

    trace_toplot = 'PCspeed_trace'
    
    # animal_exmaple = 'dodson'
    # session_example = '20250415'
    # animal_exmaple = 'dodson'
    # session_example = '20240612'
    
    animal_exmaple = 'kanga'
    session_example = '20240808'
    # animal_exmaple = 'kanga'
    # session_example = '20240809'
    # animal_exmaple = 'kanga'
    # session_example = '20240523'
    # animal_exmaple = 'kanga'
    # session_example = '20250415'


    ind_example = FRPCAfeatures_all_sessions_allevents_sum_df['session'] == session_example
    FRPCAfeatures_example = FRPCAfeatures_all_sessions_allevents_sum_df[ind_example]
    #
    ind_example = FRPCA_all_sessions_allevents_sum_df['session']==session_example
    FRPCA_example = FRPCA_all_sessions_allevents_sum_df[ind_example]

    #
    # gaze_durations = FRPCAfeatures_example['gazestop_time'] - FRPCAfeatures_example['gazestart_time']
    # alternatively
    xxx_type = 'pull_rt' # 'gaze_accum' or 'pull_rt', or etc
    gaze_durations = FRPCAfeatures_example[xxx_type]
    gaze_durations = np.array(gaze_durations)
    #
    # Compute tertile (33rd and 67th percentiles), ignoring NaNs
    q1, q2 = np.nanquantile(gaze_durations, [1/3, 2/3])
    #
    # Separate data into three groups
    ind_low = gaze_durations <= q1
    ind_mid = (gaze_durations > q1) & (gaze_durations <= q2)
    ind_high = gaze_durations > q2
    
    for ii in np.arange(0,3,1):
        if ii == 0:
            meanPCA_traj = np.nanmean(np.stack(FRPCAfeatures_example[ind_low][trace_toplot], axis=0),axis=0)
            traj_clr = 'b'
            traj_lab = 'low '+xxx_type+' ' + str(np.sum(ind_low))+' trials'
        elif ii == 1:
            meanPCA_traj = np.nanmean(np.stack(FRPCAfeatures_example[ind_mid][trace_toplot], axis=0),axis=0)
            traj_clr = 'r'
            traj_lab = 'mid '+xxx_type+' ' + str(np.sum(ind_mid))+' trials'
        elif ii == 2:
            meanPCA_traj = np.nanmean(np.stack(FRPCAfeatures_example[ind_high][trace_toplot], axis=0),axis=0)
            traj_clr = 'y'
            traj_lab = 'high '+xxx_type+' ' + str(np.sum(ind_high))+' trials'
           
        ntimepoints = np.shape(meanPCA_traj)[0]
        
        if gaze_duration_type == 'around_pull':
            xtimes = np.arange(-4,4,1/fps)
            xxx = gaussian_filter1d(meanPCA_traj,6)
        else:
            xtimes = np.arange(-4,0,1/fps)
            xxx = gaussian_filter1d(meanPCA_traj,6)
            
        ax.plot(xtimes, xxx, color=traj_clr, linewidth=2, label=traj_lab)
        ax.plot(xtimes[0], xxx[0], color=traj_clr, marker ='o',markersize = 12)
        ax.plot(xtimes[-1], xxx[-1], color=traj_clr, marker ='s',markersize = 12)
        
        ax.set_xlabel('time/s')
        ax.set_ylabel('PC trajectory speed')
    
    ax.legend()
    
    

#### sanity check plot, individual neurons' pull aligned FR and gaze related measures

In [ ]:
# this one needs to be run after the previous code becuase it need the definition of the 
# gaze_duration or gaze_accumulation from the previous code

if 0:
    
    # load the data 
    for icond_ana in np.arange(0,nconds_to_ana,1):
        cond_ana = conditions_to_ana[icond_ana]
        ind_cond = bhvevents_aligned_FR_allevents_all_dates_df['condition']==cond_ana
        ind_cond_FRPCA = FRPCAfeatures_all_sessions_allevents_sum_df['condition']==cond_ana

        for ianimal_ana in np.arange(0,nanimal_to_ana,1):
            act_animal_ana = act_animals_to_ana[ianimal_ana]
            ind_animal = bhvevents_aligned_FR_allevents_all_dates_df['act_animal']==act_animal_ana
            ind_animal_FRPCA = FRPCAfeatures_all_sessions_allevents_sum_df['act_animal']==act_animal_ana

            # get the dates
            dates_ana = np.unique(bhvevents_aligned_FR_allevents_all_dates_df[ind_animal & ind_cond]['dates'])
            ndates_ana = np.shape(dates_ana)[0]

            for idate_ana in np.arange(0,ndates_ana,1):
                date_ana = dates_ana[idate_ana]
                ind_date = bhvevents_aligned_FR_allevents_all_dates_df['dates']==date_ana
                ind_date_FRPCA = FRPCAfeatures_all_sessions_allevents_sum_df['session']==date_ana

                # get the neurons 
                neurons_ana = np.unique(bhvevents_aligned_FR_allevents_all_dates_df[ind_animal & ind_cond & ind_date]['clusterID'])
                nneurons = np.shape(neurons_ana)[0]

                # get the gaze related features
                features_ana = FRPCAfeatures_all_sessions_allevents_sum_df[ind_animal_FRPCA&ind_cond_FRPCA&ind_date_FRPCA]
                
                # Determine subplot grid (5 columns, dynamic rows)
                ncols = 5
                nrows = int(np.ceil(nneurons / ncols))

                fig, axes = plt.subplots(nrows, ncols, figsize=(ncols * 6, nrows * 6), constrained_layout=True)
                axes = np.ravel(axes)  # Flatten for easy indexing
                
                # === New heatmap plot per date for neuron correlation over time ===
                fig_corr, ax_corr = plt.subplots(figsize=(10, max(6, 0.3 * nneurons)))

                # Store r_trace and p_trace for each neuron
                r_traces_all_neurons = []
                p_traces_all_neurons = []

                for ineuron in np.arange(0,nneurons,1):
                    clusterID_ineuron = neurons_ana[ineuron]
                    ind_neuron = bhvevents_aligned_FR_allevents_all_dates_df['clusterID']==clusterID_ineuron

                    ax = axes[ineuron]  # Get the subplot for this neuron

                    for ibhvname_ana in np.arange(0,nbhvnames_to_ana,1):
                        bhvname_ana = bhv_names_to_ana[ibhvname_ana]
                        ind_bhv = bhvevents_aligned_FR_allevents_all_dates_df['bhv_name']==bhvname_ana

                        ind_ana = ind_animal & ind_bhv & ind_cond & ind_neuron & ind_date 

                        bhvevents_aligned_FR_allevents_tgt = bhvevents_aligned_FR_allevents_all_dates_df[ind_ana]
                            
                        #
                        # load and plot bhv event ('gazestart'/'gazestop') aligned FR
                        FRs_allevents_ineuron = np.array(bhvevents_aligned_FR_allevents_tgt['FR_allevents'])[0]

                        nevents = np.shape(FRs_allevents_ineuron)[1]

                        FRsmoothed_allevents_ineuron = gaussian_filter1d(FRs_allevents_ineuron, sigma=6, axis=0)

                        # separating FR based on the gaze related features (three quantiles)
                        # xxx_type = 'gaze_duration'
                        xxx_type = 'gaze_accumulation'
                        if xxx_type == 'gaze_duration':
                            gaze_durs_ineurons = np.array(features_ana['gazestop_time']-\
                                                          features_ana['gazestart_time'])
                        elif xxx_type == 'gaze_accumulation':
                            gaze_durs_ineurons = np.array(features_ana['gaze_accum'])
                        
                        ngazes = np.shape(gaze_durs_ineurons)[0]
                        
                        print(date_ana+' pull aligned FR # = '+str(nevents)+'; pull aligned gaze # = '+str(ngazes))
                        
                        # Compute quantiles
                        q1, q2 = np.nanpercentile(gaze_durs_ineurons, [33, 66])

                        # Get indices for each quantile group
                        short_idx = np.where(gaze_durs_ineurons <= q1)[0]
                        mid_idx   = np.where((gaze_durs_ineurons > q1) & (gaze_durs_ineurons <= q2))[0]
                        long_idx  = np.where(gaze_durs_ineurons > q2)[0]

                        quantile_groups = {
                            'Short': short_idx,
                            'Medium': mid_idx,
                            'Long': long_idx
                        }

                        quantile_colors = {
                            'Short': '#1f77b4',   # blue
                            'Medium': '#ff7f0e',  # orange
                            'Long': '#2ca02c'     # green
                        }

                        time_trace = np.arange(-4,4,1/fps)
                        
                        # Plot FRs by quantile
                        for label, idx in quantile_groups.items():
                            if len(idx) == 0:
                                continue

                            mean_trace = np.nanmean(FRsmoothed_allevents_ineuron[:, idx], axis=1)
                            sem_trace = np.nanstd(FRsmoothed_allevents_ineuron[:, idx], axis=1) / np.sqrt(len(idx))

                            ax.plot(time_trace, mean_trace, label=f"{bhvname_ana} - {label} (n={len(idx)})",
                                    color=quantile_colors[label])
                            ax.fill_between(time_trace, mean_trace - sem_trace, mean_trace + sem_trace,
                                            color=quantile_colors[label], alpha=0.3)
                        
                        
                        # plot the correlation coefficient
                        # === Compute time-varying correlation between FR and gaze duration ===
                        n_timepoints = FRsmoothed_allevents_ineuron.shape[0]
                        r_trace = np.full(n_timepoints, np.nan)
                        p_trace = np.full(n_timepoints, np.nan)

                        if xxx_type == 'gaze_duration':
                            gaze_durs = np.array(features_ana['gazestop_time']-\
                                                          features_ana['gazestart_time'])
                        elif xxx_type == 'gaze_accumulation':
                            gaze_durs = np.array(features_ana['gaze_accum'])
                            
                        valid_gaze_mask = ~np.isnan(gaze_durs)

                        for t in range(n_timepoints):
                            fr_t = FRsmoothed_allevents_ineuron[t, :]
                            valid_fr_mask = ~np.isnan(fr_t)
                            valid_mask = valid_fr_mask & valid_gaze_mask

                            if np.sum(valid_mask) > 2:
                                r, p = st.pearsonr(fr_t[valid_mask], gaze_durs[valid_mask])
                                r_trace[t] = r
                                p_trace[t] = p
                                
                        r_traces_all_neurons.append(r_trace)
                        p_traces_all_neurons.append(p_trace)

                        # === Plot on right Y-axis ===
                        ax2 = ax.twinx()
                        ax2.plot(time_trace, r_trace, color='black', linestyle='-', linewidth=2, label='Gaze-FR Corr')
                        ax2.set_ylabel("Corr(GazeDur, FR)", color='black')
                        ax2.tick_params(axis='y', labelcolor='black')
                        ax2.set_ylim([-1, 1])
                        ax2.axhline(0, color='gray', linestyle='--', linewidth=1)

                        # Plot significant time points (p < 0.01) as red dots
                        sig_mask = (p_trace < 0.01) & (~np.isnan(p_trace)) & (~np.isnan(r_trace))
                        ax2.plot(time_trace[sig_mask], r_trace[sig_mask], 'ro', markersize=4, label='p < 0.01')
                        
                    ax.set_title(f"Neuron {clusterID_ineuron}")
                    ax.set_xlabel("Time (s)")
                    ax.set_ylabel("Firing Rate (a.u.)")
                    # ax.set_title(act_animal_ana+' '+cond_ana+' '+date_ana+' cell#'+clusterID_ineuron)
                    ax.legend()
                    #
                    ax2.set_ylabel("Corr(GazeDur, FR)", color='black')
                    ax2.tick_params(axis='y', labelcolor='black')
                    ax2.set_ylim([-1, 1])

                # Hide empty subplots if nneurons < total grid size
                for i in range(nneurons, len(axes)):
                    fig.delaxes(axes[i])

                # Figure title
                fig.suptitle(f"{act_animal_ana} {cond_ana} {date_ana}", fontsize=14)                
                
                
                # Convert to numpy array for heatmap
                r_traces_all_neurons = np.array(r_traces_all_neurons)

                # === Sort r_traces by the time of their first peak ===
                peak_times = []
                for trace in r_traces_all_neurons:
                    if np.all(np.isnan(trace)):
                        peak_times.append(np.inf)
                    else:
                        peak_idx = np.nanargmax(trace)
                        peak_times.append(time_trace[peak_idx])

                # Get sorting indices based on peak times
                sorted_indices = np.argsort(peak_times)
                r_traces_sorted = r_traces_all_neurons[sorted_indices, :]

                # Plot heatmap of r values
                im = ax_corr.imshow(r_traces_sorted, aspect='auto', cmap='gray_r', interpolation='none',
                                    extent=[time_trace[0], time_trace[-1], 0, nneurons],
                                    vmin=-0.7, vmax=0.7)

                # Overlay significance as red dots
                for i, idx in enumerate(sorted_indices):
                    sig_times = np.where(p_traces_all_neurons[idx] < 0.01)[0]
                    for t in sig_times:
                        ax_corr.plot(time_trace[t], i + 0.5, 'r.', markersize=3)  # i+0.5 to center in the row

                # Add vertical dashed line at time = 0
                ax_corr.axvline(x=0, color='black', linestyle='--', linewidth=1)

                # Add vertical dashed line at time zero
                ax_corr.axvline(x=0, linestyle='--', color='gray', linewidth=1)

                ax_corr.set_xlabel("Time (s)")
                ax_corr.set_ylabel("Neuron (sorted by peak time)")
                ax_corr.set_title(f"Neuron-wise Corr(Gaze, FR) Heatmap (Sorted): {act_animal_ana} {cond_ana} {date_ana}")
                cbar = fig_corr.colorbar(im, ax=ax_corr)
                cbar.set_label('Pearson r')
                
                # plt.show()
                
                savefig = 0
                if savefig:
                    figsavefolder = data_saved_folder+"fig_for_basic_neural_analysis_allsessions_basicEvents_PCA_makeBhvNeuronVideos_Pullfocused_continuousBhv/"+cameraID+"/"+animal1_filenames[0]+"_"+animal2_filenames[0]+'/'+date_ana+'/'

                    if not os.path.exists(figsavefolder):
                        os.makedirs(figsavefolder)

                    fig.savefig(figsavefolder+'individualneurons_FR_in_quantiles_'+
                                 pull_trig_events_tgtname+'_'+gaze_duration_type+'_'+xxx_type+savefile_sufix+'.pdf')

                    fig_corr.savefig(figsavefolder + 'neuron_corr_heatmap_' + 
                                     pull_trig_events_tgtname+'_'+gaze_duration_type+'_'+xxx_type+savefile_sufix+'.pdf')

                # Close the figures to avoid memory issues
                plt.close(fig)
                plt.close(fig_corr)

## analysis with 'trial pooling' across sessions from the same condition
### pool sessions for each task conditions together and then run PCA
#### pool sessions based on quantiles of gaze-accumulation or gaze-length variables (e.g. 5 quantiles)

In [ ]:
# only analyze a subset of conditions
# act_animals_to_ana = np.unique(bhvevents_aligned_FR_allevents_all_dates_df['act_animal'])
# act_animals_to_ana = ['kanga']
act_animals_to_ana = ['dodson']
# act_animals_to_ana = ['kanga_partner'] # align thing to partner, remember to pair this with pull_trig_events_tgtname = 'othergaze_prob'

nanimal_to_ana = np.shape(act_animals_to_ana)[0]
#
# bhv_names_to_ana = np.unique(bhvevents_aligned_FR_allevents_all_dates_df['bhv_name'])
bhv_names_to_ana = ['pull']
# bhv_names_to_ana = ['failpull']
# bhv_names_to_ana = ['succpull']
# bhv_names_to_ana = ['succpull','failpull']
nbhvnames_to_ana = np.shape(bhv_names_to_ana)[0]
bhvname_clrs = ['r','y','g','b','c','m','#458B74','#FFC710','#FF1493','#A9A9A9','#8B4513']
#
# # the following analysis can only do one conditions 
# # multiple condition will be considered into one conditions for quantile and FR averaging analysis
# conditions_to_ana = np.unique(bhvevents_aligned_FR_allevents_all_dates_df['condition'])
# conditions_to_ana = ['MC',]
# conditions_to_ana = ['MC','SR',]
###
# For Kanga
# conditions_to_ana = ['MC', 'MC_withDodson','MC_withGinger', 'MC_withKoala', 'MC_withVermelho', ] # all MC
# conditions_to_ana = ['SR', 'SR_withDodson', ] # all SR
# conditions_to_ana = ['MC', 'MC_withDodson', 'MC_withVermelho', ] # MC with male
# conditions_to_ana = ['MC_withGinger', 'MC_withKoala', ] # MC with female
# conditions_to_ana = ['MC', ] # MC with familiar male
# conditions_to_ana = ['MC_withGinger', ] # MC with familiar female
# conditions_to_ana = ['MC_withDodson', 'MC_withVermelho', ] # MC with unfamiliar male
# conditions_to_ana = ['MC_withKoala', ] # MC with unfamiliar female
# conditions_to_ana = ['MC_DannonAuto'] # partner AL
# conditions_to_ana = ['MC_KangaAuto'] # self AL
# conditions_to_ana = ['NV','NV_withDodson'] # NV
# conditions_to_ana = ['MC', 'MC_withDodson','MC_withGinger', 'MC_withKoala', 'MC_withVermelho', 
#                      'SR', 'SR_withDodson',]
###
# For Dodson
conditions_to_ana = ['MC', 'MC_withGingerNew', 'MC_withKanga', 'MC_withKoala', ] # all MC
# conditions_to_ana = ['MC', 'MC_withKanga', ] # all MC
# conditions_to_ana = ['SR', 'SR_withGingerNew', 'SR_withKanga', 'SR_withKoala', ] # all SR
# conditions_to_ana = ['MC', 'MC_withKanga', 'MC_withKoala', ] # all MC, no gingerNew
# conditions_to_ana = ['SR', 'SR_withKanga', 'SR_withKoala', ] # all SR,  no gingerNew
# conditions_to_ana = ['MC', 'MC_withGingerNew', 'MC_withKanga', 'MC_withKoala', ] # MC with female
# conditions_to_ana = ['MC', 'MC_withGingerNew', ] # MC with familiar female
# conditions_to_ana = ['MC_withKanga', 'MC_withKoala', ] # MC with unfamiliar female
# conditions_to_ana = ['MC_KoalaAuto_withKoala'] # partner AL
# conditions_to_ana = ['MC_DodsonAuto_withKoala'] # self AL
# conditions_to_ana = ['NV_withKanga'] # NV
# conditions_to_ana = ['MC', 'MC_withGingerNew', 'MC_withKanga', 'MC_withKoala', 
#                      'SR', 'SR_withGingerNew', 'SR_withKanga', 'SR_withKoala', ]

cond_toplot_type = 'allMC'

nconds_to_ana = np.shape(conditions_to_ana)[0]

doOnlySigniNeurons = 0 # define the significant neurons using the previous code

# newly added control!!
# only look at pull aligned events that has no preceding self pull 
doSingleSelfPulls = 0

# only look at pull aligned events that do not have juice deliever effect from the previous pull
doFarLastRewards = 0

# only look at pulls that has large reaction time
doLargeRTpulls = 0
largeRTthreshold = 2 # the threshold for defining large threahold

doSuccPull = 0
doFailPull = 0

In [ ]:
import numpy as np
import pandas as pd
import scipy.signal


if 1:
    # load and prepare the data
    bhvevents_aligned_FR_and_eventFeatures_all_dates_df = pd.DataFrame(columns=['condition', 'session', 'act_animal',
                                                                                 'bhv_name', 'bhv_id', 'FR_ievent',
                                                                                 'clusterID', 'channelID',
                                                                                 'gaze_accum_ievent', 'gaze_start_ievent',
                                                                                 'gaze_stop_ievent',
                                                                                 ])
    #
    bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df = pd.DataFrame()

    # step 1 for the bhvevents_aligned_FR_allevents_all_dates_df, get and gaze-related variables and calculate features

    # it's better to match this variable with the previous one
    # add three kinds of gaze duration definition (around pull, before pull, after pull)
    gaze_duration_type = 'before_pull'  # 'around_pull', 'before_pull', 'after_pull'
    # xxx_type = 'gaze_accum'  # 'gaze_dur' or 'gaze_accum' or 'pull_rt' or other
    xxx_type = 'pull_rt'

    # special here, number of quantile to use for pooling across different days
    num_quantiles = 3 # 10

    #
    for icond_ana in np.arange(0, nconds_to_ana, 1):
        cond_ana = conditions_to_ana[icond_ana]
        ind_cond = bhvevents_aligned_FR_allevents_all_dates_df['condition'] == cond_ana

        for ianimal_ana in np.arange(0, nanimal_to_ana, 1):
            act_animal_ana = act_animals_to_ana[ianimal_ana]

            if not 'partner' in act_animal_ana:
                ind_animal = bhvevents_aligned_FR_allevents_all_dates_df['act_animal']==act_animal_ana
            elif 'partner' in act_animal_ana:
                animal_dontwant = act_animal_ana.split('_')[0]
                ind_animal = bhvevents_aligned_FR_allevents_all_dates_df['act_animal']!=animal_dontwant
                
            # get the dates
            dates_ana = np.unique(bhvevents_aligned_FR_allevents_all_dates_df[ind_animal & ind_cond]['dates'])
            ndates_ana = np.shape(dates_ana)[0]

            for idate_ana in np.arange(0, ndates_ana, 1):
                date_ana = dates_ana[idate_ana]
                ind_date = bhvevents_aligned_FR_allevents_all_dates_df['dates'] == date_ana

                # get the neurons
                neurons_ana = np.unique(
                    bhvevents_aligned_FR_allevents_all_dates_df[ind_animal & ind_cond & ind_date]['clusterID'])
                nneurons = np.shape(neurons_ana)[0]

                for ineuron in np.arange(0, nneurons, 1):
                    clusterID_ineuron = neurons_ana[ineuron]
                    ind_neuron = bhvevents_aligned_FR_allevents_all_dates_df['clusterID'] == clusterID_ineuron

                    for ibhvname_ana in np.arange(0, nbhvnames_to_ana, 1):
                        bhvname_ana = bhv_names_to_ana[ibhvname_ana]
                        ind_bhv = bhvevents_aligned_FR_allevents_all_dates_df['bhv_name'] == bhvname_ana

                        ind_ana = ind_animal & ind_bhv & ind_cond & ind_neuron & ind_date

                        bhvevents_aligned_FR_allevents_tgt = bhvevents_aligned_FR_allevents_all_dates_df[
                            ind_ana].copy()  # Use .copy() to avoid SettingWithCopyWarning

                        if not bhvevents_aligned_FR_allevents_tgt.empty:
                            channelID = bhvevents_aligned_FR_allevents_tgt['channelID'].iloc[0]

                            #
                            # load and plot bhv event ('pull') aligned FR
                            FRs_allevents_ineuron = np.array(
                                bhvevents_aligned_FR_allevents_tgt['FR_allevents'].iloc[0])
                            nevents_FR = np.shape(FRs_allevents_ineuron)[1]

                            #
                            # load and plot the pull aligned continuous bhv variables
                            conBhv_allevents_ineuron = np.array(
                                bhvevents_aligned_FR_allevents_tgt[pull_trig_events_tgtname].iloc[0])
                            conBhv_allevents_ineuron = np.array(conBhv_allevents_ineuron)
                            conBhv_allevents_ineuron = conBhv_allevents_ineuron.transpose()
                            #
                            nevents_bhv = np.shape(conBhv_allevents_ineuron)[1]
                            
                            #
                            # load the self pull variables
                            selfpull_allevents_ineuron = np.array(
                                bhvevents_aligned_FR_allevents_tgt[pull_trig_selfpull_name].iloc[0])
                            selfpull_allevents_ineuron = np.array(selfpull_allevents_ineuron)
                            selfpull_allevents_ineuron = selfpull_allevents_ineuron.transpose()
                            
                            # 
                            # load the pull reaction time information
                            pull_rt_allevents_ineuron = np.array(
                                bhvevents_aligned_FR_allevents_tgt[pull_rt_name].iloc[0])
                            
                            # 
                            # load the last reward time information
                            lastreward_time_allevents_ineuron = np.array(
                                bhvevents_aligned_FR_allevents_tgt[pull_time_pre_reward_name].iloc[0])
                            
                            # 
                            # load the successful pull or not information
                            succornot_allevents_ineuron = np.array(
                                bhvevents_aligned_FR_allevents_tgt['succornot_pull'].iloc[0])
                            
                            
                            # if the pull aligned FR and bhv have different number
                            if not nevents_FR == nevents_bhv:
                                # print(date_ana+' mismatched number')
                                if nevents_FR < nevents_bhv:
                                    conBhv_allevents_ineuron = conBhv_allevents_ineuron[:, 0:nevents_FR]
                                else:
                                    FRs_allevents_ineuron = FRs_allevents_ineuron[:, 0:nevents_bhv]

                            #
                            nevents = np.min([nevents_FR, nevents_bhv])

                            # get each bhv events
                            for bhv_id in np.arange(0, nevents, 1):
                                FRs_ievent_ineuron = FRs_allevents_ineuron[:, bhv_id]
                                conBhv_ievent_ineuron = conBhv_allevents_ineuron[:, bhv_id]
                                selfpull_ievent_ineuron = selfpull_allevents_ineuron[:, bhv_id]
                                pull_rt_ievent_ineuron = pull_rt_allevents_ineuron[bhv_id]
                                try:
                                    lastreward_time_ievent_ineuron = -lastreward_time_allevents_ineuron[bhv_id]
                                except:
                                    lastreward_time_ievent_ineuron = np.nan
                                succornot_ievent_ineuron = succornot_allevents_ineuron[bhv_id]

                                #
                                # count self pull before self pull, 
                                # the goal is to make sure the effect we will find later is not because of the preceding pulls
                                from scipy.signal import find_peaks
                                #
                                x_full = np.arange(-4,4,1/fps)
                                pre_mask = x_full<-0.15
                                try:
                                    # selfpull_num = np.trapz(pull_trig_selfpull_ievent[pre_mask], dx=dt)
                                    data = selfpull_ievent_ineuron[pre_mask]
                                    peaks, _ = find_peaks(data)
                                    selfpull_num = len(peaks)
                                except:
                                    selfpull_num = np.nan
                                                              
                            
                        
                                #
                                # for the socialgaze_prob, only use the meaningful ones
                                if 0:
                                    if (pull_trig_events_tgtname == 'socialgaze_prob') | \
                                       (pull_trig_events_tgtname == 'othergaze_prob') :
                                        trace = conBhv_ievent_ineuron
                                        time_trace = np.arange(-4, 4, 1/30)
                                        filtered_trace = keep_closest_cluster_single_trial(trace, time_trace)
                                        conBhv_ievent_ineuron = filtered_trace
                            
                        
                                #
                                # analyze the pull triggered behavioral events
                                if (pull_trig_events_tgtname == 'socialgaze_prob') | \
                                   (pull_trig_events_tgtname == 'othergaze_prob') :
                                    # calculate the gaze start and gaze stop time, and finally gaze duration
                                    try:
                                        timewins = np.arange(-4, 4, 1 / fps)  # make sure it align with the setting in the previous section

                                        if 1:
                                            first_increase_idx = np.where(
                                                np.diff(conBhv_ievent_ineuron) > 0)[0][0] + 1
                                            #
                                            last_decrease_idx = np.where(
                                                np.diff(conBhv_ievent_ineuron) < 0)[0][-1] + 1  # Find last decrease
                                            #
                                            gazestart_time = timewins[first_increase_idx].copy()
                                            gazestop_time = timewins[last_decrease_idx].copy()
                                        if 0:
                                            # Find peaks
                                            peaks, _ = scipy.signal.find_peaks(conBhv_ievent_ineuron)
                                            #
                                            # Get first and last peak
                                            first_peak = peaks[0]
                                            last_peak = peaks[-1]
                                            #
                                            gazestart_time = timewins[first_peak].copy()
                                            gazestop_time = timewins[last_peak].copy()
                                        #
                                        # change the gazestart and gazestop time based on the gaze duration definition
                                        if gaze_duration_type == 'around_pull':
                                            gazestart_time = gazestart_time
                                            gazestop_time = gazestop_time
                                        if gaze_duration_type == 'before_pull':
                                            if (gazestart_time > 0):
                                                gazestart_time = np.nan
                                                gazestop_time = np.nan
                                            elif (gazestop_time > 0):
                                                gazestop_time = 0
                                        if gaze_duration_type == 'after_pull':
                                            if (gazestop_time < 0):
                                                gazestart_time = np.nan
                                                gazestop_time = np.nan
                                            elif (gazestart_time < 0):
                                                gazestart_time = 0

                                        #
                                        if (gazestop_time < gazestart_time):
                                            gazestart_time = np.nan
                                            gazestop_time = np.nan
                                    except:
                                        gazestart_time = np.nan
                                        gazestop_time = np.nan

                                    # calculate the gaze accumulation (use auc to estimate)
                                    try:
                                        timewins = np.arange(-4, 4, 1 / fps)  # make sure it align with the setting in the previous section
                                        dt = 1 / fps  # sampling interval in seconds
                                        #
                                        if gaze_duration_type == 'around_pull':
                                            auc = np.trapz(conBhv_ievent_ineuron, dx=dt)
                                        if gaze_duration_type == 'before_pull':
                                            auc = np.trapz(conBhv_ievent_ineuron[timewins < 0], dx=dt)
                                            # auc = np.trapz(conBhv_ievent_ineuron[(timewins<0)&(timewins>-3)], dx=dt)
                                        if gaze_duration_type == 'after_pull':
                                            auc = np.trapz(conBhv_ievent_ineuron[timewins > 0], dx=dt)
                                        #
                                        gaze_accum = auc
                                    #
                                    except:
                                        gaze_accum = np.nan
                                        
                                    # remove trial with zero gaze_accumulation
                                    if gaze_accum == 0:
                                        gaze_accum = np.nan

                                bhvevents_aligned_FR_and_eventFeatures_all_dates_df = pd.concat(
                                    [bhvevents_aligned_FR_and_eventFeatures_all_dates_df, pd.DataFrame({
                                        'condition': [cond_ana],
                                        'act_animal': [act_animal_ana],
                                        'bhv_name': [bhvname_ana],
                                        'session': [date_ana],
                                        'bhv_id': [bhv_id],
                                        'clusterID': [clusterID_ineuron],
                                        'channelID': [channelID],
                                        'gaze_accum_ievent': [gaze_accum],
                                        'gaze_start_ievent': [gazestart_time],
                                        'gaze_stop_ievent': [gazestop_time],
                                        'FR_ievent': [FRs_ievent_ineuron],
                                        'selfpull_num_ievent': [selfpull_num],
                                        'pull_rt_ievent':[pull_rt_ievent_ineuron],
                                        'lastreward_time_ievent':[lastreward_time_ievent_ineuron],
                                        'succornot_pull_ievent':[succornot_ievent_ineuron],
                                         pull_trig_events_tgtname:[conBhv_ievent_ineuron],
                                    })], ignore_index=True)

    # only look at successful pull
    if doSuccPull:
        ind_good = bhvevents_aligned_FR_and_eventFeatures_all_dates_df['succornot_pull_ievent']==1
        bhvevents_aligned_FR_and_eventFeatures_all_dates_df = bhvevents_aligned_FR_and_eventFeatures_all_dates_df[ind_good]
    
    # only look at failed pull
    if doFailPull:
        ind_good = bhvevents_aligned_FR_and_eventFeatures_all_dates_df['succornot_pull_ievent']==0
        bhvevents_aligned_FR_and_eventFeatures_all_dates_df = bhvevents_aligned_FR_and_eventFeatures_all_dates_df[ind_good]
    
                                
    # only look at the pulls that has no pull preceeding
    if doSingleSelfPulls:
        ind_good = bhvevents_aligned_FR_and_eventFeatures_all_dates_df['selfpull_num_ievent']==0
        bhvevents_aligned_FR_and_eventFeatures_all_dates_df = bhvevents_aligned_FR_and_eventFeatures_all_dates_df[ind_good]
    
    #
    # remove events that still has the effect from previous juice
    if doFarLastRewards:
        # reward should be beyond the -4s time window we are looking at, and also consider the reward effect last ~1s
        ind_good = bhvevents_aligned_FR_and_eventFeatures_all_dates_df['lastreward_time_ievent']<=(-4-1)
        bhvevents_aligned_FR_and_eventFeatures_all_dates_df = bhvevents_aligned_FR_and_eventFeatures_all_dates_df[ind_good]
                    
    #
    # only consider pulls that has large reaction time
    if doLargeRTpulls:
        # reward should be beyond the -4s time window we are looking at, and also consider the reward effect last ~1s
        ind_good = bhvevents_aligned_FR_and_eventFeatures_all_dates_df['pull_rt_ievent']>=largeRTthreshold
        bhvevents_aligned_FR_and_eventFeatures_all_dates_df = bhvevents_aligned_FR_and_eventFeatures_all_dates_df[ind_good]
                    
    #
    # remove trial with zero (nan) in the gaze accumulation level
    ind_nan = np.isnan(bhvevents_aligned_FR_and_eventFeatures_all_dates_df['gaze_accum_ievent'])
    bhvevents_aligned_FR_and_eventFeatures_all_dates_df = bhvevents_aligned_FR_and_eventFeatures_all_dates_df[~ind_nan]

            
    # add the quantile information using the all sessions' data
    # consider each date separately

    # Create a list to store the DataFrames for each date
    all_dates_dfs = []

    # Get unique dates
    unique_dates = bhvevents_aligned_FR_and_eventFeatures_all_dates_df['session'].unique()

    for date_ana in unique_dates:
        date_df = bhvevents_aligned_FR_and_eventFeatures_all_dates_df[
            bhvevents_aligned_FR_and_eventFeatures_all_dates_df['session'] == date_ana].copy()

        if xxx_type == 'gaze_accum':
            quantile_tgt = date_df['gaze_accum_ievent'].dropna()  # Handle potential NaNs
            n_unique = len(quantile_tgt.unique())
            n_bins = min(num_quantiles, n_unique - 1)  # Calculate the maximum possible bins
            if n_bins > 1:  # Only proceed if we can make at least 2 bins
                try:
                    quantile_bins = np.nanquantile(quantile_tgt, np.linspace(0, 1, num_quantiles + 1))
                    date_df['gaze_accum_quantile'] = pd.cut(
                        date_df['gaze_accum_ievent'],
                        bins=quantile_bins,
                        labels=False,
                        include_lowest=True,
                        duplicates='drop'  # Drop duplicate bin edges
                    )
                    quantile_col = 'gaze_accum_quantile'
                    title_prefix = 'Gaze Accumulation'
                except ValueError as e:
                    print(f"Warning: Error calculating quantiles on date {date_ana}: {e}")
                    date_df['gaze_accum_quantile'] = np.nan
                    quantile_col = 'gaze_accum_quantile'
                    title_prefix = 'Gaze Accumulation'
            else:
                date_df['gaze_accum_quantile'] = np.nan
                quantile_col = 'gaze_accum_quantile'
                title_prefix = 'Gaze Accumulation'
                print(f"Warning: Not enough distinct data for quantiles on date {date_ana}")

        #
        elif xxx_type == 'gaze_dur':
            date_df['gaze_duration_ievent'] = \
                date_df['gaze_stop_ievent'] - \
                date_df['gaze_start_ievent']
            quantile_tgt = date_df['gaze_duration_ievent'].dropna()  # Handle potential NaNs
            n_unique = len(quantile_tgt.unique())
            n_bins = min(num_quantiles, n_unique - 1)  # Calculate the maximum possible bins
            if n_bins > 1:  # Only proceed if we can make at least 2 bins
                try:
                    quantile_bins = np.nanquantile(quantile_tgt, np.linspace(0, 1, num_quantiles + 1))
                    date_df['gaze_duration_quantile'] = pd.cut(
                        date_df['gaze_duration_ievent'],
                        bins=quantile_bins,
                        labels=False,
                        include_lowest=True,
                        duplicates='drop'  # Drop duplicate bin edges
                    )
                    quantile_col = 'gaze_duration_quantile'
                    title_prefix = 'Gaze Duration'
                except ValueError as e:
                    print(f"Warning: Error calculating quantiles on date {date_ana}: {e}")
                    date_df['gaze_duration_quantile'] = np.nan
                    quantile_col = 'gaze_duration_quantile'
                    title_prefix = 'Gaze Duration'
            else:
                date_df['gaze_duration_quantile'] = np.nan
                quantile_col = 'gaze_duration_quantile'
                title_prefix = 'Gaze Duration'
                print(f"Warning: Not enough distinct data for quantiles on date {date_ana}")
        
        #
        else:
            quantile_tgt = date_df[xxx_type+'_ievent'].dropna()  # Handle potential NaNs
            n_unique = len(quantile_tgt.unique())
            n_bins = min(num_quantiles, n_unique - 1)  # Calculate the maximum possible bins
            if n_bins > 1:  # Only proceed if we can make at least 2 bins
                try:
                    quantile_bins = np.nanquantile(quantile_tgt, np.linspace(0, 1, num_quantiles + 1))
                    date_df[xxx_type+'_quantile'] = pd.cut(
                        date_df[xxx_type+'_ievent'],
                        bins=quantile_bins,
                        labels=False,
                        include_lowest=True,
                        duplicates='drop'  # Drop duplicate bin edges
                    )
                    quantile_col = xxx_type+'_quantile'
                    title_prefix = xxx_type
                except ValueError as e:
                    print(f"Warning: Error calculating quantiles on date {date_ana}: {e}")
                    date_df[xxx_type+'_quantile'] = np.nan
                    quantile_col = xxx_type+'_quantile'
                    title_prefix = xxx_type
            else:
                date_df[xxx_type+'_quantile'] = np.nan
                quantile_col = xxx_type+'_quantile'
                title_prefix = xxx_type
                print(f"Warning: Not enough distinct data for quantiles on date {date_ana}")

        all_dates_dfs.append(date_df)

    # Concatenate the DataFrames back together
    bhvevents_aligned_FR_and_eventFeatures_all_dates_df = pd.concat(all_dates_dfs, ignore_index=True)

    # average for each neuron the firing rate of each quantile
    if 'quantile_col' in locals():
        # Iterate through unique (clusterID, session) pairs
        for (cluster_id, session_id) in bhvevents_aligned_FR_and_eventFeatures_all_dates_df[
            ['clusterID', 'session']].drop_duplicates().itertuples(index=False):
            neuron_session_df = bhvevents_aligned_FR_and_eventFeatures_all_dates_df[
                (bhvevents_aligned_FR_and_eventFeatures_all_dates_df['clusterID'] == cluster_id) &
                (bhvevents_aligned_FR_and_eventFeatures_all_dates_df['session'] == session_id)
            ].copy()

            for q_val in neuron_session_df[quantile_col].dropna().unique():
                quantile_df = neuron_session_df[neuron_session_df[quantile_col] == q_val]
                if not quantile_df.empty:
                    all_fr_traces = np.vstack(quantile_df['FR_ievent'].tolist())
                    mean_fr_trace = np.nanmean(all_fr_traces, axis=0)

                    # Get representative metadata
                    condition = quantile_df['condition'].iloc[0]
                    act_animal = quantile_df['act_animal'].iloc[0]
                    bhv_name = quantile_df['bhv_name'].iloc[0]
                    channelID = quantile_df['channelID'].iloc[0]

                    new_row = {
                        'condition': condition,
                        'session': session_id,
                        'act_animal': act_animal,
                        'bhv_name': bhv_name,
                        'clusterID': cluster_id,
                        'channelID': channelID,
                        quantile_col: int(q_val),
                        'mean_FR_trace': mean_fr_trace
                    }
                    bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df = pd.concat(
                        [bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df,
                         pd.DataFrame([new_row])],
                        ignore_index=True
                    )

In [ ]:
print(np.unique(bhvevents_aligned_FR_and_eventFeatures_all_dates_df['selfpull_num_ievent']))
print(np.max(bhvevents_aligned_FR_and_eventFeatures_all_dates_df['lastreward_time_ievent']))
print(np.max(bhvevents_aligned_FR_and_eventFeatures_all_dates_df['succornot_pull_ievent']))

In [ ]:
bhvevents_aligned_FR_and_eventFeatures_all_dates_df.keys()

In [ ]:
## It's for defining slope ramping feature for each neurons, and regress over the gaze accumulation or other variables
if 1:    
    #
    bhveventsFeatures_and_FRslopes_all_dates_df = pd.DataFrame(columns=['condition', 'session', 'act_animal',
                                                                         'bhv_name', 'clusterID', 'channelID',                                                                   
                                                                                 ])
    
    # xvar_name = 'gaze_accum' # or 'gaze_accum', 'gaze_start', 'gaze_stop', 'selfpull_num','pull_rt'
    xvar_name = 'pull_rt' # or 'gaze_accum', 'gaze_start', 'gaze_stop', 'selfpull_num','pull_rt'','pull_rt'
    
    for icond_ana in np.arange(0, nconds_to_ana, 1):
        cond_ana = conditions_to_ana[icond_ana]
        ind_cond = bhvevents_aligned_FR_and_eventFeatures_all_dates_df['condition'] == cond_ana

        for ianimal_ana in np.arange(0, nanimal_to_ana, 1):
            act_animal_ana = act_animals_to_ana[ianimal_ana]
            ind_animal = bhvevents_aligned_FR_and_eventFeatures_all_dates_df['act_animal'] == act_animal_ana

            # get the dates
            dates_ana = np.unique(bhvevents_aligned_FR_and_eventFeatures_all_dates_df[ind_animal & ind_cond]['session'])
            ndates_ana = np.shape(dates_ana)[0]

            for idate_ana in np.arange(0, ndates_ana, 1):
                date_ana = dates_ana[idate_ana]
                ind_date = bhvevents_aligned_FR_and_eventFeatures_all_dates_df['session'] == date_ana

                # get the neurons
                neurons_ana = np.unique(
                    bhvevents_aligned_FR_and_eventFeatures_all_dates_df[ind_animal & ind_cond & ind_date]['clusterID'])
                nneurons = np.shape(neurons_ana)[0]

                for ineuron in np.arange(0, nneurons, 1):
                    clusterID_ineuron = neurons_ana[ineuron]
                    ind_neuron = bhvevents_aligned_FR_and_eventFeatures_all_dates_df['clusterID'] == clusterID_ineuron

                    for ibhvname_ana in np.arange(0, nbhvnames_to_ana, 1):
                        bhvname_ana = bhv_names_to_ana[ibhvname_ana]
                        ind_bhv = bhvevents_aligned_FR_and_eventFeatures_all_dates_df['bhv_name'] == bhvname_ana

                        ind_ana = ind_animal & ind_bhv & ind_cond & ind_neuron & ind_date

                        bhvevents_aligned_FR_and_eventFeatures_allevents_tgt = bhvevents_aligned_FR_and_eventFeatures_all_dates_df[
                            ind_ana].copy() 

                        channelID_ineuron = bhvevents_aligned_FR_and_eventFeatures_allevents_tgt['channelID'].iloc[0]
                        
                        #
                        # calculate the ramping slope and other FR related features
                        # get the firing rate
                        xxx = np.array(list(bhvevents_aligned_FR_and_eventFeatures_allevents_tgt['FR_ievent']))

                        # Time vector: from -4s to 4s (e.g., 240 points for 30Hz)
                        time_vector = np.arange(-4,4,1/fps)

                        # Step 1: Compute average firing rate and find time of max abs deviation before 0s
                        mean_fr = np.nanmean(xxx, axis=0)
                        pre_mean_fr = mean_fr[time_vector<0]

                        # Find index of max absolute deviation from baseline (i.e., strongest ramp up or down)
                        baseline = np.nanmean(mean_fr)  # optional: use entire -4 to 0s as baseline
                        abs_dev = np.abs(mean_fr - baseline)
                        peak_idx = np.argmax(abs_dev)
                        #
                        abs_dev = np.abs(pre_mean_fr - baseline)
                        prepull_peak_idx = np.argmax(abs_dev)

                        # Get corresponding index in the full time vector
                        peak_time = time_vector[peak_idx]
                        #
                        prepull_peak_time = time_vector[prepull_peak_idx]

                        
                        # Step 2: Compute slope from -4s to peak_time or 0 for each trial
                        #         in addition, get the firing rate at the peak time
                        slopes = []
                        # flexible end time or fixed end time?
                        dofixedendtime = 0
                        #
                        for trial_fr in xxx:
                            # 
                            if not dofixedendtime:
                                if peak_time < 0:
                                    x = time_vector[:peak_idx + 1]  # from -4s to peak_time
                                    y = trial_fr[:peak_idx + 1]
                                else:
                                    # zero_idx = np.where(time_vector>=0)[0][0]
                                    # x = time_vector[:zero_idx]  # from -4s to 0
                                    # y = trial_fr[:zero_idx]
                                    x = time_vector[:prepull_peak_idx + 1]  # from -4s to peak_time
                                    y = trial_fr[:prepull_peak_idx + 1]
                            #
                            elif dofixedendtime:
                                endtime_idx = np.where(time_vector>=-0.85)[0][0]
                                starttime_idx = np.where(time_vector>=-3.25)[0][0]
                                x = time_vector[starttime_idx:endtime_idx]  # from -4s to the peak time (-0.8s) based on the average FR
                                y = trial_fr[starttime_idx:endtime_idx]
                            #
                            slope, intercept, r_value, p_value, std_err = st.linregress(x, y)
                            slopes.append(slope)
                        #
                        slopes = np.array(slopes)  
                        #
                        # if the neuron fr is ramping down overall, flip the slopes
                        if np.nanmean(slopes)<0:
                            slopes = -slopes
                            
                        #
                        if not dofixedendtime:
                            if peak_time < 0:
                                peakFRs = xxx[:,peak_idx]
                            else:
                                # zero_idx = np.where(time_vector>=0)[0][0]
                                # peakFRs = xxx[:,zero_idx]
                                peakFRs = xxx[:,prepull_peak_idx]
                        # 
                        elif dofixedendtime:
                            endtime_idx = np.where(time_vector>=-0.85)[0][0]
                            peakFRs = xxx[:,endtime_idx]
                        
                        
                        
                        
                        # 
                        # get the gaze_accumulation or other variables 
                        xvar = np.array(list(bhvevents_aligned_FR_and_eventFeatures_allevents_tgt[xvar_name+'_ievent']))
                        
                        # 
                        # regression between FR slopes and xvar (e.g. gaze accumulation)
                        slope2, intercept2, r_value2, p_value2, std_err2 = st.linregress(xvar, slopes)
                        #
                        if (r_value2 > 0.99) | (r_value2 < -0.99):
                            r_value2 = np.nan
                            p_value2 = np.nan
                            
                        #
                        # correlation between the peak FR and xvar (e.g. gaze accumulation)
                        slope3, intercept3, r_value3, p_value3, std_err3 = st.linregress(xvar, peakFRs)
                        #
                        if (r_value3 > 0.99) | (r_value3 < -0.99):
                            r_value3 = np.nan
                            p_value3 = np.nan
                        
                        # put the data together
                        bhveventsFeatures_and_FRslopes_all_dates_df = pd.concat(
                                    [bhveventsFeatures_and_FRslopes_all_dates_df, pd.DataFrame({
                                        'condition': [cond_ana],
                                        'act_animal': [act_animal_ana],
                                        'bhv_name': [bhvname_ana],
                                        'session': [date_ana],
                                        'clusterID': [clusterID_ineuron],
                                        'channelID': [channelID_ineuron],
                                        'FRpeaktime': [peak_time],
                                        'FRpeaktime_prepull': [prepull_peak_time],
                                        'xvar_name': [xvar_name],
                                        'corr_prepull_FRslope_and_xvar':[r_value2],
                                        'pcorr_prepull_FRslope_and_xvar':[p_value2],
                                        'corr_prepull_peakFR_and_xvar':[r_value3],
                                        'pcorr_prepull_peakFR_and_xvar':[p_value3],
                                    })], ignore_index=True)
                        

    # do some plotting
    yvar_toplot_name = 'FRslope' # FRslope, peakFR
    
    #
    # Assuming df is already your DataFrame:
    df = bhveventsFeatures_and_FRslopes_all_dates_df.copy()

    # Define significance
    df['significance'] = df['pcorr_prepull_'+yvar_toplot_name+'_and_xvar'] < 0.05

    # Drop NaNs in correlation column before analysis
    df_clean = df.dropna(subset=['corr_prepull_'+yvar_toplot_name+'_and_xvar'])

    # Define significance
    df_clean['significance'] = df_clean['pcorr_prepull_'+yvar_toplot_name+'_and_xvar'] < 0.05

    # Count neurons
    n_sig = df_clean['significance'].sum()
    n_nonsig = len(df_clean) - n_sig

    # T-test against 0 (only on non-NaN correlations)
    t_stat, t_pval = st.ttest_1samp(df_clean['corr_prepull_'+yvar_toplot_name+'_and_xvar'], 0)
    meanvalue = np.nanmean(df_clean['corr_prepull_'+yvar_toplot_name+'_and_xvar'])

    # Plot
    figgg = plt.figure(figsize=(8, 5))
    seaborn.swarmplot(
        x='xvar_name',
        y='corr_prepull_'+yvar_toplot_name+'_and_xvar',
        data=df_clean,
        hue='significance',
        palette={True: 'red', False: 'gray'},
        size=6
    )

    plt.axhline(0, color='black', linestyle='--', linewidth=1)
    plt.ylabel("Correlation with Pre-pull "+yvar_toplot_name)
    plt.xlabel("Behavioral Variable")
    plt.title("Swarm Plot of Correlations per Variable in "+cond_toplot_type)

    # Custom legend with counts
    plt.legend(
        title=f'Significant (p < 0.05)\nRed: {n_sig}, Gray: {n_nonsig}',
        # labels=['False', 'True'],  # must match unique values in significance
        loc='best'
    )

    # Show t-test result
    plt.text(-0.3, np.nanmax(df['corr_prepull_'+yvar_toplot_name+'_and_xvar']) * 0.9,
             f'T-test vs 0: p = {t_pval:.3g};\n mean = {meanvalue:.2g}', ha='center', fontsize=12)

    plt.tight_layout()
    # plt.show()
    
    #
    # save the dataframe that have significant encoding information
    savedata = 1
    if savedata:
        datasavefolder = data_saved_folder+"data_for_basic_neural_analysis_allsessions_basicEvents_PCA_makeBhvNeuronVideos_Pullfocused_continuousBhv/"+\
                        cameraID+"/"+animal1_filenames[0]+"_"+animal2_filenames[0]+"/significant_neuron_info/"
        
        if not os.path.exists(datasavefolder):
            os.makedirs(datasavefolder)
            
        with open(datasavefolder+'/'+recordedanimals[0]+'_PullFocus_significant_neuron_of_encoding_'+xxx_type+'.pkl', 'wb') as f:
            pickle.dump(df_clean, f)  
            
        
        
    
    savefig = 1
    if savefig:
        figsavefolder = data_saved_folder+"fig_for_basic_neural_analysis_allsessions_basicEvents_PCA_makeBhvNeuronVideos_Pullfocused_continuousBhv/"+\
                        cameraID+"/"+animal1_filenames[0]+"_"+animal2_filenames[0]+"/FR_and_gaze_quantile_fig/"

        if not os.path.exists(figsavefolder):
            os.makedirs(figsavefolder)

        if doSuccPull:
            figgg.savefig(figsavefolder+cond_toplot_type+'_bhvevents_aligned_'+yvar_toplot_name+'_and_'+xvar_name+'_correlation_'+
                          'succpull'+'_'+pull_trig_events_tgtname+'_'+gaze_duration_type+'_'+savefile_sufix+'.pdf')
        elif doFailPull:
            figgg.savefig(figsavefolder+cond_toplot_type+'_bhvevents_aligned_'+yvar_toplot_name+'_and_'+xvar_name+'_correlation_'+
                          'failpull'+'_'+pull_trig_events_tgtname+'_'+gaze_duration_type+'_'+savefile_sufix+'.pdf')
        else:
            figgg.savefig(figsavefolder+cond_toplot_type+'_bhvevents_aligned_'+yvar_toplot_name+'_and_'+xvar_name+'_correlation_'+
                          bhvname_ana+'_'+pull_trig_events_tgtname+'_'+gaze_duration_type+'_'+savefile_sufix+'.pdf')
        
                        

In [ ]:
# time_vector[endtime_idx]

In [ ]:
plt.hist(bhveventsFeatures_and_FRslopes_all_dates_df['FRpeaktime_prepull'])

In [ ]:
timepointx = np.arange(-4,4,1/fps)
xxx = np.array(list(bhvevents_aligned_FR_and_eventFeatures_allevents_tgt['FR_ievent']))

plt.plot(timepointx,xxx[2,:])
plt.plot([0,0],[-1,4],'--')



In [ ]:
xxx = bhvevents_aligned_FR_and_eventFeatures_all_dates_df['pull_rt_ievent']
yyy = bhvevents_aligned_FR_and_eventFeatures_all_dates_df['gaze_accum_ievent']

ind_nan = np.isnan(xxx) | np.isnan(yyy)

plt.plot(xxx[~ind_nan],yyy[~ind_nan],'.')

rr,pp = st.pearsonr(xxx[~ind_nan],yyy[~ind_nan])
print('corr:'+str(rr))
print('pvalue:'+str(pp))

In [ ]:
##
# do some big picture plot; here is to have a sense of the gaze-accumulation distribution of succ and failed pull
# ...if the setting is set for that
## 
if 0:
    from scipy.stats import ks_2samp

    # to make the condition more general
    # Define the function for generalizing condition
    def generalize_condition(cond):
        if cond == "MC" or cond.startswith("MC_with"):
            return "MC"
        elif cond == "SR" or cond.startswith("SR_with"):
            return "SR"
        else:
            return cond  # default to original condition if no match

    # Apply the function to create the new column
    bhvevents_aligned_FR_and_eventFeatures_all_dates_df["condition_general"] = \
        bhvevents_aligned_FR_and_eventFeatures_all_dates_df["condition"].apply(generalize_condition)
    
    
    # 
    plt.figure(figsize=(20,6))
    data_toplot = bhvevents_aligned_FR_and_eventFeatures_all_dates_df
    # seaborn.kdeplot(data=data_toplot,x='gaze_accum_ievent',hue='bhv_name')
    seaborn.kdeplot(data=data_toplot,x='gaze_accum_ievent',hue='condition_general',
                   common_norm=False)  # don't normalize across groups
    # seaborn.histplot(data=data_toplot,x='gaze_accum_ievent',hue='bhv_name')

    # Compute group-wise quantiles (deciles from 10% to 90%)
    # quantiles_df = data_toplot.groupby('bhv_name')['gaze_accum_ievent'].quantile(np.linspace(0.1, 0.9, 9)).reset_index()
    quantiles_df = data_toplot.groupby('condition_general')['gaze_accum_ievent'].quantile(np.linspace(0.1, 0.9, 9)).reset_index()
    quantiles_df.rename(columns={'gaze_accum_ievent': 'quantile_value'}, inplace=True)

    # Draw vertical lines for each quantile
    if 0:
        # palette = dict(zip(data_toplot['bhv_name'].unique(), seaborn.color_palette()))  # color matching seaborn
        palette = dict(zip(data_toplot['condition_general'].unique(), seaborn.color_palette()))  # color matching seaborn

        for _, row in quantiles_df.iterrows():
            plt.axvline(
                row['quantile_value'],
                # color=palette[row['bhv_name']],
                color=palette[row['condition_general']],
                linestyle='--',
                alpha=1
            )

    plt.title('KDE Plot with Quantile Lines')
    plt.show()
    
    # KS test: compare MC vs SR
    if 0:
        group_MC = data_toplot[data_toplot['condition_general'] == 'MC']['gaze_accum_ievent'].dropna()
        group_SR = data_toplot[data_toplot['condition_general'] == 'SR']['gaze_accum_ievent'].dropna()

        ks_stat, ks_pval = ks_2samp(group_MC, group_SR)
        print(f"KS test: statistic={ks_stat:.4f}, p-value={ks_pval:.4g}")
    
    
    

In [ ]:
# only consider the significant neurons based on the previous analysis
if 1:
    if doOnlySigniNeurons:
        
        #
        # Rename 'session' column in the first DataFrame to 'dates' for merging
        bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df = bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df.rename(columns={'session': 'dates'})
        # Merge the DataFrames
        merged_df = pd.merge(bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df, significant_neurons_data_df,
                            on=['dates', 'act_animal', 'bhv_name', 'clusterID','condition'],
                            how='inner')
        # Filter for significant neurons
        significant_bhv_df = merged_df[merged_df['significance_or_not'] == True]
        significant_bhv_df = significant_bhv_df.rename(columns={'dates':'session'})
        bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df = significant_bhv_df

        #
        # Rename 'session' column in the first DataFrame to 'dates' for merging
        bhvevents_aligned_FR_and_eventFeatures_all_dates_df = bhvevents_aligned_FR_and_eventFeatures_all_dates_df.rename(columns={'session': 'dates'})
        # Merge the DataFrames
        merged_df = pd.merge(bhvevents_aligned_FR_and_eventFeatures_all_dates_df, significant_neurons_data_df,
                            on=['dates', 'act_animal', 'bhv_name', 'clusterID','condition'],
                            how='inner')
        # Filter for significant neurons
        significant_bhv_df = merged_df[merged_df['significance_or_not'] == True]
        significant_bhv_df = significant_bhv_df.rename(columns={'dates':'session'})
        bhvevents_aligned_FR_and_eventFeatures_all_dates_df = significant_bhv_df


In [ ]:
# bhvevents_aligned_FR_and_eventFeatures_all_dates_df
# bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df

In [ ]:
# step 1 - 2:
# do a basic plot for sanity check - mean FR across all units in the pool condition for each quantile
if 1:
    from scipy.integrate import cumtrapz

    doQuantMeanFRs = 1
    # only do one example session, all neurons in that session, average
    doExampleSession = 0
    # only do one example neuron in one example cell 
    doExampleNeuron = 0
    
    timewins = np.arange(-4, 4, 1/30)
    n_timepoints = len(timewins)
        
    #
    if doExampleSession:
        # examplesess = '20240606'
        examplesess =  '20240808'
        #
        if doQuantMeanFRs:
            ind = bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df['session']==examplesess
            quantile_values = np.sort(bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df[ind][quantile_col].unique())
        else:
            ind = bhvevents_aligned_FR_and_eventFeatures_all_dates_df['session']==examplesess
            quantile_values = np.sort(bhvevents_aligned_FR_and_eventFeatures_all_dates_df[ind][quantile_col].unique())      
    # 
    elif doExampleNeuron:
        examplesess =  '20240808'
        exampleneuron = '129'
        # 
        # has to do not QuantMeanFRs
        ind_1 = bhvevents_aligned_FR_and_eventFeatures_all_dates_df['session']==examplesess
        ind_2 = bhvevents_aligned_FR_and_eventFeatures_all_dates_df['clusterID']==exampleneuron
        quantile_values = np.sort(bhvevents_aligned_FR_and_eventFeatures_all_dates_df[ind_1&ind_2][quantile_col].unique()) 
    #
    else:
        if doQuantMeanFRs:
            quantile_values = np.sort(bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df[quantile_col].unique())
        else:
            quantile_values = np.sort(bhvevents_aligned_FR_and_eventFeatures_all_dates_df[quantile_col].unique())
    
    quantile_values = quantile_values[~np.isnan(quantile_values)]
    
    y_label = 'Firing Rate (Hz)'
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    if doExampleNeuron:
        ax2 = ax.twinx()  # Secondary y-axis for AUC

    for i_quantile, q_val in enumerate(quantile_values):
    
        #
        if doExampleSession:
            if doQuantMeanFRs:
                ind_quantile = bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df[ind][quantile_col] == q_val
                fr_traces_quantile = np.vstack(bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df[ind][ind_quantile]['mean_FR_trace'].tolist())
            else:
                ind_quantile = bhvevents_aligned_FR_and_eventFeatures_all_dates_df[ind][quantile_col] == q_val
                fr_traces_quantile = np.vstack(bhvevents_aligned_FR_and_eventFeatures_all_dates_df[ind][ind_quantile]['FR_ievent'].tolist())
        #
        elif doExampleNeuron:
            ind_quantile = bhvevents_aligned_FR_and_eventFeatures_all_dates_df[ind_1&ind_2][quantile_col] == q_val
            fr_traces_quantile = np.vstack(bhvevents_aligned_FR_and_eventFeatures_all_dates_df[ind_1&ind_2][ind_quantile]['FR_ievent'].tolist())
            gaze_traces_quantile = np.vstack(bhvevents_aligned_FR_and_eventFeatures_all_dates_df[ind_1&ind_2][ind_quantile][pull_trig_events_tgtname].tolist())
     
        #
        else:
            if doQuantMeanFRs:
                ind_quantile = bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df[quantile_col] == q_val
                fr_traces_quantile = np.vstack(bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df[ind_quantile]['mean_FR_trace'].tolist())
            else:
                ind_quantile = bhvevents_aligned_FR_and_eventFeatures_all_dates_df[quantile_col] == q_val
                fr_traces_quantile = np.vstack(bhvevents_aligned_FR_and_eventFeatures_all_dates_df[ind_quantile]['FR_ievent'].tolist())

            
        #     
        mean_fr = np.nanmean(fr_traces_quantile, axis=0)
        sem_fr = np.nanstd(fr_traces_quantile, axis=0) / np.sqrt(np.sum(~np.isnan(fr_traces_quantile[:, 0]))) # Standard Error of the Mean
        
        ax.plot(timewins, mean_fr, label=f'Quantile {int(q_val)}')
        ax.fill_between(timewins, mean_fr - sem_fr, mean_fr + sem_fr, alpha=0.3)

        
        #
        if doExampleNeuron:
            # Accumulated AUC for each gaze trace, then average
            auc_gaze_all = np.array([cumtrapz(trace, timewins, initial=0) for trace in gaze_traces_quantile])
            mean_auc_gaze = np.nanmean(auc_gaze_all, axis=0)
            sem_auc_gaze = np.nanstd(auc_gaze_all, axis=0) / np.sqrt(np.sum(~np.isnan(auc_gaze_all[:, 0])))
            
            # Plot AUC Gaze
            ax2.plot(timewins, mean_auc_gaze, linestyle='--', color='lightblue', label=f'Gaze AUC {int(q_val)}')
            ax2.fill_between(timewins, mean_auc_gaze - sem_auc_gaze, mean_auc_gaze + sem_auc_gaze, alpha=0.2, color='lightblue')

        
        
    ax.set_xlabel('Time (s)')
    ax.set_ylabel(y_label)
    ax.set_title(f'Mean Firing Rate by {title_prefix} Quantile '+gaze_duration_type)
    ax.axvline(0, color='k', linestyle='--', linewidth=0.8, label='Pull Onset')
    ax.legend()
    
    if doExampleNeuron:
        ax2.set_ylabel('Accumulated Gaze AUC')
        lines1, labels1 = ax.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
    
    fig.tight_layout()

    
    
    savefig = 1
    if savefig:
        figsavefolder = data_saved_folder+"fig_for_basic_neural_analysis_allsessions_basicEvents_PCA_makeBhvNeuronVideos_Pullfocused_continuousBhv/"+\
                        cameraID+"/"+animal1_filenames[0]+"_"+animal2_filenames[0]+"/FR_and_gaze_quantile_fig/"
    
        if doExampleSession:
            figsavefolder = data_saved_folder+"fig_for_basic_neural_analysis_allsessions_basicEvents_PCA_makeBhvNeuronVideos_Pullfocused_continuousBhv/"+\
                        cameraID+"/"+animal1_filenames[0]+"_"+animal2_filenames[0]+"/"+examplesess+"/"
        
        if doExampleNeuron:
            figsavefolder = data_saved_folder+"fig_for_basic_neural_analysis_allsessions_basicEvents_PCA_makeBhvNeuronVideos_Pullfocused_continuousBhv/"+\
                        cameraID+"/"+animal1_filenames[0]+"_"+animal2_filenames[0]+"/"+examplesess+"/"

        if not os.path.exists(figsavefolder):
            os.makedirs(figsavefolder)

        if not xxx_type == 'pull_rt':
            if not doExampleNeuron:
                fig.savefig(figsavefolder+'bhvevents_aligned_averaged_FR_acrossNeurons_separate_quantiles_'+bhvname_ana+'_'+
                         pull_trig_events_tgtname+'_'+gaze_duration_type+'_'+cond_toplot_type+savefile_sufix+'.pdf')
            elif doExampleNeuron:
                fig.savefig(figsavefolder+'bhvevents_aligned_averaged_FR_exampleNeurons_separate_quantiles_'+bhvname_ana+'_'+
                         pull_trig_events_tgtname+'_'+gaze_duration_type+'_'+cond_toplot_type+savefile_sufix+'.pdf')
        #
        else:
            if not doExampleNeuron:
                fig.savefig(figsavefolder+'bhvevents_aligned_averaged_FR_acrossNeurons_separate_quantiles_'+bhvname_ana+'_'+
                         quantile_col+'_'+gaze_duration_type+'_'+cond_toplot_type+savefile_sufix+'.pdf')
            elif doExampleNeuron:
                fig.savefig(figsavefolder+'bhvevents_aligned_averaged_FR_exampleNeurons_separate_quantiles_'+bhvname_ana+'_'+
                         quantile_col+'_'+gaze_duration_type+'_'+cond_toplot_type+savefile_sufix+'.pdf')
                

In [ ]:
# Step 2 - calculate the PCA with the pooled data

from sklearn.decomposition import PCA
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import splprep, splev
from mpl_toolkits.mplot3d import Axes3D  # Import for 3D plotting
import scipy.stats as stats
from statsmodels.stats.multicomp import MultiComparison

if 1:
    timewins = np.arange(-4, 4, 1 / 30)
    n_timepoints = len(timewins)
    quantile_col = 'gaze_accum_quantile'  # Or 'gaze_duration_quantile' or 'pull_rt_quantile'
    # quantile_col = 'pull_rt_quantile'  # Or 'gaze_duration_quantile' or 'pull_rt_quantile'
    y_label = 'Firing Rate (Hz)'
    title_prefix = 'gaze_accum_quantile'  # Or 'Gaze Duration' or 'pull_rt_quantile'
    # title_prefix = 'pull_rt_quantile'  # Or 'Gaze Duration' or 'pull_rt_quantile'
    smooth_kernel_size = 6
    n_bootstrap_iterations = 100
    n_neurons_to_sample = 200
    
    # Get unique neurons
    unique_neurons = bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df[['clusterID', 'session']].drop_duplicates()
    n_neurons = len(unique_neurons)

    # Get unique quantiles
    unique_quantiles = np.sort(bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df[quantile_col].unique())
    n_quantiles = len(unique_quantiles)

    # Initialize the data matrix
    data_matrix = np.empty((n_neurons, n_timepoints * n_quantiles))
    neuron_index_lookup = {}

    # Populate the data matrix
    neuron_counter = 0
    for neuron_row in unique_neurons.itertuples(index=False):
        cluster_id = neuron_row.clusterID
        session = neuron_row.session
        neuron_index_lookup[(cluster_id, session)] = neuron_counter
        neuron_counter += 1

        for i_quantile, q_val in enumerate(unique_quantiles):
            # Get the mean FR trace for the current neuron and quantile
            mean_fr_trace_df = bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df[
                (bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df['clusterID'] == cluster_id) &
                (bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df['session'] == session) &
                (bhvevents_aligned_FR_and_eventFeatures_meanFReachQuant_all_dates_df[quantile_col] == q_val)
            ]

            if not mean_fr_trace_df.empty:
                mean_fr_trace = mean_fr_trace_df['mean_FR_trace'].values[0]  # Take the first element
                data_matrix[neuron_index_lookup[(cluster_id, session)], i_quantile * n_timepoints:(i_quantile + 1) * n_timepoints] = mean_fr_trace
            else:
                data_matrix[neuron_index_lookup[(cluster_id, session)], i_quantile * n_timepoints:(i_quantile + 1) * n_timepoints] = np.nan  # Handle missing data

    # Prepare to store results
    all_quantile_lengths = np.zeros((n_bootstrap_iterations, n_quantiles))
    all_quantile_curvatures = np.zeros((n_bootstrap_iterations, n_quantiles))
    all_boot_pca_data = np.zeros((n_bootstrap_iterations, 10, n_timepoints * n_quantiles))  # Store all PCA results

    # Perform Bootstrapping
    for boot_iter in range(n_bootstrap_iterations):
        # 1. Randomly sample neurons
        sampled_neuron_indices = np.random.choice(n_neurons, n_neurons_to_sample, replace=True)
        sampled_data_matrix = data_matrix[sampled_neuron_indices, :].transpose()  # Neuron dimension becomes the columns

        # 2. Run PCA
        pca = PCA(n_components=10)  # Project to 10 PCs
        pca.fit(np.nan_to_num(sampled_data_matrix))
        pca_data = pca.transform(np.nan_to_num(sampled_data_matrix)).transpose()  # Project and transpose
        all_boot_pca_data[boot_iter, :, :] = pca_data

        # Calculate length and curvature for each quantile
        for i_quantile, q_val in enumerate(unique_quantiles):
            start_col = i_quantile * n_timepoints
            end_col = (i_quantile + 1) * n_timepoints
            quantile_data = pca_data[:, start_col:end_col]  # (10, n_timepoints)

            # 
            if gaze_duration_type == 'around_pull':
                # Calculate Length
                length = np.sum(np.sqrt(np.sum(np.diff(quantile_data[:10, :], axis=1)**2, axis=0))) # Use only first 10 PCs
                all_quantile_lengths[boot_iter, i_quantile] = length

                # Calculate Curvature (using spline interpolation)
                t = np.linspace(0, 1, n_timepoints)
                try:
                    spl = splprep(quantile_data[:10, :], s=0)  # Use only first 10 PCs, s=0 for no smoothing
                    spl_deriv2 = splev(t, spl[0], der=2)
                    curvature = np.mean(np.sqrt(spl_deriv2[0]**2 + spl_deriv2[1]**2 + spl_deriv2[2]**2))
                    all_quantile_curvatures[boot_iter, i_quantile] = curvature
                except:
                    all_quantile_curvatures[boot_iter, i_quantile] = np.nan  # Handle spline fitting errors
            #
            elif gaze_duration_type == 'before_pull':
                ind_tgt = timewins<0
                # Calculate Length
                length = np.sum(np.sqrt(np.sum(np.diff(quantile_data[:10, ind_tgt], axis=1)**2, axis=0))) # Use only first 10 PCs
                all_quantile_lengths[boot_iter, i_quantile] = length

                # Calculate Curvature (using spline interpolation)
                t = np.linspace(0, 1, n_timepoints)
                try:
                    spl = splprep(quantile_data[:10, ind_tgt], s=0)  # Use only first 10 PCs, s=0 for no smoothing
                    spl_deriv2 = splev(t, spl[0], der=2)
                    curvature = np.mean(np.sqrt(spl_deriv2[0]**2 + spl_deriv2[1]**2 + spl_deriv2[2]**2))
                    all_quantile_curvatures[boot_iter, i_quantile] = curvature
                except:
                    all_quantile_curvatures[boot_iter, i_quantile] = np.nan  # Handle spline fitting errors
            #
            elif gaze_duration_type == 'after_pull':
                ind_tgt = timewins>0
                # Calculate Length
                length = np.sum(np.sqrt(np.sum(np.diff(quantile_data[:10, ind_tgt], axis=1)**2, axis=0))) # Use only first 10 PCs
                all_quantile_lengths[boot_iter, i_quantile] = length

                # Calculate Curvature (using spline interpolation)
                t = np.linspace(0, 1, n_timepoints)
                try:
                    spl = splprep(quantile_data[:10, ind_tgt], s=0)  # Use only first 10 PCs, s=0 for no smoothing
                    spl_deriv2 = splev(t, spl[0], der=2)
                    curvature = np.mean(np.sqrt(spl_deriv2[0]**2 + spl_deriv2[1]**2 + spl_deriv2[2]**2))
                    all_quantile_curvatures[boot_iter, i_quantile] = curvature
                except:
                    all_quantile_curvatures[boot_iter, i_quantile] = np.nan  # Handle spline fitting errors

                    
    # ---plotting---
    # --- plotting number 1 ---
    # Plot Length and Curvature (as before)
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
    
    # Create a Pandas DataFrame for easier plotting with Seaborn
    df = pd.DataFrame(all_quantile_lengths, columns=[f'Quantile {i+1}' for i in range(num_quantiles)])
    # Melt the DataFrame to long format, which is ideal for Seaborn
    df_melted = pd.melt(df, var_name='Quantile Group', value_name='Length')
    # Create the swamp plot (also known as a violin plot)
    seaborn.swarmplot(ax = axes[0], x='Quantile Group', y='Length', data=df_melted, 
                      hue='Quantile Group', palette='viridis')
    #axes[0].boxplot(all_quantile_lengths)
    # axes[0].set_xticks(np.arange(1, n_quantiles + 1))
    axes[0].set_xlabel('Quantile')
    axes[0].set_ylabel('PC Trajectory Length')
    axes[0].set_title('Bootstrapped PC Trajectory Lengths')

    # Create a Pandas DataFrame for easier plotting with Seaborn
    df = pd.DataFrame(all_quantile_curvatures, columns=[f'Quantile {i+1}' for i in range(num_quantiles)])
    # Melt the DataFrame to long format, which is ideal for Seaborn
    df_melted = pd.melt(df, var_name='Quantile Group', value_name='Length')
    # Create the swamp plot (also known as a violin plot)
    seaborn.swarmplot(ax = axes[1], x='Quantile Group', y='Length', data=df_melted, 
                      hue='Quantile Group', palette='viridis')
    # axes[1].boxplot(all_quantile_curvatures)
    # axes[1].set_xticks(np.arange(1, n_quantiles + 1))
    axes[1].set_xlabel('Quantile')
    axes[1].set_ylabel('PC Trajectory Curvature')
    axes[1].set_title('Bootstrapped PC Trajectory Curvatures')

    # do the regression for each bootstrap iteration and then plot the average regression line
    from sklearn.linear_model import LinearRegression
    
    # Prepare x-axis (quantile indices)
    quantiles = np.arange(0, n_quantiles).reshape(-1, 1)

    # Store predicted regression lines from each bootstrap
    predicted_lengths = []
    predicted_curvatures = []
    reg_coeff_lengths = []
    reg_coeff_curvs = []

    for i in range(n_bootstrap_iterations):
        # Regression for length
        y_len = all_quantile_lengths[i]
        model_len = LinearRegression().fit(quantiles, y_len)
        slope = model_len.coef_
        reg_coeff_lengths.append(slope[0])
        pred_len = model_len.predict(quantiles)
        predicted_lengths.append(pred_len)

        # Regression for curvature
        y_curv = all_quantile_curvatures[i]
        model_curv = LinearRegression().fit(quantiles, y_curv)
        slope = model_curv.coef_
        reg_coeff_curvs.append(slope[0])
        pred_curv = model_curv.predict(quantiles)
        predicted_curvatures.append(pred_curv)

    # Convert to arrays
    predicted_lengths = np.array(predicted_lengths)
    predicted_curvatures = np.array(predicted_curvatures)

    # Mean and 95% CI across bootstraps
    mean_len = np.mean(predicted_lengths, axis=0)
    ci_len_low = np.percentile(predicted_lengths, 2.5, axis=0)
    ci_len_high = np.percentile(predicted_lengths, 97.5, axis=0)

    mean_curv = np.mean(predicted_curvatures, axis=0)
    ci_curv_low = np.percentile(predicted_curvatures, 2.5, axis=0)
    ci_curv_high = np.percentile(predicted_curvatures, 97.5, axis=0)

    # Length
    axes[0].plot(quantiles, mean_len, color='red', label='Mean Regression')
    axes[0].fill_between(quantiles.flatten(), ci_len_low, ci_len_high, color='red', alpha=0.3, label='95% CI')
    axes[0].set_title('PC Trajectory Length')
    axes[0].set_xlabel('Quantile')
    axes[0].set_ylabel('Length')
    axes[0].legend()

    # Curvature
    axes[1].plot(quantiles, mean_curv, color='blue', label='Mean Regression')
    axes[1].fill_between(quantiles.flatten(), ci_curv_low, ci_curv_high, color='blue', alpha=0.3, label='95% CI')
    axes[1].set_title('PC Trajectory Curvature')
    axes[1].set_xlabel('Quantile')
    axes[1].set_ylabel('Curvature')
    axes[1].legend()

    plt.tight_layout()
    plt.show()
    
    t_stat, p_value = stats.ttest_1samp(reg_coeff_lengths, 5)
    # Print the results
    print('slopes of the length regression'+f"T-statistic: {t_stat}"+'; '+f"P-value: {p_value}")
    #
    t_stat, p_value = stats.ttest_1samp(reg_coeff_curvs, 5)
    # Print the results
    print('slopes of the curvature regression'+f"T-statistic: {t_stat}"+'; '+f"P-value: {p_value}")

    # --- plotting number 1 end ---


    
    
    
    # Plot Average 3D Traces
    fig2 = plt.figure(figsize=(10, 8))
    ax = fig2.add_subplot(111, projection='3d')

    mean_traces_3d = np.mean(all_boot_pca_data[:, :3, :], axis=0)  # Average across bootstrap iterations, use only first 3 PCs

    for i_quantile, q_val in enumerate(unique_quantiles):
        start_col = i_quantile * n_timepoints
        end_col = (i_quantile + 1) * n_timepoints
        mean_quantile_data = mean_traces_3d[:, start_col:end_col]

        # Smooth the trajectory
        smooth_x = gaussian_filter1d(mean_quantile_data[0, ind_tgt], sigma=smooth_kernel_size)
        smooth_y = gaussian_filter1d(mean_quantile_data[1, ind_tgt], sigma=smooth_kernel_size)
        smooth_z = gaussian_filter1d(mean_quantile_data[2, ind_tgt], sigma=smooth_kernel_size)

        # Plot the smoothed trajectory
        ax.plot(smooth_x, smooth_y, smooth_z, label=f'Quantile {int(q_val)}')

        # Mark start and end points
        ax.scatter(smooth_x[0], smooth_y[0], smooth_z[0], marker='o', color='k')  # Start
        ax.scatter(smooth_x[-1], smooth_y[-1], smooth_z[-1], marker='x', color='k')  # End

    ax.set_xlabel('PC 1')
    ax.set_ylabel('PC 2')
    ax.set_zlabel('PC 3')
    ax.set_title(f'Average 3D Trajectories in Neuron-Reduced PC Space by {title_prefix} Quantile')
    ax.legend()
    plt.show()

    # --- Statistical Analysis (Paired t-tests with Holm-Bonferroni) ---
    print("\n--- Statistical Analysis (Paired t-tests with Holm-Bonferroni) ---")

    from scipy import stats
    from statsmodels.sandbox.stats.multicomp import multipletests

    def perform_pairwise_paired_ttests(data, group_labels, alpha=0.01):
        unique_groups = np.unique(group_labels)
        n_groups = len(unique_groups)
        p_values = []
        comparisons = []

        for i in range(n_groups):
            for j in range(i + 1, n_groups):
                group1_data = data[:, i]
                group2_data = data[:, j]
                t_stat, p_val = stats.ttest_rel(group1_data, group2_data)
                p_values.append(p_val)
                comparisons.append((unique_groups[i], unique_groups[j]))

        reject, p_corrected, _, _ = multipletests(p_values, method='holm', alpha=alpha)

        results_df = pd.DataFrame({'Comparison': comparisons,
                                   'p_value': p_values,
                                   'p_corrected': p_corrected,
                                   'reject_null': reject})
        return results_df

    # Perform pairwise paired t-tests for Length
    length_pairwise_results = perform_pairwise_paired_ttests(all_quantile_lengths, unique_quantiles)
    print("\nPairwise Paired t-tests for Length:")
    print(length_pairwise_results)

    # Perform pairwise paired t-tests for Curvature
    curvature_pairwise_results = perform_pairwise_paired_ttests(all_quantile_curvatures, unique_quantiles)
    print("\nPairwise Paired t-tests for Curvature:")
    print(curvature_pairwise_results)
    
    
    savefig = 0
    if savefig:
        figsavefolder = data_saved_folder+"fig_for_basic_neural_analysis_allsessions_basicEvents_PCA_makeBhvNeuronVideos_Pullfocused_continuousBhv/"+\
                        cameraID+"/"+animal1_filenames[0]+"_"+animal2_filenames[0]+"/FR_and_gaze_quantile_fig/"

        if not os.path.exists(figsavefolder):
            os.makedirs(figsavefolder)

        fig.savefig(figsavefolder+'bhvevents_aligned_PCfeatures_sparate_quantiles_'+bhvname_ana+'_'+
                     pull_trig_events_tgtname+'_'+gaze_duration_type+'_'+cond_toplot_type+savefile_sufix+'.pdf')
        
        fig2.savefig(figsavefolder+'bhvevents_aligned_PCtrajectory_sparate_quantiles_'+bhvname_ana+'_'+
                     pull_trig_events_tgtname+'_'+gaze_duration_type+'_'+cond_toplot_type+savefile_sufix+'.pdf')
    
    
    

In [ ]:
np.shape(sampled_data_matrix)

In [ ]:
gaze_duration_type